# Workbook to scrape the links Julien has collected

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from io import BytesIO
from typing import Iterable, Optional
from requests import Response, Session
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

try:
    from pdfminer.high_level import extract_text as pdf_extract_text
except ImportError:  # pragma: no cover
    pdf_extract_text = None

In [2]:
xl_path = 'data/julien_gt.xlsx'

In [3]:
df = pd.read_excel(xl_path)

In [11]:
import random
import string
df["unique_id"] = [
    ''.join(random.choices(string.ascii_letters + string.digits, k=16))
    for _ in range(len(df))
]


In [13]:
import openpyxl

wb = openpyxl.load_workbook(xl_path)
sheets = wb.sheetnames
ws = wb[sheets[0]]

# Get column names from first row, standardize for lowercase
column_names = [cell.value.lower() if isinstance(cell.value, str) else cell.value for cell in next(ws.iter_rows(min_row=1, max_row=1))]
print(column_names)

# Names of link columns to process
link_col_names = ["first link", "second link", "third link"]

link_url_dict = {col: [] for col in link_col_names}
hyperlinks = []  # List to store all hyperlinks
hyperlinks_by_col = {col: [] for col in link_col_names}  # To keep per-column counts

for col_name in link_col_names:
    try:
        idx = column_names.index(col_name) + 1  # openpyxl columns are 1-based
    except ValueError:
        print(f"No '{col_name}' column found.")
        idx = None

    if idx:
        for row in ws.iter_rows(min_row=2, min_col=idx, max_col=idx):
            cell = row[0]
            target = cell.hyperlink.target if cell.hyperlink else None
            link_url_dict[col_name].append(target)
            if target:
                hyperlinks.append(target)
                hyperlinks_by_col[col_name].append(target)
    else:
        link_url_dict[col_name] = [None] * len(df)

wb.close()

# Assign to standardized df columns
for col_name in link_col_names:
    df[col_name.replace(" ", "_") + "_url"] = link_url_dict[col_name]

# Print some checks
for col_name in link_col_names:
    print(f"\nSample {col_name}_url:", df[col_name.replace(' ', '_') + '_url'].head().tolist())
    print(f"Number of hyperlinks from '{col_name}': {len(hyperlinks_by_col[col_name])}")

print(f"\nTotal {len(hyperlinks)} hyperlinks collected. Sample:", hyperlinks[:5])


['name', 'first link', 'second link', 'third link', 'source', 'provincial', 'duplicate', 'plan or strategy', 'climate finance?', None, 'criterion 1', 'criterion 2', 'criterion 3', 'criterion 4', 'mit = real economy mitigation; adp = real economy adaptation; fsp = financial sector policy; icf = international climate finance']

Sample first link_url: ['https://www.canada.ca/content/dam/eccc/documents/pdf/climate-change/ghg-offset/indigenous-toolkit/Planning-a-Project-in-GHG-Offset-System-EN.pdf', None, None, 'https://laws-lois.justice.gc.ca/eng/regulations/SOR-2019-266/index.html', None]
Number of hyperlinks from 'first link': 161

Sample second link_url: [None, None, None, None, None]
Number of hyperlinks from 'second link': 13

Sample third link_url: [None, None, None, None, None]
Number of hyperlinks from 'third link': 10

Total 184 hyperlinks collected. Sample: ['https://www.canada.ca/content/dam/eccc/documents/pdf/climate-change/ghg-offset/indigenous-toolkit/Planning-a-Project-in-GH

In [14]:
def build_scraping_session(
    total_retries: int = 3,
    backoff_factor: float = 0.5,
    status_forcelist: tuple[int, ...] = (429, 500, 502, 503, 504),
) -> Session:
    """Create a requests session with sensible defaults for retries."""
    session = requests.Session()
    retry = Retry(
        total=total_retries,
        read=total_retries,
        connect=total_retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
        allowed_methods=("GET", "HEAD"),
        raise_on_status=False,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    session.headers.update(
        {
            "User-Agent": (
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                "AppleWebKit/537.36 (KHTML, like Gecko) "
                "Chrome/119.0.0.0 Safari/537.36"
            )
        }
    )
    return session

In [15]:
def _is_pdf_response(response: Response, url: str) -> bool:
    content_type = response.headers.get("content-type", "").lower()
    return "pdf" in content_type or url.lower().split("?")[0].endswith(".pdf")

In [16]:
def _extract_text_from_pdf(content: bytes) -> str:
    """
    Extract text from PDF bytes using pdfminer.six.
    """
    try:
        from pdfminer.high_level import extract_text as pdfminer_extract_text
    except ImportError:
        raise ImportError(
            "pdfminer.six is required for PDF extraction. Install with `pip install pdfminer.six`."
        )
    return pdfminer_extract_text(BytesIO(content))

In [17]:
def _extract_text_from_html(response: Response) -> str:
    response.encoding = response.encoding or response.apparent_encoding
    soup = BeautifulSoup(response.text, "html.parser")
    return soup.get_text("\n", strip=True)

In [18]:
def scrape_url(
    url: str,
    session: Optional[Session] = None,
    timeout: int = 30,
) -> dict:
    """
    Fetch a single URL and return a dict with metadata and extracted text.

    Returns
    -------
    dict with keys: url, status, content_type, content, error (optional)
    """
    sess = session or build_scraping_session()
    created_session = session is None
    try:
        response = sess.get(url, timeout=timeout)
        response.raise_for_status()
    except requests.RequestException as exc:  # network or HTTP errors
        if created_session:
            sess.close()
        return {
            "url": url,
            "status": "error",
            "content_type": None,
            "content": "",
            "char_count": 0,
            "error": str(exc),
        }

    try:
        if _is_pdf_response(response, url):
            content = _extract_text_from_pdf(response.content)
            content_type = "pdf"
        else:
            content = _extract_text_from_html(response)
            content_type = "html"
        result = {
            "url": url,
            "status": "success",
            "content_type": content_type,
            "content": content,
            "char_count": len(content),
        }
    except Exception as exc:
        result = {
            "url": url,
            "status": "error",
            "content_type": None,
            "content": "",
            "char_count": 0,
            "error": f"Failed to parse content: {exc}",
        }
    finally:
        if created_session:
            sess.close()
    return result

In [19]:
def scrape_links(
    urls: Iterable[str],
    session: Optional[Session] = None,
    timeout: int = 30,
    verbose: bool = False,
) -> pd.DataFrame:
    """Scrape a collection of URLs and return a dataframe of results."""
    sess = session or build_scraping_session()
    created_session = session is None
    records = []
    for idx, url in enumerate(urls, start=1):
        if verbose:
            print(f"[{idx}] Fetching {url}")
        record = scrape_url(url, session=sess, timeout=timeout)
        if verbose:
            if record["status"] == "success":
                print(f"[{idx}] Success ({record['char_count']:,} chars)")
            else:
                error_msg = record.get("error", "unknown error")
                print(f"[{idx}] Failed: {error_msg}")
        records.append(record)
    if created_session:
        sess.close()
    return pd.DataFrame.from_records(records)

In [23]:
%%time

results_df = scrape_links(hyperlinks, verbose=True)

success_mask = results_df["status"] == "success"
total = len(results_df)
successes = success_mask.sum()
errors = total - successes

print(f"Scrape summary: {successes}/{total} succeeded, {errors} failed.")
print("Character count stats (successes only):")
if successes:
    print(results_df.loc[success_mask, "char_count"].describe())
else:
    print("No successful scrapes to summarize.")

low_char = results_df.loc[success_mask & (results_df["char_count"] < 1000), ["url", "char_count"]]
if not low_char.empty:
    print("\nPotentially incomplete (char_count < 1000):")
    display(low_char)

results_df.head()


[1] Fetching https://www.canada.ca/content/dam/eccc/documents/pdf/climate-change/ghg-offset/indigenous-toolkit/Planning-a-Project-in-GHG-Offset-System-EN.pdf
[1] Success (10,556 chars)
[2] Fetching https://laws-lois.justice.gc.ca/eng/regulations/SOR-2019-266/index.html
[2] Success (3,718 chars)
[3] Fetching https://www.canada.ca/en/environment-climate-change/services/climate-change/low-carbon-economy-fund/challenge/applicant-guide-2023.html
[3] Success (118,982 chars)
[4] Fetching https://www.canada.ca/en/environment-climate-change/services/climate-change/low-carbon-economy-fund/indigenous-leadership.html
[4] Success (8,872 chars)
[5] Fetching https://www.canada.ca/en/environment-climate-change/services/climate-change/low-carbon-economy-fund/implementation-readiness.html
[5] Success (9,331 chars)
[6] Fetching https://www.canada.ca/en/environment-climate-change/services/climate-change/low-carbon-economy-fund/low-carbon-leadership.html
[6] Success (3,026 chars)
[7] Fetching https://www.b

Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value


[16] Success (468,348 chars)
[17] Fetching https://prod-natural-resources.azure.cloud.nrcan-rncan.gc.ca/energy-efficiency/home-energy-efficiency/canada-greener-homes-initiative/canada-greener-homes-loan
[17] Success (14,016 chars)
[18] Fetching https://prod-natural-resources.azure.cloud.nrcan-rncan.gc.ca/climate-change/sreps
[18] Success (8,113 chars)
[19] Fetching https://www.canada.ca/en/environment-climate-change/services/environmental-funding/programs/nature-smart-climate-solutions-fund.html
[19] Success (6,970 chars)
[20] Fetching https://agriculture.canada.ca/en/programs/agricultural-climate-solutions-farm-climate-action-fund
[20] Success (21,844 chars)
[21] Fetching https://natural-resources.canada.ca/corporate/planning-reporting/summary-interdepartmental-evaluation-green-municipal-fund
[21] Success (8,641 chars)
[22] Fetching https://www.canada.ca/en/environment-climate-change/services/managing-pollution/energy-production/fuel-regulations/clean-fuel-regulations/about.html
[22] 

Cannot set gray stroke color because /'P0' is an invalid float value


[38] Success (147,537 chars)
[39] Fetching https://natural-resources.canada.ca/funding-partnerships/green-construction-through-wood-gcwood-program
[39] Success (8,571 chars)
[40] Fetching https://laws-lois.justice.gc.ca/eng/regulations/SOR-2010-201/index.html
[40] Success (3,892 chars)
[41] Fetching https://www.canada.ca/en/environment-climate-change/news/2023/12/canadas-electric-vehicle-availability-standard-regulated-targets-for-zero-emission-vehicles.html
[41] Success (23,077 chars)
[42] Fetching https://assets.cmhc-schl.gc.ca/sites/cmhc/nhs/co-investment-fund/nhs-co-invest-fund-highlight-sheet-en.pdf
[42] Success (4,624 chars)
[43] Fetching https://tc.canada.ca/en/road-transportation/innovative-technologies/zero-emission-vehicles/incentives-zero-emission-vehicles-izev
[43] Success (4,473 chars)
[44] Fetching https://natural-resources.canada.ca/sites/nrcan/files/energy/images/publications/2022/A%20CLEAN%20POWER%20ROADMAP%20FOR%20ATLANTIC%20CANADA-ACC.pdf


Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P1' is an invalid float value
Cannot set gray stroke color because /'P2' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P1' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value
Cannot set gray stroke color becau

[44] Success (44,757 chars)
[45] Fetching https://ceefc-cfuec.ca/leeff-factsheet/
[45] Success (7,089 chars)
[46] Fetching https://www.canada.ca/en/department-finance/programs/financial-sector-policy/securities/debt-program/canadas-green-bond-framework.html
[46] Success (56,999 chars)
[47] Fetching https://agriculture.canada.ca/en/programs/agricultural-climate-solutions-farm-climate-action-fund
[47] Success (21,844 chars)
[48] Fetching https://www.infrastructure.gc.ca/dmaf-faac/index-eng
[48] Success (5,648 chars)
[49] Fetching https://natural-resources.canada.ca/funding-partnerships/clean-energy-rural-remote-communities-program
[49] Success (8,185 chars)
[50] Fetching https://www.canada.ca/en/indigenous-services-canada/news/2018/03/historic-indigenous-led-transmission-project-to-connect-16-remote-first-nations-communities-to-provincial-power-grid.html
[50] Success (9,826 chars)
[51] Fetching https://www.canada.ca/en/services/environment/weather/climatechange/climate-plan/oil-gas-emiss

Cannot set gray non-stroke color because /'P89' is an invalid float value
Cannot set gray non-stroke color because /'P91' is an invalid float value
Cannot set gray non-stroke color because /'P93' is an invalid float value
Cannot set gray non-stroke color because /'P95' is an invalid float value


[73] Success (17,308 chars)
[74] Fetching https://natural-resources.canada.ca/energy-efficiency/transportation-energy-efficiency/green-freight/green-freight-program/green-freight-program-applicant-s-guide-stream-1-assess-retrofit
[74] Success (3,490 chars)
[75] Fetching https://natural-resources.canada.ca/energy-efficiency/transportation-energy-efficiency/green-freight/green-freight-program/gfp-stream-2-fleet-modernization
[75] Success (2,715 chars)
[76] Fetching https://tc.canada.ca/en/programs/green-shipping-corridor-program
[76] Success (3,830 chars)
[77] Fetching https://housing-infrastructure.canada.ca/zero-emissions-trans-zero-emissions/index-eng.html
[77] Success (10,888 chars)
[78] Fetching https://publications.gc.ca/collections/collection_2025/licc-hicc/T94-46-2024-eng.pdf
[78] Failed: HTTPSConnectionPool(host='publications.gc.ca', port=443): Max retries exceeded with url: /collections/collection_2025/licc-hicc/T94-46-2024-eng.pdf (Caused by SSLError(SSLCertVerificationError(1

,url,status,content_type,content,char_count,error
0,https://www.canada.ca/content/dam/eccc/documen...,success,pdf,The Essentials:\nPlanning a Project in \nCanad...,10556,NaN
1,https://laws-lois.justice.gc.ca/eng/regulation...,success,html,Output-Based Pricing System Regulations\nSkip ...,3718,NaN
2,https://www.canada.ca/en/environment-climate-c...,success,html,Low Carbon Economy Challenge Applicant Guide 2...,118982,NaN
3,https://www.canada.ca/en/environment-climate-c...,success,html,Indigenous Leadership Fund - Canada.ca\nSkip t...,8872,NaN
4,https://www.canada.ca/en/environment-climate-c...,success,html,Implementation Readiness Fund - Canada.ca\nSki...,9331,NaN


In [24]:
# We want to match the URLs in results_df to the unique_id fields from df (which could be named e.g., original_df).
# The link from results_df can appear in any of first_link_url, second_link_url, third_link_url in df.
# Let's assume 'df' is already defined with columns: ["unique_id", "first_link_url", "second_link_url", "third_link_url", ...]

# Create a url -> unique_id mapping from the source df for all 3 possible url columns
url_id_map = {}
for i, row in df.iterrows():
    for linkcol in ["first_link_url", "second_link_url", "third_link_url"]:
        url = row.get(linkcol, None)
        if pd.notnull(url):
            url_id_map[url] = row["unique_id"]

# Now map the unique_ids into results_df, using their "url" column to look up in url_id_map
results_df["matched_unique_id"] = results_df["url"].map(url_id_map)

# If you want to see if there were any that did not match, you can check with:
unmatched_rows = results_df[results_df["matched_unique_id"].isnull()]
print(f"{len(unmatched_rows)} of {len(results_df)} URLs did not match a unique_id from df.")

# If you want to position "matched_unique_id" column first:
cols = ["matched_unique_id"] + [c for c in results_df.columns if c != "matched_unique_id"]
results_df = results_df[cols]


0 of 184 URLs did not match a unique_id from df.


### healing those that did not scrape properly

In [25]:
error_df = results_df[results_df["status"] == "error"]

In [26]:
for url in error_df.url:
    print(url)

https://publications.gc.ca/site/eng/9.935408/publication.html
https://publications.gc.ca/collections/collection_2025/licc-hicc/T94-46-2024-eng.pdf
https://www.imf.org/en/-/media/files/publications/pp/2023/english/ppea2023003.pdf
https://www.afdb.org/en/topics-and-sectors/initiatives-and-partnerships/canada-african-development-bank-climate-fund
https://www.adb.org/what-we-do/funds/canadian-climate-fund-for-the-private-sector-in-asia-2
https://www.imf.org/en/about/faq/resilience-and-sustainability-trust
https://www.adb.org/what-we-do/funds/canadian-climate-fund-for-the-private-sector-in-asia


In [33]:
error_df

,matched_unique_id,url,status,content_type,content,char_count,error
31,JAYE9ES5gamFNfMr,https://publications.gc.ca/site/eng/9.935408/p...,error,None,,0,"HTTPSConnectionPool(host='publications.gc.ca',..."
77,axxzOXaUnYi9W4yB,https://publications.gc.ca/collections/collect...,error,None,,0,"HTTPSConnectionPool(host='publications.gc.ca',..."
102,3AHfTSNelYuhb8eq,https://www.imf.org/en/-/media/files/publicati...,error,None,,0,"HTTPSConnectionPool(host='www.imf.org', port=4..."
103,vswV5GcFju9btwKa,https://www.afdb.org/en/topics-and-sectors/ini...,error,None,,0,403 Client Error: Forbidden for url: https://w...
167,8Ui54HFehYuBl7ur,https://www.adb.org/what-we-do/funds/canadian-...,error,None,,0,403 Client Error: Forbidden for url: https://w...
169,3AHfTSNelYuhb8eq,https://www.imf.org/en/about/faq/resilience-an...,error,None,,0,"HTTPSConnectionPool(host='www.imf.org', port=4..."
180,8Ui54HFehYuBl7ur,https://www.adb.org/what-we-do/funds/canadian-...,error,None,,0,403 Client Error: Forbidden for url: https://w...


In [34]:
JAYE9ES5gamFNfMr_text = '''Passer au contenu principalPasser à « À propos de ce site »Switch to basic HTML version
Language selection
Français
Government of Canada/ Gouvernement du Canada
Search
Search Canada.ca
Search Canada.ca
MenuMainMenu 
You are here:
Canada.caAbout governmentGovernment communicationsGovernment of Canada PublicationsBrowse publications
We’re Live! Our newly enhanced website is now live, complete with improved search functionality to help you find what you need faster and more easily.

If you need assistance using the updated search, please visit our Search help page for tips and guidance.

Strategic Innovation Fund's net zero accelerator initiative - Innovation, Science and Economic Development Canada : independent auditor's report.: FA1-26/2024-1-4E-PDF
"This audit focused on whether Innovation, Science and Economic Development Canada managed the Strategic Innovation Fund's Net Zero Accelerator to decarbonize the manufacturing industries in accordance with Canada's climate goals and with due regard to value for money for Canadians"--Focus of the audit, page 3.

Permanent link to this Catalogue record:
publications.gc.ca/pub?id=9.935408&sl=0

MARC XML formatMARC HTML format
Publication information
Department/Agency	
Canada. Commissioner of the Environment and Sustainable Development, author, issuing body.
Title	Strategic Innovation Fund's net zero accelerator initiative - Innovation, Science and Economic Development Canada : independent auditor's report.
Series title	
Reports of the Commissioner of the Environment and Sustainable Development to the Parliament of Canada, 2561-1801 ; report 4
Publication type	Monograph - View Master Record
Language	[English]
Other language editions	[French]
Format	Digital text
Electronic document	
View FA1-26-2024-1-4-eng.pdf (PDF, 933 KB).
View parl_cesd_202404_04_e_44471.html (HTML, N/A).
Note(s)	
Cover title.
Issued also in French under title: L'initiative accélérateur net zéro du Fonds stratégique pour l'innovation - Innovation, Sciences et Développement économique Canada : rapport de l'auditeur indépendant.
Publishing information	
[Ottawa] : Office of the Auditor General of Canada = Bureau du vérificateur général du Canada, 2024.
©2024
Description	1 online resource (iv, 30 pages) : illustrations.
ISBN	9780660708034
Catalogue number	
FA1-26/2024-1-4E-PDF
Subject terms	
Canada. Innovation, Science and Economic Development Canada -- Auditing.
Technological innovations -- Canada -- Finance -- Auditing.
Greenhouse gas mitigation -- Canada -- Finance -- Auditing.
Request alternate formats
To request an alternate format of a publication, complete the Government of Canada Publications email form. Use the form’s “question or comment” field to specify the requested publication.
Page details
Date modified: 2025-11-04
About this site
Government of Canada
All contacts
Departments and agencies
About government
Themes and topics
Jobs
Immigration and citizenship
Travel and tourism
Business
Benefits
Health
Taxes
Environment and natural resources
National security and defence
Culture, history and sport
Policing, justice and emergencies
Transport and infrastructure
Canada and the world
Money and finance
Science and innovation
Indigenous peoples
Veterans and military
Youth
Government of Canada CorporateSocial mediaMobile applicationsAbout Canada.caTerms and conditionsPrivacy
Symbol of the Government of Canada'''

In [35]:
axxzOXaUnYi9W4yB_text = ''' 
1
Aussi disponible en français sous le titre : Guide étape par étape pour les demandes, volet
Immobilisations, Fonds pour les solutions de transport en commun en milieu rural - Édition révisée, 2024.
Information contained in this publication or product may be reproduced, in part or in whole, and by any
means, for personal or public non-commercial purposes without charge or further permission, unless
otherwise specified. Commercial reproduction and distribution are prohibited except with written
permission from Infrastructure Canada.
For more information, contact:
Housing, Infrastructure and Communities Canada
180 Kent Street, Suite 1100
Ottawa, Ontario K1P 0B6
info@infc.gc.ca
© His Majesty the King in Right of Canada, as represented by the Minister of Housing, Infrastructure and
Communities, 2024.
Cat. No. No. T94-46/2024E-PDF
ISBN 978-0-660-47282-3
2
Table of Contents
How to Submit an Application to the Rural Transit Solutions Fund ............................................................ 4
Step One: Register for an HICC Applicant Account....................................................................................4
Create your organization’s profile.........................................................................................................5
Have you previously applied to Housing, Infrastructure and Communities Canada program? .........5
How do I edit my organization’s profile?...........................................................................................5
Are you a New Applicant to a Housing, Infrastructure and Communities Canada program?............5
Step Two: Managing your HICC Applicant Account...................................................................................6
Managing Permissions – Account Administrator/Owner ......................................................................6
My Dashboard .......................................................................................................................................7
Apply for Funding ..................................................................................................................................7
Help .......................................................................................................................................................7
Applicant Guides ...................................................................................................................................7
Step Three: Apply for Funding...................................................................................................................8
Step-by-Step Instructions for Capital Project Applications ......................................................................... 8
Important tips when preparing your application ......................................................................................8
Section 1 – Privacy And Eligibility ............................................................................................................. 9
Section 2 – Applicant Identification ........................................................................................................ 10
Lead Applicant.........................................................................................................................................10
Lead Applicant Contacts..........................................................................................................................10
Project Details.........................................................................................................................................11
Project Location.......................................................................................................................................11
Recipient Capacity Assessment...............................................................................................................12
Section 3 – Project Details ...................................................................................................................... 15
Project Rationale .....................................................................................................................................15
Asset Type ...............................................................................................................................................18
Rolling Stock and Other Capital Assets....................................................................................................21
Accessibility Standard..............................................................................................................................21
Climate Considerations ...........................................................................................................................22
Expected Outcomes ................................................................................................................................23
Section 4 – Environmental Assessment .................................................................................................. 24
3
Section 5 - Project Finances and Supplementary Information ................................................................. 24
Project Cash Flow Table ..........................................................................................................................26
Estimated Project Schedule.....................................................................................................................29
Section 6 - Project Risk ........................................................................................................................... 30
Section 7 - Required Documents ............................................................................................................ 31
Section 8 - Attestation ........................................................................................................................... 32
Validation Status .....................................................................................................................................32
Privacy and Eligibility ...............................................................................................................................32
Annex A: Project Activities List and Guidance for Engagement with Indigenous Peoples.......................... 33
Annex B: Duty to Consult with Indigenous Peoples and Impact Assessment Act Requirements ................ 35
Annex C: How to Create a KML File ........................................................................................................ 47
 
4
How to Submit an Application to the Rural
Transit Solutions Fund
• Before you begin your application, please ensure that your organization is eligible for funding
under the Rural Transit Solutions Fund in accordance with the eligible recipients as outlined in the
“Who Can Apply” section of the Application Guide for the Capital Projects Stream of the Rural
Transit Solutions Fund (available on the website and via the applicant portal). Housing,
Infrastructure and Communities Canada (HICC) will also validate your organization’s eligibility
when your application has been submitted. You should also review the requirements for
supporting documentation that will need to be attached in the HICC Applicant Portal as outlined
in the “Supporting Documents” section of the Application Guide.
• Applicants must register for an HICC Applicant Portal to access the Rural Transit Solutions Fund
application form. By accessing your account through the HICC Applicant Portal, you will be able to
create, save, edit, and submit applications to the Rural Transit Solutions Fund, as well as view
your organization’s list of submissions through “My Dashboard”.
• Applicants who are unable to apply through the HICC Applicant Portal are asked to reach out to
the Rural Transit Solutions Fund Team at rtsf-fstcr@infc.gc.ca.
• Applicants who have an existing HICC Applicant Account can skip to Step Two.
Step One: Register for an HICC Applicant Account
• Enter the HICC Applicant Portal. Please use the most recent version of the following supported
web browsers:
o Microsoft Edge
o Apple Safari
o Google Chrome
o Mozilla Firefox
o Internet Explorer (not recommended)
• Click New User
• Confirm you have read the Privacy Notice Statement
• Register your account by:
o Providing a valid e-mail address
o Creating a username and password
5
o Activate your account by logging out after saving and returning to the sign in page.
(Please note that you may get an error message if you do not complete this step).
Passwords must contain characters from at least three of the following four classes:
uppercase, lowercase, digit, and non-alphanumeric.
Create your organization’s profile
Have you previously applied to Housing, Infrastructure and Communities Canada program?
Applicants who have previously applied to an HICC program should have an existing profile created using
information provided in previous applications. You can find your profile by searching the Existing
Organization Name list. Once you find your organization’s profile, review the information provided and
ensure that it is up to date. Please edit your organization’s profile if necessary. If your organization is not
found, you can proceed to create a New Organization profile.
How do I edit my organization’s profile?
You can edit your organization’s profile once you have made an HICC Applicant Portal Account by clicking
on Profile.
1. Once you enter your Profile page, click on Manage Organizations. Here, you will be able to locate
your organization.
2. Edit as needed and click Save.
Are you a New Applicant to a Housing, Infrastructure and Communities Canada program?
1. If you are a new applicant, create your organization’s profile by clicking New Organization.
2. Fill in the required fields including:
o The organization’s full legal name
o Contact information
o Organization address
o Organization type
3. Click Submit.
4. Once you have created your organization’s profile, find your organization from the Existing
Organization Name list.
5. Enter the name and contact information for your organization.
6. Click Save.
7. Sign out by clicking on your name in the top right corner, selecting Sign out and sign back into
your HICC Applicant Account. (Please note that you may get an error message if you do not
complete this step).
6
Step Two: Managing your HICC Applicant Account
With your HICC Applicant Account, you can apply to the Rural Transit Solutions Fund. The HICC Applicant
Portal is the place to access your organization’s dashboard, application forms and online resources.
Managing Permissions – Account Administrator/Owner
If you are the first person in your organization to register for an HICC Applicant Portal account, you will
automatically be the primary contact for your organization’s account and be assigned the role of Account
Administrator/Owner.
As Account Administrator/Owner, you will be responsible for:
• Reviewing and approving (or declining) requests from other members of your organization to
access the HICC Applicant Portal and the Rural Transit Solutions Fund application form.
• Determining whether an approved team member(s) has “read only” capabilities or “update”
(populating an application) capabilities.
• Determining whether an approved team member(s) can Attest to and/or Submit project
applications.
• Determining whether an approved team member(s) can be your “delegate”, which permits them
to have the same account privileges as you and undertake the above-noted actions on your
behalf (as the team Account Administrator/Owner).
How to grant access to your application for other members of your organization
For other members of your organization to access the HICC Applicant Portal and work on the Rural Transit
Solutions Fund Application Form, you must grant them permission through your Profile page. It is
only after you have made your HICC Applicant Portal Account that other members of your team can
proceed to make their own HICC Applicant Portal accounts and start an application.
First, have your team members create their own HICC Applicant Portal Accounts. When asked to identify
the Lead Applicant Organization, ensure that they “link” themselves to your organization by selecting it
from the Existing Organization List. It is only through this process that they will be linked to your
organization and be able to access your organization’s Rural Transit Solutions Fund Application. Once your
team members have created their accounts, you will receive an e-mail notification titled: “Your action
required”.
Next, go to your Profile page. Click on Manage Permissions. Here, you will see a list of individuals who are
requesting access to the HICC Applicant Portal.
1. Click the drop-down menu located next to their name and account information.
2. Click Edit.
3. Answer the questions as prompted:
o Enable Portal Access
o Portal contact role
7
▪ Update
▪ Enable updating of application contents
▪ No ability to modify the organization’s contact and account information
▪ No ability to attest and/or submit the Application Form
▪ Read
▪ Read-only mode
o Status
o Can Attest
o Can Submit
o Is Delegate
Once you have assigned roles to your team members and saved this information, they will be able to see
your Rural Transit Solutions Fund application form in their own accounts under My Dashboard. Any
changes that are made and saved to your application form by your team will be visible to all accounts
linked to the application form.
My Dashboard
Your dashboard will list your ongoing and completed application forms at HICC. Use the dashboard to
navigate to your application form(s) and keep track of the progress and status of your application(s). You
do not need to complete your online application in one attempt. The application can be saved and edited
as many times as necessary.
Apply for Funding
To apply, you will find a link to the application form on the Applicant Portal. There is no limit on the
number of applications that can be submitted by an eligible applicant. Each project can be submitted as
its own application. Alternatively, projects can be bundled into a single application.
An application, regardless of bundling, must describe how the project(s) will support the objectives of the
Rural Transit Solutions Fund and satisfy the merit criteria of the program.
Help
While using the Applicant Portal, applicants will have access to support services at any stage of the
application process. To contact support services, click on “HICC support” at the top of any page within the
Applicant Portal. If, at any time, your account is inaccessible, please email the support team
at crmsupport-soutiengrc@HICC.gc.ca.
Applicant Guides
You will find links to the Application Guide for the Capital Projects Stream of Rural Transit Solutions Fund
and other applicant resources in this section.
8
Step Three: Apply for Funding
After creating and confirming your organization account, to access the Rural Transit Solutions Fund
Application Form:
• Click Apply for Funding
• Click Apply for the Canada Public Transit Funds (CPTF)
• Click Apply for the Rural Transit Solutions Fund
• Click Apply for a Capital Project Submission
• Confirm the “Privacy and Eligibility” Statements
o Read the Privacy and Third-Party Notice Statement and confirm that you have read the
statement by selecting “Yes”.
o Read the Organization Eligibility Self-Assessment and confirm that your organization
meets all the eligibility requirements for funding under the Rural Transit Solutions Fund
and that you are duly authorized to submit this application on behalf of the Organization
by selecting “Yes”. Refer to the Rural Transit Solutions Fund Applicant Guide for details
on organization eligibility.
Step-by-Step Instructions for Capital Project
Applications
Important tips when preparing your application
• All applicants are encouraged to prepare their application in an external document and then copy
and paste their answers into the fields within the application.
• To edit or delete information that is displayed in table format, click on the icon to either edit or
delete.
• Unless otherwise noted, the maximum character count for questions requiring a narrative answer
(text box) is 2,000 characters.
• Frequently, click “Save” to save your answers. This allows you to pause and resume work on your
application without losing any information.
• Questions marked with a red asterisk (*) are required and will need to be completed to validate
and submit your application.
• Once you have completed all questions in a section, click Validate. In addition to saving your
work, this will verify that you have submitted all the required information, and the tab at the top
of the page will turn green.
• All sections must be validated in order to submit your application to HICC.
9
Section 1 – Privacy And Eligibility
Eligible organizations that have a minimum of two years of transit experience, including local
governments working with a partner who has transit experience, and/or have completed a public transit
feasibility study after January 1st, 2020 can apply for a Rural Transit Solution Capital Projects stream
contribution of up to $10 million. Eligible organizations with no prior transit experience and that have not
completed a public transit study can apply for a contribution of up to $250,000.
Capital projects are expected to be completed by 2029-2030.
Does your organization have a
minimum of two years of
experience managing or
implementing a transit
service?*
The dropdown menu will provide you the following options and you
will be able to select only one:
Yes – This option is also applicable if you are a local government
partnering with a transit organization with over two years of
experience.
No – However, the organization has completed a recent public
transit feasibility study (IMPORTANT: The feasibility study must be
submitted with the application - see Guidelines for more details)
No – The organization is applying for less than $250,000 in federal
contribution
Lead Applicant

The name of your organization which was created as part of your
applicant profile should be prefilled here.
Eligible Organization Type* The dropdown menu will provide you the following options, and you
will be able to select only one:
o Province
o Territory
o Municipality
o Local or Regional Government
o Public Sector body
o Indigenous governing body
o Indigenous Development Corporation
o Not for profit mandated to improve Indigenous
outcomes/serve Indigenous communities
o Registered not-for-profit organization
Select the population size that
best describes the area served
by the proposed transit
service.*
Please select the option that best describes the community being
served: 
10
Section 2 – Applicant Identification
To prevent losing your work, please click “Save and Continue” in each section or click on “Save All”. Use
“Validate” when you have completed all questions in this section to check that all mandatory sections are
complete.
Lead Applicant
The Lead Applicant* will be prefilled from your Account Profile. When you have selected your
organization, press “Save and Continue”.
Lead Applicant Contacts
The system will ask for a Primary Contact* and a Secondary Contact. The contact names of individuals
that you entered in your profile will appear in the drop-down menu. Once you have made your selections,
press “Save and Continue”.
• Proposed transit service is primarily serving a community or
communities with a population of less than 50,000
• Proposed transit service is primarily serving a community or
communities with a population of less than 50,000 within a
Census Metropolitan Area (CMA), AND the proposed project
offers transit targeting rural areas and communities.
• Proposed transit service is primarily serving a community or
communities with a population between 50,000 and
150,000, AND the proposed project connects rural areas
• Proposed transit service is primarily serving rural
communities located outside census agglomeration (CAs)
and/or CMAs
• Proposed transit service is primarily serving a community or
communities, located within a CMA, with a population of
more than 150,000
For information on Census Metropolitan Areas (CMAs), please
consult this map: https://www12.statcan.gc.ca/censusrecensement/2021/geo/maps-cartes/referencemapscartesdereference/sgc-cgt/files-fichiers/2021-12572-01-B.pdf
Project Title* In this section you will be asked to provide, in 300 characters or less,
a concise but meaningful project title and the location of the project
(e.g., Purchase of a zero-emission vehicle to provide on-demand
transportation service in [community name, province, territory]).
11
Project Details
Project Title

You should see the project name you previously entered
prefilled.
Are you partnering with other
organization(s) to deliver the
project?*
Please select “Yes” or “No”
If you selected “Yes”, there will be an option to add partner
organization details.
Partner Organizations;
Organization Type; Address Line 1;
Role of the Partner. (If Yes is
selected above).
The name of the organization should be the legal name. In terms
of describing the role of partners, please provide a brief
description of the mandate of the partner organization(s) along
with their roles and responsibilities in relation to the delivery of
the proposed project. Note: Paid contractors and professional
service providers should not be identified in this section.
Press “+ Add Partner Organizations”, and you may add other
partner organizations. Remember to click on Disk to save each
partner and press “Save and Continue” when you are finished.
Project Location
Please specify the communities
(municipality(ies)/region(s)/town(s)
that will be served by the transit
service (generally where most of
the riders are located.)*

Select from the list (If the location does not exist, please explain
in the additional information text box located in the next
section)
Click“+ Add Project Location” and select the “Province or
Territory” where the project is located. Also, select
“City/Municipality” where the project is located. A minimum of
one must be selected to submit.
You may add multiple project locations. Click “+ Add Project
Location” to add additional locations and click on disk to save the
entry.
Remember to press “Save and Continue” when you are finished.
Additional information You may specify the name of the location in this text box if it is
not listed among the options in the drop-down menu. This
section can also include more on routings for your project (e.g.,
Community A to Community B, and so on) and articulate the
location of the transit hub.
Name, Name of the Document
+Add KML Asset Map.*

A KML Asset map is required to submit an application. For more
information on how to create a KML Asset map, please see
Annex C of this document.
• Select “Add” and name and attach at least one KML file
that identifies the project’s asset(s) location(s). The file 
12
should show the project location and footprint of project
assets. If you upload more than one KML file, please
ensure the name of each file includes the project name
and specific details about the file you are attaching (e.g.,
“KML-charging station-Province of XX”).
• The KML file will be used for the Duty to Consult with
Indigenous peoples and Impact Assessment
Act (IAA) requirements.
• If your project only involves the acquisition of rolling
stock, your KML could represent the area that will be
covered by the transit solution.
Is the project within the boundaries
of a Modern Treaty or selfgovernment agreement?*
Please select “Yes” or “No.” Additional information on
Indigenous rights is available on the Crown-Indigenous Relations
and Northern Affairs Canada website.
If yes, please indicate which
Modern Treaty or self-government
agreement?
Please fill in the text box, as applicable.
Please indicate if there are specific
obligations that relate to the
project in the treaty or agreement.
Please fill in the text box, as applicable.
When you are done completing this section, please select “Save
and Continue”.
To prevent losing your work, please click “Save and Continue” in each section or click on “Save All”. Use
“Validate” when you have completed all questions in this section to check that all mandatory sections are
complete.
Recipient Capacity Assessment
The following questions are about your capacity to manage the requested funding. Note that more
information may be requested at the agreement negotiation stage.
Organization Capacity
The following questions relate to the management and practices of the applicant’s organization.
Does the organization have a
functioning governance board or
structure?
A functioning board entails an existing board, or equivalent
governance group, with a required number of people to operate,
for which regular meetings are held, and accountable for the
functioning and governance of the organization.
It’s understood that not all organizations function by way of a
board. Applicants who have a formal, functioning management
structure to run the organization (e.g., a council such as an
Indigenous Band Council) should select “Yes”.
The applicant selects ‘’Yes’’ or ‘’No’’.
13
Does the organization have
current written policies and
procedures on Human Resources
and Occupational Safety in place
for delivering projects and
services?
This consideration refers to the organization’s ability: to plan,
manage and complete:
• Human Resources activities such as hiring and
training follow set procedures; and,
• Having a process in place addressing occupational
safety.
The applicant selects ‘’Yes’’ or ‘’No’’.
Financial Management
The following questions relate to the ability of the applicant to plan, direct and control financial
activities.
Does the organization have a
financial management system in
place (e.g., bookkeeping, tracking
expenses, general ledgers, and
monitoring)?
Applicants must confirm whether the organization has a financial
management system in place. A well-functioning system is
important for ensuring proper tracking and reporting of expenses
and compliance with funding requirements.
A financial management system doesn’t necessarily strictly entail
a computer-based program; any process, including paper-based
such as the use of a general ledger, is appropriate as long as
funds are being accurately tracked.
The applicant selects ‘’Yes’’ or ‘’No’’.
Does the organization have
policies and procedures in place
regarding internal controls that are
regularly reviewed and updated
(e.g., segregation of duties,
cheque signing controls, authority
to make purchases or incur
expenses)?
Applicants must confirm that the organization has policies and
procedures regarding internal controls, and that these are
regularly reviewed and updated.
Examples of internal controls include but are not limited to:
shared responsibilities of a key process that disperses the critical
functions of that process to more than one person, ensuring
authorities for expenditure approvals and cheque signing are
designated and known within the organization, and ensuring
expenditures are reviewed and approved by at least 2 separate
authorized staff within the organization.
The applicant selects ‘’Yes’’ or ‘’No’’.
Does the organization owe any
amounts to the Government of
Canada?
The applicant must declare whether the organization owes any
outstanding amounts to the Government of Canada. This includes
any amounts owing to the federal Crown, under legislation or
contribution agreements, that constitute an overdue debt.
The applicant selects ‘’Yes’’ or ‘’No’’.
14
We encourage applicants with monies owing to settle that
amount and keep proof of it in the event that this application is
approved. The applicant acknowledges that any amount owing is
a debt due to the federal Crown that may be set-off against any
amounts payable by Canada under a funding agreement.
Does the organization have staff
dedicated and trained to manage
the requested funding?
Applicants must confirm whether the organization has trained
employees dedicated to financial management functions to
mitigate risks in managing funding received. Having dedicated
financial positions in the organization held by trained staff is
sufficient to respond “Yes” to this question. Training
requirements or standards are determined by the applicant but
must be aligned to industry best practices.
The applicant selects ‘’Yes’’ or ‘’No’’.
Are controls in place to monitor
for compliance of sub-agreements
(financial and results)?
If No: How will the organization
monitor for compliance of subagreements (financial and results)?
This question is only applicable if the applicant will manage its
own agreements with other organizations through a subagreement. All other applicants should select “Not Applicable”.
In cases where there will be a flow of funds to sub-entities,
monitoring sub-agreements is critical for ensuring that all
partners adhere to funding requirements. Applicants must
confirm whether the organization has controls in place to ensure
compliance with the financial and result requirements of subagreements related to the requested funding. They must also
have the ability to track and record the monies flowing to subagreements.
Examples of controls include but are not limited to: managing
progress reporting functions, financial reviews, closeout reports
and audits.
The applicant selects ‘’Yes’’, ‘’No’’, or ‘’Not Applicable’’. If the
applicant selects “No”, they must specify how the organization
plans to monitor for compliance of sub-agreements.
Achieving Results
The following questions relate to the applicant’s ability to manage the funding in a way that achieves
the expected results outlined in the funding application and in alignment with the funding program’s
objectives.
Are processes in place to manage
and monitor the progress of
project(s) being delivered?

Applicants must confirm whether the organization has
established processes to effectively manage and monitor the
progress of projects being delivered. 
15
If No: Describe how the
organization intends to manage
and monitor the progress of
project(s)

Examples of established processes for monitoring progress
include but are not limited to: project management and oversight
functions, reporting requirements, schedules and templates,
tracking requirements or systems, data reporting requirements
and regularly scheduled meetings to track progress.
The applicant selects ‘’Yes’’ or ‘’No’’. If the applicant selects “No”,
the applicant must describe how the organization intends to
manage and monitor the progress of project(s).
Section 3 – Project Details
Project Rationale
Please choose the objectives of
your project (select all that apply).*
• Increase use of public transit relative to car travel
• Contribute to climate change mitigation and and/or climate
resilience
• Improve public transit options for all, especially equitydeserving groups (e.g., low-income)

Please describe the project that you
are undertaking. (For example,
what capital assets do you propose
to purchase, and how the project is
expected to meet the objectives
selected above? *
Provide a complete description of the work to be completed,
including the nature of the project and a description of all major
quantifiable components. Describe how the project will meet
the objectives of the Rural Transit Solutions Fund and its main
benefit(s), including information on the communities and any
vulnerable populations that will benefit. Please note that this
information may be used by HICC for public reporting.
Does your proposed project plan
address specific challenges outlined
in your Municipal Plan or alternate
local community planning
documentation?*
Please select “Yes”, “No” or “N/A.”
If Yes, please explain.* If “Yes” is selected, please explain any linkages the proposed
project has to a transit plan (e.g., municipal/regional/feasibility
studies or local municipal, and regional strategies, plans, Band
Council Resolutions).
Does the project intend to link
more than one Census
Metropolitan Area (CMA) or Census
Agglomeration (CA) with a
population of 50,000?*
Please select “Yes” or “No”
16
Does the project intend to cross
Province/Territory boundaries?*
Please select “Yes” or “No”. (Note: interregional travel is
permitted if the distances meet the day-to-day travel
requirement of the fund).
Does the project intend to cross
international boundaries?*
Please select “Yes” or “No”. Note: international routes (e.g., to
the United States) are not be eligible under the Fund.
Is your project intended for daily
use?* (e.g., The proposed transit
solutions intends to offer a sameday return service).
Please select “Yes”, “No”, or “N/A”.
Does your organization have a
policy and/or practices to ensure
that the vehicle(s) and other capital
assets are for public transit usage*
Please select “Yes” or “No”.
Applicants should have a policy in place to confirm that any
purchased vehicles or assets will be used for community benefit.
If your organization does not have such a policy, please describe
how you will make sure that the vehicles and assets are used for
public transit.
Please describe the current level of
access to public transportation
options in the community.*
Please include information on any existing transit services in the
community or communities where the project will operate,
including those run by your organization or your delivery
partners. Describe the degree to which any existing services are
inclusive and broadly available to the community and/or how
the existing service may not meet the needs of some community
members.
Will the transit solution replace
existing assets (e.g., on an existing
route/service), or help expand the
transit network (e.g., offering a new
route/service)?*
Please indicate whether the proposed transit solution is
intended to replace existing assets, such as upgrading vehicles
or improving current routes, or if it will expand the transit
network by introducing a new route or service that adds to the
current options.
Will your project serve Indigenous
populations?*
Please select “Yes” or “No”.
If yes, select all that apply*

o First Nations
o Inuit
o Métis
o Multiple Indigenous/Other
If Multiple Indigenous/Other is
selected, then specify.*
How will this project be developed
with Indigenous peoples? Please
explain.*
If your proposed project will serve Indigenous peoples, please
describe how (process and timelines) Indigenous peoples have
been/will be involved in its development.
17
Describe the direct and ongoing
benefits that this project will bring
to Indigenous people.*
Please describe the direct and ongoing results/outcomes that
are expected to benefit Indigenous people in the project’s
development and implementation.
Please describe the population,
including any vulnerable
populations, which are expected to
use the proposed service.*
Please provide examples of the vulnerable populations which
are expected to benefit from the project. Please identify how
your project will serve vulnerable populations. If possible, you
should list the sources of information (e.g., letters of support
from community service organizations, studies, reports, and
public consultations, etc.) that support your analysis.
Note: Vulnerable populations include but are not limited to
Indigenous peoples, racialized peoples, youth, persons with
disabilities, seniors, linguistic minorities, newcomers to Canada
(e.g., immigrants, refugees), women, persons experiencing
poverty, persons experiencing homelessness, and 2SLGBTQI+.
Please explain any benefits that are
expected for the local community
from the rural transit solution. As
applicable, include quality of life,
safety and security, and/or access
to employment opportunities.*
Describe the expected benefits to the targeted population(s)
resulting from the implementation of the project. Include as
many aspects of the benefits as possible in this description (e.g.,
access to health services, access to public goods/government
services, aging-in-place, safety and security, access to education,
living standards (poverty reduction), social connection, and
access to culture/recreation activities). Please identify, if
possible, how these benefits were identified (e.g., studies,
reports, public consultations, etc.).
Please explain how the rural transit
solution is expected to contribute
to the economy of the community.*
Describe the potential economic benefits expected from the
implementation of the proposed project. Examples may include:
estimates of job creation, access to employment/training, and
access to local business (local spending). Please note that
descriptions and estimates provided should be specific to the
project and community.
Please describe your organization
(budget and the number of
employees), your experience
operating a transit service and how
you expect to manage the project,
including your management
structure.*

Please describe your organization by providing your annual
budget, number of employees, and experience with transit
operations. Outline how you’ll manage the project, including
your management team, key objectives and milestones, asset
procurement policies and sources, and how you’ll monitor
budget and cash flow.
Please explain how success of the
project will be measured.*
Please provide information on the evaluation framework,
including project outcomes and performance measurement
strategy that demonstrates how data collected will be used to
improve the transit solution over time. 
18
Please explain how the operations
of the transit service, including the
maintenance of the assets, will be
funded over the next three to five
years, and confirm that these
operating funds are secured. If
these funds have not been secured,
please explain your strategy to
ensure the continued operation of
your transit service*
Describe how you will fund the operations and maintenance of
the transit service over the next 3-5 years, confirming if the
operating funds are already secured. If not, outline your plan to
ensure long-term financial sustainability, including covering
ongoing costs like insurance, permits, fuel, training, and
maintenance. Explain how you intend to meet these expenses,
whether through planned revenue sources like fares, or other
long-term funding sources.
Please explain how the proposed
asset(s) will integrate new
characteristics to encourage more
people to use public transit, and
thus reduce private vehicle trips.*
Describe how the new assets or services will encourage more
people to choose public transit over private vehicles. Outline
any strategies or promotional efforts planned to boost ridership,
especially among vulnerable groups, and explain why you chose
specific methods (e.g., radio, social media, transit apps). If
possible, highlight design features that enhance user
experience, such as comfort, safety, convenience, affordability,
and enjoyment.
How do you plan to communicate
the proposed transit solution or
services to encourage more people
to use public transit?*
Describe your communication plan to promote the new transit
solution or services to the community. Explain how you will
inform and engage potential users, such as through social
media, local advertising, community events, or partnerships
with local organizations. Highlight any specific strategies aimed
at reaching diverse or underserved populations and encouraging
regular transit use.
Please provide details on current or
future community and stakeholder
engagement activities that have
been or will be conducted in
relation to the project, including
those that have included
Indigenous peoples, if applicable.*
Please provide information on community and stakeholder
engagement activities. This would include the type of activity
(e.g., consultations, meetings, surveys) along with the date,
location, audience). Explain how these activities contribute to
the development of the proposed project.
Asset Type
It is important to identify the asset type and activities that apply to the project using the asset lists below.
This is information is essential for HICC to determine both Environmental Impact Assessment
requirements and Indigenous consultation requirements that would apply to the project. All projects
must comply with the Impact Assessment Act and the Canadian Environmental Assessment Act (2012),
Indigenous consultation and accommodation obligations, and modern treaty obligations. It is therefore
critical that the correct asset type and activities are identified. Depending on the asset type and level
identified, you may be asked to supply additional information. 
19
Please note that depending on where the project is located, an environmental impact assessment may be
required prior to undertaking certain activities. Any Indigenous consultation requirements identified for
the project would also need to be completed prior to undertaking certain activities.
Before selecting the asset type for your project, please review the entirety of Level 1, 2 and 3 assets
lists and ensure you are familiar with all the activities required for your project, to allow you to select the
most appropriate asset types from the list, and that these align with the description of the project.
Each asset of the project will need to be created individually. Using the “Add” buttons under the
appropriate level(s), select the applicable “Asset Type”; and “Work Description” receiving investments for
your project, and specify the “Length (km)” and asset “Count” (number of assets), as applicable. The
drop-down menus will display the following items.
Level 1 – Asset Type and Work Descriptions
No physical works

o Rolling stock purchases (includes zero-emission buses, electric
rolling stock: trolley buses, light rail cars, subway cars, street cars)
Physical works -
Replacement of existing
assets
o Replacement of sidewalks, cross-walks, pavement markings, street
marking, signage, curb dividers, bike racks, washrooms, lockers,
lights, benches, greenery, speed bumps, and raised barriers
Physical works –
Installation of new assets
or enhancement of
existing assets in
developed areas

o Sidewalks
o Cross-walks
o Pavement markings
o Street Marking
o Signage
o Curb dividers
o Lights
o Speed bumps
o Raised barriers installed to existing transportation routes (installed
for protection of active commuters)
o Ramps for accessibility, hand railings, wheelchair accessibility,
installation to existing transit facilities
Transit

o Installation of charging stations (on developed land, minimal
ground disturbance, no vegetation clearing required)
o Upgrading existing Transit Stations (installation of technology,
station improvements/retrofits on developed land)
o Retrofits of conventional fuel buses to ZEB (zero-emissions buses)
o Upgrades to the interior of buses, installation of driver shields,
installation of securing belts, driver barriers, ventilation, sanitation
systems
o Upgrades to electronic systems, fare collection system, mobile
communications systems, installation of electronic equipment tvs,
monitors
20
o Upgrades to public sector commuter ferry infrastructure (e.g.,
installation of technology, ventilation, sanitation systems)
o Installation of bus stops, bus shelters and bike racks that are not
on federal lands (are less than 100 m2
in area, and not within 15 m
of waterbody)
Level 2 – Asset Type and Work Description
Public Spaces

o Curb Extensions – in a developed area
o Cross-walks (Pedestrians crossing – installation of fully activated
signaled safety crossing in a developed area)
o Ramps for accessibility, hand railings, wheelchair accessibility –
installation to active transportation multi-use pathways
o Upgrades, retrofits, modification of existing public spaces that are
located on municipal or provincial lands that are developed or
previously disturbed (includes existing pedestrian tunnel retrofit)
Transit

o Retrofits, rehabilitation, modification, upgrades of existing bus
depot, transit facility, commuter hub on municipal lands or
industrial park
o Installation of charging stations on undeveloped land (less than
100 m2
) – Project on Federal Lands may have EA requirement,
may have DTC requirements
o Expanding existing Transit Stations (1000 m2 or less on developed
land)
o Developing new transit Facility (1000 m2 or less on developed land

Level 3 – Asset Type and Work Description
Transit o Installation of charging stations (more than 100 m2 on
undeveloped land) may require vegetation clearing (removal of
trees, bush, including roots, works taking place within 15 m of a
waterbody)
o Developing new transit facility on underdeveloped land (1,000 m2
or less)

Project is not described by any of the above
If the project is not described by any of the options, you may check the box.
21
Rolling Stock and Other Capital Assets
Does your project include the
purchase or replacement of
rolling stock?*
Please select “Yes” or “No”.
If Yes, a series of drop down menus will appear according to the
following fields:

Asset Type

o Truck (4-9 seats)
o Car (5-7 seats)
o Van (5-11 seats)
o Shuttle/Mini-Bus (10-30 seats)
o Bus (31-50 seats)
o Large Bus (51 seats or more)
Description

o Electric
o Gas/Diesel
o Hybrid
o Plug-in Hybrid
o Propane
o Other
Other If you identified “other’ in the description list, you may provide
details here.
Count The number of assets as per the identified asset type.
Name of Document In this section you can upload supporting documentation, e.g.,
vehicle descriptions.
Press the disk icon to save. You may add more rolling stock assets as
applicable by pressing “+Add Rolling Stock”.
Does your project include the
acquisition of other Capital
Assets?*
Please select “Yes” or “No”
If yes, please provide more
information:
Please describe the additional assets sought as part of this
application.
Accessibility Standard
Please confirm that the project
will meet or exceed the highest
published accessibility standards
(e.g., the Canadian Standards
Association Technical Standard
Accessible Design for the Built
Environment (CAN/CSA B651-23,
or newer) in a jurisdiction, in
Please select “Yes”, “Not Applicable”, “Unsure or Unknown”
For more information, please consult the Canadian Standards
Association Technical Standard Accessible Design for the Built
Environment (CSA/ASC B651:23, Accessible design for the built
environment) https://www.csagroup.org/wpcontent/uploads/2430328.pdf
 
22
addition to applicable provincial
or territorial building codes,
requirements for Accessible
Transportation Services and
relevant municipal by-laws?*

Climate Considerations
Describe the current and future climate risks facing the project over its entire lifespan. For more
information, please see the Climate Change Considerations section of the Application Guide for the
Capital Stream of the Rural Transit Solutions Fund. You can also contact the HICC Climate help desk:
Reference the climate help desk Webform: https://support-infc.canada.ca
Email: Climate-Infra-Climat@infc.gc.ca; or,
Toll free phone: 1-833-834-0243
The Climate Help Desk is part of the Climate Toolkit for Housing and Infrastructure.
Are you considering the option of
using low carbon technologies
(e.g., zero-emission vehicles) to
deliver your public transit
solution?*
Choose “Yes,” “No,” or “N/A.” If “Yes,” share details about any
low-carbon options you’re planning, like electric vehicles. If “No,”
briefly explain why, like cost limits or technology availability.
Does the project include fixed
assets or other eligible expenses
affected by climate impacts?*
Choose “Yes,” “No,” or “N/A.” If “Yes,” you’ll be asked more about
climate impacts. Buildings, vehicles, and equipment that last a
long time may be affected by things like storms, floods, or
extreme heat.
Is the project site currently, or
may the project site be in the
future, subject to flooding
(coastal, riverine or overland),
wildfire, extreme heat,
permafrost thaw, or other climate
hazards during the asset’s
lifespan?*
Choose “Yes,” “No,” or “N/A.” If “Yes,” think about any risks in
your area now or in the future, like floods, wildfire, extreme heat,
or other hazards that could affect your project site.
If yes, please list all relevant
climate hazards.*
List any specific climate risks in your area, like “flooding,”
“heatwaves,” or “wildfire.” You can look up local climate data or
talk to local experts if needed.
Identify what best available
climate data was used to assess
the project’s probability for
applicable climate hazards.*
Choose either “Current or historic climate data” (like past climate
info) or “Future climate data” (like forecasts of climate change
effects). Mention any local or national data sources you used. 
23
Describe how the project will
mitigate the climate impacts and
risks identified above. If no
climate risk reduction measures
would be implemented, please
provide a justification.*
Describe any steps you’ll take to protect your project from climate
risks, like raising buildings to avoid flood damage or using heatresistant materials. If you don’t plan any steps, explain why (e.g.,
no risks identified, budget limits).
To prevent losing your work, please click “Save and Continue” in each section or click on “Save All”. Use
“Validate” when you have completed all questions in this section to check that all mandatory sections are
complete.
Expected Outcomes
Outline the type of service you plan to offer and estimate, to your best knowledge, key indicators that will
help us understand the impact of your project.
Please identify the type of
service that you intend to
provide. (select all that apply)*
Select all relevant options for the types of transit services you plan
to provide, such as fixed routes, route/point deviation service/ondemand (demand-response service), paratransit, multimodal
transit, micromobility transit or other.

Specify if 'Other' If you choose “Other”, provide a brief description of the service
type (e.g., community carpool).
Seat Capacity* Estimate your fleet’s seat capacity before and after the project. For
example your current fleet consists of one car with four seats and
you purchase a second car with four seats your seat capacity after
your project will be eight seats.
Ridership per year* Estimate to the best of your knowledge the expected number of
rides per year. Use current figures if available, or project based on
similar services and local demand. Each individual trip taken by a
person counts as one ride. For example, if a family of three uses the
transit service to travel to an appointment and then returns home,
this would count as six rides in total.
Vehicle-km per year (optional) Include the total annual distance your vehicles will travel, if
available. This is optional but helps estimate service reach and
operational requirements.
Additional comments Use this space to add any context for your estimates, such as
assumptions, rural or underserved area considerations, or expected
seasonal changes in ridership.
24
Section 4 – Environmental Assessment
All projects under the Permanent Public Transit Program are required to comply with the Impact
Assessment Act and, Indigenous consultation and accommodation obligations, and modern treaty
obligations. See Annex A for additional information on Environmental Assessment (EA) & Indigenous
Consultation (IC).
Applicants may be required to complete questions related to environmental assessment and Indigenous
consultation, depending on the type of assets selected. See Annex B of this document for guidance on the
Detailed EA Project Description, Environmental Assessment (EA) - Indigenous Consultation Checklist and
Environmental Assessment (EA) Questionnaire.
Section 5 - Project Finances and
Supplementary Information
This section describes the project finances, including details of the project budget, sources of funding
(e.g., funding contributors), fiscal year breakdown (e.g., program contribution by fiscal year), and other
information related to project costs.
It is recommended that you complete the Project Cash Flow Table before proceeding to other elements
of this section. The Project Cash Flow Table will provide you with the amount required to calculate the
Requested Federal Contribution.
Project Financials (Project Estimated Costs)
Total Estimated Project Cost* This portion should outline the expected costs of the project based on
researched and documented cost estimates. This should consider the
total costs – not only the Federal Contribution requested through the
Rural Transit Solutions Fund, but also the applicant’s share of the costs,
along with the total from other contributors. Please consult the
Application Guide for a list of eligible and ineligible expenses under the
“Budgeting” section.
Requested Federal Contribution* Please state the total funding request under the Rural Transit Solutions
Fund. This amount must be based on Eligible Costs, not total costs.
Eligible costs: costs outlined in the project budget that are eligible for
reimbursement under the Rural Transit Solutions Fund.

Ineligible costs: costs outlined in the project budget that are not eligible
for reimbursement under the Rural Transit Solutions Fund.
25
IMPORTANT: Operational costs such as salaries, fuel, and insurance are
ineligible expenses and are not to be included as an eligible costs.
Including operational expenses and other ineligible costs as part of your
total eligible costs may result in your application not being considered
for funding.
See the Applicant Guide for the Capital Project Stream for a list of eligible
and ineligible costs under the Rural Transit Solutions Fund.
Eligible organizations with a minimum of two years of transit experience,
including local government partnering with a transit organization,
and/or that have completed a public transit feasibility study, can receive
up to $10 million from the Rural Transit Solutions Fund. Eligible
organizations with no prior transit experience and have not completed a
public transit feasibility study may receive up to $250,000.
The Total Estimated Eligible Expenditures + Contingency is the amount
used to calculate the Rural Transit Solutions Fund Contribution.

Below is a list of maximum Rural Transit Solutions Fund funding limits based upon eligible organization
type:
Eligible Organization Maximum Rural Transit Solutions Fund contribution
(% of eligible expenses)
• Recipient is located in the Territories
• Recipient is an Indigenous
community/organization
• Territorial Governments
Up to 100%
• Recipient is located in a province Up to 80%
• Provincial government or
provincial crown corporation
Up to 60%
Applicant's Share of Total Cost*

Please indicate your financial contribution as a share of the total
estimated cost. This refers to the amount of total costs that your
organization will contribute and that is not part of the federal funding
being requested.


 
26
Other contributors
In this section, indicate if the proposed project is receiving funding from other sources. This includes
funding from the applicant organization. Explanations for the fields in this section are provided below.
Add as many rows as there are contributors to the project by clicking “+”.
Name of Contributor/Other
Contributor.
Enter the legal name of the organization or entity.

Contribution Amount Please round up the dollar value of the contribution amount to be
provided by the organization/entity.

Funding Secured? Please select “Yes” or “No”.

Is there a letter of support? Please select “Yes” or “No”.

Upload document Please attach the letter of support here.
Once you have filled out the fields, press on the disk icon to save. Then you may add other contributors to
the project by clicking “+Add another Contributors”
Total from Other Contributors This field will be populated based on the information provided under
“Other Contributors”.
Project Cash Flow Table
In this section you can add multiple project components (e.g., fixed assets, rolling stock) in the fields, and
their estimated cost by Pressing “+Add a Project Component”. The field will appear as per the below.
Please make sure you keep the following considerations in mind when you fill out the Cash Flow Table:
• The project components should be consistent with your asset request. You will also be asked to
provide a breakdown of the estimated eligible expenditure per fiscal year.
• If the Project Component is not eligible (e.g., operational expenses such as drivers salary or fuel
etc.), please leave the Estimated Eligible Expenditure at zero. Consequently, the Breakdown by
fiscal year will be set to zero. The inclusion of non-eligible costs in the calculation of the federal
contribution could result in the application not being considered for funding.
• The contingency amount should be in the stage of your project as outlined below.
• The Total Estimated Expenditures + Contingency amount equals the Total Estimated Project Cost. 
27

Project Cashflow Table
Estimated Eligible
Expenditure Annual
Breakdown by fiscal year

Project
Components
Estimated
Expenditure
Estimated
Eligible
Expenditure
2026-
2027
2027-
2028
2028-
2029
2029-
2030
Total Eligible
Expenditure
Individual Cost
Items

Subtotals
Contingency
Total Estimated
Expenditures +
Contingency

Total Estimated
Eligible
Expenditures +
Contingency

The Total Estimated Eligible Expenditures + Contingency is the amount used to calculate the Rural Transit
Solutions Fund Contribution.
Fiscal year breakdown
This section of the application form details the amount of funding from Housing, Infrastructure and
Communities Canada that the applicant will claim for a given fiscal year (April 1st to March 31st). Provide
a breakdown of the program contribution costs for each fiscal year of the project. Round up to the
nearest dollar. 
28
Note: the fiscal year breakdown must be based on when expenditures will be incurred and not when they
will be submitted to Housing, Infrastructure and Communities Canada for reimbursement. For example, if
expenditures will be incurred in December 2025, but will not be claimed for reimbursement until April
2026, then these expenditures would be listed in the 2025-26 fiscal year.
The following table is also included to help applicants determine the most applicable contingency amount
to add to their Cash Flow table.
Class Contingencies
Class A: Estimates made after bids for a project have
been received, evaluated, verified and once a
contract is ready to be signed.
Budgets for projects at this stage usually include a
contingency of 5% to 10%.
Class B: Estimates made at the “Detailed Design”
stage when the project is ready for tendering.
Budgets for projects at this stage usually include a
contingency of 11% to 15%.
Class C: Estimates at the “Preliminary Design” stage
and may be referred to as pre-tendering estimated.
Budgets for projects at this stage usually include a
contingency of 16% to 20%.
Class D: Estimates at the “Conceptual Design” stage. Budgets for projects at this stage usually include a
contingency of 21% to 30%.


Non-competitive procurement

This section asks if any non-competitive contracts will be used for the proposed project. Non-competitive
procurement is defined as a contract that is entered into without soliciting bids and that a particular
supplier is the only one acceptable.
Will any non-competitive
contracts over the following
thresholds be used toward
eligible costs for the project?*
• A non-competitive
contract over $40,000
for construction or
goods
In this section, indicate whether any non-competitive contracts over the
indicated thresholds will be used. 
29
• A non-competitive
contract over $100,000
for services
Contract #1
Name of Entity, Amount, Nature
of the Work, and Rationale for
the non-competitive contract


If applicable, enter information about the contract, including the name of
the entity receiving the contract, the amount of the contract, and the
nature of the work to be performed. Also include a rationale for why the
contract cannot be awarded through a competitive process. Click on the
[+] to add an additional row for each contract.
Estimated Project Schedule
Project start date.* Please select from the calendar.

Estimated Project end date.* Please select from the calendar.
Note: Projects are expected to be completed by 2029-2030.

Applicant agrees to be
responsible for the ongoing
maintenance cost for the years
following the end of the
contribution.*
Please select the box.

If this application is approved,
the applicant will award and
manage all contracts paid with
RTSF funds in a way that is
transparent, competitive,
consistent with value-for-money
principles, or in a manner
otherwise acceptable to
Canada.*
Please select the box. Contracts that do not respect this condition could
be deemed ineligible under the Fund.

Does your organization have the
capacity to maintain the
operation of the asset(s) for a
period of 5 years?*
Please select the box.

Is your organization currently or
will your organization be the
owner of the asset or asset(s).*
Please Select “Yes” or “No” or “N/A”

If No, please specify.* Please identify the legal owner of the asset(s); as well as who will retain
ownership of the asset(s) following the completion of the project. 
30
To prevent losing your work, please click “Save and Continue” in each section or click on “Save All”. Use
“Validate” when you have completed all questions in this section to check that all mandatory sections are
complete.
Section 6 - Project Risk
When answering questions in this section, identify all relevant risks that may impact the project, and
provide a risk response and/or mitigation measures. Note that more information may be requested at the
agreement negotiation stage.
Please identify all relevant risks
that may impact the successful
completion or forecasted end
date of the project (e.g.,
interdependencies, labour and
supply chain issues, public
sensitivity) and provide
mitigation measure(s) or a risk
response.*
Consider:
• Risks associated with potential delays in obtaining permits and
approvals, with land acquisition or other project components (the
extent that approvals have already been sought, permits secured,
and land “shovel-ready” increases the success that a project will be
started in good time and be completed on a timely basis);
• Risks to timelines created by interdependencies with other projects
or initiatives;
• Risks of delays associated with skilled labour shortages or supply
chain issues; and
• Risks associated with public perception of the project. Have
community groups and stakeholders been consulted? Have concerns
raised been addressed?
Please identify all relevant risks
that may increase the costs (cost
overrun) of the project and
explain how these risks may be
mitigated.*
HICC does not provide additional funding for cost overruns. As such, it is
important to be as accurate as possible when developing project cost
estimates. Contributory risk factors that should be considered include,
but are not limited to, the following:
• Fluctuations in the availability and or costs of labour, equipment and
materials;
• Project delays due to weather conditions, permitting issues or other
factors; and,
• Unexpected project scope changes.
Is the project using specialized
technology or equipment that is
new/unfamiliar to the applicant
or industry? If so, please
describe the organization’s
experience using the specialized
technology or equipment (if
any), and identify relevant risks
If the project will be using specialized technology or equipment:
• Consider the overall familiarity of the industry with this technology or
equipment. Is it widely implemented, or is it still emerging?
• What experience (if any) does the organization have with the
specialized technology or equipment? If the organization lacks direct
experience, what steps will be taken to build expertise?
• Does the applicant have a strategy for using the technology, and a
contingency plan in case of failure?
31
and corresponding mitigation
measures.*
Section 7 - Required Documents
This is the section where you must attach the required documents in accordance with the applications
requirements. Please note in accordance with the Application Guide, certain documents may be needed
depending on your organization type or if you have partners.
Eligible Recipient Document type
Public sector bodies that are
established by or under provincial or
territorial statute, or by regulation, or
are wholly-owned by a province,
territory, municipal or regional
government
• Letter of Support (Financial Support): From project partners
confirming their financial contribution.
• Audited Financial Statement: Provide the most recent audited
financial statement.
• Feasibility Transit Study, if applicable.
Indigenous governing bodies   • Letter of Support (Financial Support): From project partners
confirming their financial contribution.
• Band Council Resolution: If applicable, confirming the applicant's
responsibility for transit services in their community.
• Audited Financial Statement: Provide the most recent audited
financial statement.
• Feasibility Transit Study, if applicable.
Federally or provincially incorporated
not-for-profit organizations whose
mandate is to improve Indigenous
outcomes, organizations serving
Indigenous communities living in
urban centers and First Nations living
off-reserve 
• Letter of Support (Financial Support): From project partners
confirming their financial contribution.
• Letter of support for Indigenous governing bodies: If applicable,
confirming support for the transit solution and acknowledging
the applicant's role for transit services in the community.
• Proof of Incorporation: Provide proof, such as articles of
incorporation.
• Audited Financial Statement: Provide the most recent audited
financial statement.
• Feasibility Transit Study, if applicable.
Indigenous development
corporations 
• Letter of Support (Financial Support): From project partners
confirming their financial contribution.
• Letter of support from Indigenous Governing Bodies or Band
Council Resolution: If applicable, confirming support for the
transit solution and acknowledging the applicant's role for transit
services in the community.
• Audited Financial Statement: Provide the most recent audited
financial statement. 
32
• Feasibility Transit Study, if applicable.
Federally or Provincially incorporated
Not-for-profit organizations 
• Letter of Support (Mandate of the Organization): From an order
of government (e.g., municipality, regional government or
province) or from an Indigenous Governing Body, confirming the
applicant's mandate to deliver transit services or recognition in
the community/region.
• Letter of Support (Financial Support): From project partners
confirming their financial contribution.
• Proof of Incorporation: Provide proof, such as articles of
incorporation.
• Audited Financial Statement: Provide the most recent audited
financial statement.
• Feasibility Transit Study, if applicable.
To upload a document, please select “Choose File” Once it is uploaded it will appear under the “Name of
Document” field. You may add multiple files in this section. Please note that HICC may seek additional
information and documentation.
To prevent losing your work, please click “Save and Continue” in each section or click on “Save All”. Use
“Validate” when you have completed all questions in this section to check that all mandatory sections are
complete.
Section 8 - Attestation
Validation Status
This section will validate that you have completed all the required sections of your application. If the
section shows as green, it is considered complete for the purposes of submission. If it is red, you must
ensure that the required section is filled out.
Privacy and Eligibility
A Privacy Notice statement will appear, which you must read. Once you have completed reading it, you
must attest that you have read the Privacy Notice Statement by selecting “Yes” or “No”. If you select No,
you will be unable to move to the next section of the application and to submit your application. 
33
Annex A: Project Activities List and
Guidance for Engagement with Indigenous
Peoples
Many asset classes eligible through Rural Transit Solutions Fund have potential for impacts to the
environment and could potentially adversely impact Aboriginal and/or treaty rights. While most impacts
can be avoided or mitigated with good environmental construction practices, Indigenous peoples and
their rights could still be adversely impacted by the project or related activities. Therefore, HICC
encourages applicants to start a dialogue with Indigenous peoples potentially impacted by the project as
early as possible (ideally during planning). Early engagement provides applicants with an idea of the scope
and potential costs of Indigenous consultation activities, which are retroactively eligible expenditures.
1. Identify if the project could impact Aboriginal and/or treaty rights
In general, projects not likely to have impacts include:
• Projects without physical works, such as desktop studies, purchase of technology, or the purchase
of rolling stock/fleet vehicles
• Take place entirely within existing buildings or structures
• Take place along existing active transportation routes that have minimal environmental impact
In general, projects that could have impacts include:
• Physical works on previously undeveloped or undisturbed lands
• Physical works taking place in or near a waterbody
• Excavation or vegetation clearing
• Construction/installation of new assets, or the expansion of existing assets, such as buildings or
fixed structures, transit infrastructure (e.g., roadways, railways, tunnels, bridges), or active
transportation infrastructure (e.g., bike lanes, paths/trails, footbridges, pedestrian tunnels)
2. Identify which Indigenous communities to engage
The Aboriginal and Treaty Rights Information System can provide the location of Indigenous
communities in relation to the project area and information on their rights. Enter the project location
and select a search buffer to identify communities in proximity to the project area, and those that
may be impacted by downstream project effects. See the ATRIS user guide.
3. Start a dialogue with each potentially impacted Indigenous community
HICC recommends sending a letter/email to each potentially impacted community, including: a brief
introduction to the Rural Transit Solutions Fund and the project’s purpose; detailed project
information including maps and project activities; and a request that communities share any concerns
related to potential impacts to their rights within a reasonable timeframe (HICC suggests providing at
least 30 days). If a community responds with concerns about the project or an interest in learning
more, HICC recommends arranging a phone call or meeting with them. If a response is not received
within the timeframe suggested in the letter, the applicant should follow up with additional emails or
phones calls.

 
34

4. Track communications and concerns
Document all engagement and consultation activities. Record concerns raised by Indigenous
communities, discussion outcomes, planned future activities, and related expenses incurred or
anticipated. Supporting documents can be submitted with the application. If you would like a
template letter to send to Indigenous communities, or a template engagement/consultation log,
please email your request to: EAIC-EECA@infc.gc.ca.
35
Annex B: Duty to Consult with Indigenous
Peoples and Impact Assessment Act
Requirements
HICC funding of projects is contingent upon Canada ensuring the respect of federal environmental
statutory requirements. Projects submitted to HICC may be subject to the Impact Assessment Act,
Northern Regulatory Regimes, and/or other federal environmental statutory requirements such as the
Fisheries Act, the Canadian Navigable Waters Act, and the Species at Risk Act that are administered by
other federal government departments. For more information, please see the “Additional Application
Considerations” section of the Application Guide for the Capital Stream of the Rural Transit Solutions
Fund.
The funding of projects by HICC is also contingent upon Canada fulfilling the constitutional duty to consult
and, where appropriate, accommodate Indigenous peoples when activities could adversely impact
Aboriginal and treaty rights. Applicants are responsible for providing detailed project information to assist
HICC determine whether the funding of projects may trigger the Department’s constitutional duty to
consult and, where appropriate, accommodate Indigenous peoples. HICC encourages funding applicants
to engage with Indigenous peoples early in project planning and design.
In the application, you must provide detailed project information to assist HICC in determining whether a
project would be subject to Environmental Impact Assessment or Indigenous Consultation requirements.
Depending on the project, applicants may be required as part of the application to complete a “Detailed
Environmental Assessment Project description”, an “Environmental Assessment - Indigenous Consultation
Checklist”, and an Environmental Assessment Questionnaire. The online application form for the Rural
Transit Solutions Fund will guide you through these steps accordingly. The information that follows
provides further information on the sections of the form that you may be required to fill out, as
applicable.
Detailed Environmental Assessment (EA) Project Description
Please describe the project you are
undertaking.*
Please describe the project
purpose, components, associated
works, undertakings/activities and
schedule. When describing the
project components and activities
(e.g., clearing, demolition of
existing works, excavation or
drilling, dredging, decommissioning
etc.), please provide a description
of the environment where this work
is taking place. For example, are the
works and related undertakings
Please provide detailed information in the text box provided.
This question is asking for a detailed project description. When
describing the project purpose include details related to any
increases in infrastructure capacity, if the project includes
replacement or repairs to existing infrastructure, or if new
infrastructure will be developed or increase an existing building
footprint. This section can also describe how the project may
benefit Indigenous peoples or the environment.
Please include a detailed breakdown per component, including
the size of project components (in square meters). When
describing project activities be as descriptive as possible (e.g.,
clearing vegetation, scrubbing, grading, demolition of existing
works, excavation or drilling, dredging, new construction,
retrofitting, repairing or rehabilitating, expansions of buildings 
36
taking place on developed land
(e.g., deforested, built-up, or
agricultural land), or land that is
forested, vegetated or previously
undeveloped? It is important to
describe any work taking place near
water and to indicate who owns
and controls the land that will be
affected by the project. Clearly
indicate the size of the project (e.g.,
new construction, expansions) and
project activities (e.g., area to be
cleared) in m².

or infrastructure footprint, or decommissioning existing
structures etc.).
Please provide a description of the environment where this
work is taking place. For example, are the works and related
undertakings taking place on developed land (e.g., deforested,
built-up, or agricultural land), or land that is forested, vegetated,
or previously undeveloped? It is important to describe any work
taking place in water, over water, near water, or under a water
body and the distance of works from any waterbodies. Also,
describe the lands where the project is located and indicate who
owns and controls the land that will be affected by the project.
Clearly indicate the size of the project (e.g., new construction,
expansions include side of building footprint), various project
components (e.g., new trails include length in m or km) and
project activities (e.g., size of area to be cleared) in m².
Environmental Assessment (EA) and Indigenous Consultation Checklist
Is the project located on
federal lands (includes Indian
Reserve lands)?*
Please select “Yes”, “No”, or “Unknown”.
This question is asking if the project or any of its components or
project activities are located on federal lands.
A Federal land (defined in the Impact Assessment Act), includes:
a. lands that belong to Canada, that Canada has the power to
dispose of, and all waters on and airspace above those lands,
other than lands under the administration and control of the
Commissioner of Yukon, the Northwest Territories, or
Nunavut;
b. the following lands and areas: (i) the internal waters of
Canada, in any area of the sea not within a province, (ii) the
territorial sea of Canada, in any area of the sea not within a
province, (iii) the exclusive economic zone of Canada, and (iv)
the continental shelf of Canada; and
c. reserves, surrendered lands and any other lands that are set
apart for the use and benefit of a band and that are subject
to the Indian Act, and all waters on and airspace above those
reserves or lands.
Examples of federal lands are: Department of National Defense lands,
Transport Canada lands, Port Authority lands, Parks Canada lands,
Indian Reserve lands, and National Wildlife Areas.
Is the project located in a
Migratory Bird Sanctuary,
National Wildlife Area or
Marine Protected Area?*
Please select “Yes”, “No”, or “Unknown”.
This question is asking if the project or any of its components or
project activities are located in a Migratory Bird Sanctuary, National
Wildlife Area, or Marine Protected Area.
37
All National Wildlife Areas are federal lands. A Migratory Bird
Sanctuary, or Marine Protected Area could be federal land, but this
depends on the who owns the lands and may need to be investigated
further by federal authorities.
Migratory Bird Sanctuary – An area which provides safe refuge for
migratory birds in the terrestrial and marine environment, managed
by The Canadian Wildlife Service of Environment and Climate Change
Canada. They are listed under the Schedule in the Migratory Bird
Sanctuary Regulations, which prescribe rules and prohibitions
regarding the taking, injuring, destruction or molestation of migratory
birds or their nests or eggs in the sanctuaries. Hunting of listed
species under the Act is not permitted in any Migratory Bird
Sanctuary.
National Wildlife Area – An area which is created and managed for
the purposes of wildlife conservation, research, and interpretation.
Environment and Climate Change Canada uses an ecosystem
approach to manage and plan for National Wildlife Areas. National
Wildlife Areas can only be designated on lands owned by the federal
government (e.g., federal land).
Marine Protected Area – A part of the ocean that is legally protected
and managed to achieve the long-term conservation of
nature. Further information about Marine Protected Areas.
Does the project include works
in-water (includes streams,
rivers, lakes, ponds,
wetlands)?*
Please select “Yes”, “No”, or “Unknown”.
This question asks if the project or any associated works will take
place within a waterbody. This includes but is not limited to:
shoreline work below the high-water mark, installation of underwater
cable or piping, blasting shoreline/embankment areas, installation of
bridge or dock piers in-water, and the placement of fill in a
waterbody.
A water body can include a lake, a canal, a reservoir, an ocean, a river
and its tributaries and a wetland, up to the annual high-water mark,
but does not include a sewage or waste treatment lagoon, a mine
tailings pond, an artificial irrigation pond, a dugout, or a ditch that
does not contain fish habitat as defined in subsection 2(1) of
the Fisheries Act.
Is the project and works within
15 m of a water body?*
Please select “Yes”, “No”, or “Unknown”.
The question asks if the project, or any associated project works, will
take place within 15 metres of a water body.
A water body can include a lake, a canal, a reservoir, an ocean, a river
and its tributaries and a wetland, up to the annual high-water mark,
but does not include a sewage or waste treatment lagoon, a mine 
38
tailings pond, an artificial irrigation pond, a dugout or a ditch that
does not contain fish habitat as defined in subsection 2(1) of
the Fisheries Act.
Is the project and works taking
place on undeveloped or
undisturbed lands?*
Please select “Yes”, “No”, or “Unknown”.
The question asks if the project, or any associated project works, will
take place on undeveloped or undisturbed lands.
An Undeveloped land is land not cleared of vegetation, in a natural
state, and not currently used for human purposes. Undeveloped
lands include undeveloped shorelines, riverbanks or gullies,
grasslands, forested areas, and scrub/brush areas.
A Developed land is land that was previously cleared of vegetation
and used for human purposes. Developed lands include highly
developed urban areas, rural areas that were previously cleared and
used for agricultural purposes, brownfields, railways, or road right of
way (ROW).
An Undisturbed land is land in its natural state and not currently used
for human purposes.
Disturbed land is land that has been altered by humans and includes
physical disturbance of the surface layer.
Does the project require
vegetation clearing? (removal
of vegetation that has not
previously been cleared)*
Please select “Yes”, “No”, or “Unknown”.
The question asks if the project, or any associated project works,
requires vegetation clearing.
Vegetation clearing is the intensive removal of undisturbed
vegetation including trees, stumps, logs, bush, shrubs, and grasses,
including tree root systems, and requires the use of heavy equipment
or industrial machinery for clearing and grubbing an area. This does
not include removing vegetation in previously developed areas,
manicured lawn or turf areas, or grassed ditches.
Developed land is land that was previously cleared of vegetation and
used for human purposes. Developed lands include highly developed
urban areas, rural areas that were previously cleared and used for
agricultural purposes, brownfields, railways, or road right of way.
Vegetation removal includes the removal of lawn or turfed areas or
grass from ditches, and is not considered vegetation clearing.
Vegetation removal is less intensive than vegetation clearing as it is
not occurring in an undisturbed or an undeveloped area.
Does the project involve the
placement of temporary or
permanent fill in a water
body?*

Please select “Yes”, “No”, or “Unknown”.
The question asks if the project, or any associated project works,
involves the placement of temporary or permanent fill in a water
body.
39
Fill may include soil, clay, stone, or rock as well as other substances
being used for the purpose of structure development or isolation of
the worksite.
A water body can include a lake, a canal, a reservoir, an ocean, a river
and its tributaries and a wetland, up to the annual high-water mark,
but does not include a sewage or waste treatment lagoon, a mine
tailings pond, an artificial irrigation pond, a dugout or a ditch that
does not contain fish habitat as defined in subsection 2(1) of
the Fisheries Act.
Does the project require
excavation?*

Please select “Yes”, “No”, or “Unknown”.
The question asks if the project requires any work that involves
excavation or digging.
Typically, excavation requires the use of heavy machinery to move
earth and prepare a construction site.
If the project requires the use of heavy machinery to excavate select
“Yes”. If the project does not require the use of heavy machinery to
excavate select “No”. When it is unclear if excavation is required,
please select “Unknown”.
Does the project involve the
disturbance of known or
suspected subsurface
contamination?*
Please select “Yes”, “No”, or “Unknown”.
This question is asking if the project requires disturbance of known or
suspected subsurface contamination or removal of any contaminated
soils or contaminated areas (e.g., removal of old inground storage
tanks or removal of old railroad ties).
Subsurface Contamination is any addition of undesirable substances
to soils and/or groundwater caused by human activities is considered
to be contamination (e.g., Landfill leachate, leaking gasoline storage
tanks, leaking septic tanks, and accidental spills).
Will the project involve the
removal of, or cause damage
to, any structure or resource
that is of known historical,
archaeological, paleontological
or architectural significance?*
Please select “Yes”, “No”, or “Unknown”.
This question is asking if the project will either damage a structure of
known historical, archaeological, paleontological, or architectural
significance or if the project requires the removal of one of the above
(e.g., removal of a historical building, damage an area of known
archaeological significance).
Will the project cause a
change to migratory birds or
nests, as defined in subsection
2(1) of the Migratory Birds
Convention Act, 1994?*

Please select “Yes”, “No”, or “Unknown”.
This question is asking whether there is any potential to impact
migratory birds and/or their nests. Key questions to consider include
whether construction activities will be taking place during migratory
birds nesting season and whether nesting birds could be impacted
and whether project activities, such as tree cutting, or vegetation
removal could cause damage to migratory birds and/or their nests?
40
The Migratory Bird Convention Act provides a description of
a Migratory bird and includes the sperm, eggs, embryos, tissue
cultures, and parts of the bird.
A Nest means the nest of a migratory bird and includes parts of the
nest.
Will the project cause a
change to wildlife species, or
residences or critical habitats
of wildlife species as defined
in subsection 2(1) of
the Species at Risk Act that are
listed in Schedule 1 of that
Act?*
Please select “Yes”, “No”, or “Unknown”.
The question is asking if the project will result in a change or cause an
impact to wildlife species, or the residences or critical habitats of
wildlife species listed in the Species at Risk Act. Changes could include
alteration, disturbance, or destruction of wildlife species or habitat
that could result in impacts to individuals of a species or that could
have impacts at the population level.
A list of species can be found here using the Species at Risk Public
Registry.
Wildlife Species as defined in the Species at Risk Act refers to a
species, subspecies, variety or geographically or genetically distinct
population of animal, plant, or other organism, other than a
bacterium or virus, that is wild by nature and (a) is native to Canada;
or (b) has extended its range into Canada without human
intervention and has been present in Canada for at least 50 years.
Residence means a dwelling-place, such as a den, nest, or other
similar area or place, that is occupied or habitually occupied by one
or more individuals during all or part of their life cycles, including
breeding, rearing, staging, wintering, feeding, or hibernating.
Critical Habitat means the habitat that is necessary for the survival or
recovery of a listed wildlife species and that is identified as the
species’ critical habitat in the recovery strategy or in an action plan
for the species.
Is your project within 10 km of
an Indian Residential School
Site?*

Please select “Yes”, “No”, or “Unknown”.
If the project is located within 10 km of an Indian Residential School
site, please select “Yes”. If the project is taking place on an Indian
Residential School site or within a residential school building, please
select “Yes”.
Given the recent discoveries of unmarked graves across Canada, HICC
is looking to identify, at an early stage, any projects that may fit into
this category to ensure these projects are identified.
A list of Indian Residential School sites in Canada is available
at Recognized Indian Residential Schools.
Press “Save” and “Continue” once you have completed this section.
 
41
Environmental Assessment (EA) Questionnaire
Is the project located on
federal lands?*
Please select “Yes” or “No”
This question is asking if the project or any of its components or
project activities are located on federal lands and Indian Reserve
lands.
A Federal land (defined in the Impact Assessment Act),
a. lands that belong to Canada, that Canada has the power to
dispose of, and all waters on and airspace above those lands,
other than lands under the administration and control of the
Commissioner of Yukon, the Northwest Territories or
Nunavut;
b. the following lands and areas: (i) the internal waters of
Canada, in any area of the sea not within a province, (ii) the
territorial sea of Canada, in any area of the sea not within a
province, (iii) the exclusive economic zone of Canada, and (iv)
the continental shelf of Canada; and
c. reserves, surrendered lands and any other lands that are set
apart for the use and benefit of a band and that are subject to
the Indian Act, and all waters on and airspace above those
reserves or lands.
Examples of federal lands are: Department of National Defense lands,
Transport Canada lands, Port Authority lands, Parks Canada lands, and
Indian Reserve lands. All National Wildlife Areas are federal lands. A
Migratory Bird Sanctuary, or Marine Protected Area could be federal
land, but this depends on the who owns the lands and may need to be
investigated further by federal authorities.
If “Yes” is selected, is the
project taking place on Indian
Reserve Lands?*
Please select “Yes” or “No”.

If “Yes” is selected, provide
name of reserve and land
code (if applicable).*
When a project is taking place on Indian Reserve lands the next sub
question asks for the name of the reserve and if there is a land code
associated with the reserve.
Does the project involve
construction of a new
physical asset or the
expansion of an existing
physical asset (vehicle
storage, expansion/new
utilities infrastructure)?*
Please select “Yes”, “No”.
The question asks if the project will result in a new physical asset (I.e.,
new physical infrastructure of any kind) or if there will be an expansion
of an existing physical asset.
The Keyhole Markup Language (.kml) files that you upload on the
Project Details page should show the new build location(s) or area of
expansion.
42
Construction of a new physical asset is the development of new
infrastructure, which can include the construction of a new building or
a new park or new roadway.
Expansion of existing infrastructure includes widening an existing
roadway, construction of an addition to an existing building that
increases the building footprint, or construction of a new level on an
existing building. If the building is being expanded only vertically
(adding another level to an existing building) please indicate the
expansion is vertical. Any expansion that results in an increase in the
footprint of an existing building is considered an expansion.
If “Yes” is selected, indicate
the size of project area in
m².*

Does the project involve
vegetation clearing?*

Please select “Yes”, “No”.
Vegetation clearing is the intensive removal of undisturbed vegetation
including trees, stumps, logs, bush, shrubs, and grasses, including tree
root systems, and requires the use of heavy equipment or industrial
machinery for clearing and grubbing an area. This does not include
removing vegetation in previously developed areas, manicured lawn
or turf areas, or grassed ditches.
Developed land is land that was previously cleared of vegetation and
used for human purposes. Developed lands include highly developed
urban areas, rural areas that were previously cleared and used for
agricultural purposes, brownfields, railways, or road right of way.
Vegetation removal includes the removal of lawn or turfed areas or
grass from ditches and is not considered vegetation clearing.
Vegetation removal is less intensive than vegetation clearing as it is
not occurring in an undisturbed or an undeveloped area.
If “Yes” is selected, indicate
the size of area to be cleared
of vegetation in m².*
Is the project on developed
or disturbed land?*

Please select “Yes” or “No”.
The question asks if the project, or any associated project works, will
take place on undeveloped or undisturbed lands.
A Developed land is land that was previously cleared of vegetation and
used for human purposes. Developed lands include highly developed
urban areas, rural areas that were previously cleared and used for
agricultural purposes, brownfields, railways, or road right of way.
An Undeveloped land is land not cleared of vegetation, in a natural
state, not currently used for human purposes. Undeveloped lands 
43
include undeveloped shorelines, riverbanks or gullies, grasslands,
forested areas, and scrub/brush areas.
A Disturbed land is land that has been altered by humans and includes
physical disturbance of the surface layer.
An Undisturbed land is land in its natural state and not currently used
for human purposes.
Does the project involve
excavation?*

Please select “Yes”, “No”.
The question asks if the project, or any associated project works, will
require excavation or digging.
Typically, excavation requires the use of heavy machinery to move
earth and prepare a construction site.
If the project requires the use of heavy machinery to excavate select
“Yes”. If the project does not require the use of heavy machinery to
excavate select “No”.
If “Yes” is selected, indicate
the size of area to be
excavated in m².*
Does the project involve
works in water (includes
streams, rivers, lakes, pond,
wetlands)?*
Describe the nature of the works in water.
Works in water include: shoreline work below the high-water mark,
installation of underwater cable or piping, blasting
shoreline/embankment areas, installation of bridge or dock piers inwater, and the placement of fill in a waterbody.
Water includes the following types of water bodies: a lake, a canal, a
reservoir, an ocean, a river and its tributaries and a wetland, up to the
annual high-water mark, but does not include a sewage or waste
treatment lagoon, a mine tailings pond, an artificial irrigation pond, a
dugout or a ditch that does not contain fish habitat as defined in
subsection 2(1) of the Fisheries Act.
When there are works in water the sub-question asks for a description
of the work or activities that will take place in water.
If “Yes” is selected please
explain.
Does the project involve
works within 15 m of a water
body?*
Please select “Yes” or “No”.
The question asks if the project, or any associated project works such
as vegetation clearing for site preparation, a new outbuilding,
installation of fencing, or creating a new walking trail will take place
within 15m of a water body.
A water body can include a lake, a canal, a reservoir, an ocean, a river
and its tributaries and a wetland, up to the annual high-water mark,
but does not include a sewage or waste treatment lagoon, a mine
tailings pond, an artificial irrigation pond, a dugout or a ditch that does 
44
not contain fish habitat as defined in subsection 2(1) of the Fisheries
Act.
Does the project involve the
operation of a motor vehicle
on water (water taxi, vessel,
ferry, etc.)?*
Please select “Yes” or “No”.
The question asks if the project will involve the use or operation of a
motorized vehicle (e.g., boat or vessel) on water.
If “Yes” is selected, attach
route that the vehicle will
travel (.kml) AND provide
details on the vehicle use,
schedule, route, and area of
travel.
The sub-question asks for details to be included on the route and use
of the vehicle.
Attach the KML file by selecting “Upload Document”. Please name
(e.g., “KML file for Motor Vehicle on Water for Transit Systemin
Village-Creek, Province of XX”) and attach your file.
Has the project been
assessed by the Impact
Assessment Agency of
Canada to determine if there
is a federal EA requirement?*
Please select “Yes”, or “No”.
If “Yes” is selected, provide
the response from the Impact
Assessment Agency of
Canada explaining their
review.*
The question asks if the project was submitted to the Impact
Assessment Agency of Canada to review for any requirements related
to the Impact Assessment Act.
Does the project have any
requirements related to the
Impact Assessment Act?*
Please select “Yes”, “No” or “Maybe”.
The Impact Assessment Agency of Canada is responsible for
conducting impact assessments under the Impact Assessment Act. This
Act outlines a process for assessing the impacts of major projects and
projects carried out on federal lands or outside of Canada.
Projects that are listed on the Physical Activities Regulations of the
Impact Assessment Act are very likely to have a requirement under the
Impact Assessment Act.
Projects that are taking place on federal lands (lands belonging to
Canada such as Department of Defense land, Transport Canada land,
and Indian Reserve land) could have a requirement under Section 82
of the Impact Assessment Act.
Is the project described on
the Physical Activities
Regulations of the Impact
Assessment Act?*
Please select “Yes”, “No”, or “Maybe”.
The question asks if the project is listed on the Physical Activities
Regulations of the Impact Assessment Act. Projects that are listed on
the Physical Activities Regulations of the Impact Assessment Act are
very likely to have a requirement under the Impact Assessment Act.
If “Yes” is selected, the subquestion asks if have you
Please select “Yes” or “No”. 
45
provided the Impact
Assessment Agency of
Canada with a project
description as per Section
10(1) of the Impact
Assessment Act?
The sub-question asks if a project description was provided to the
Impact Assessment Agency of Canada for their review.
Does the project have any
requirements related to a
Modern Treaty, SelfGovernment Agreement, or
Northern EA Regime?*
Please select “Yes” or “No”
Modern treaties negotiated with Indigenous peoples (after 1975) may
include consultation and participation requirements, ownership of
lands, wildlife harvesting rights, financial settlements, participation in
land use and management in specific areas, self-government, resource
revenue sharing and measures to participate in the Canadian
economy, and preparations for when the agreement takes effect (such
as implementation planning).
Different forms of governance or self-government have been
negotiated in Canada. One example is the Nunavut Agreement, a
modern treaty where the self-government aspirations of Inuit are
expressed through public government.
The environmental assessment process in Canada's northern
territories is based on a unique co-management approach, rooted in
the legal and cultural frameworks of land claims agreements with
Indigenous peoples.
Additional information on Indigenous rights is available at Crown
Indigenous Relations and Northern Affairs Canada.
If “Yes” is selected, please
identify which Modern
Treaty, Self-Government
Agreement, or Northern EA
Regime from the drop-down
menu options.*
Choose an option from the drop-down menu.
Has there been engagement
with Indigenous groups about
the project?*
Please select “Yes”, “No”, or “Unknown”.
If “Yes” is selected, which
Indigenous groups have been
notified of the project?
Please list all groups and
attach any records of
consultation, meeting
minutes, letters of support, a
Attach the documents using “Upload Document”. Please name
(e.g., “Record of Consultation for Transit Solution in Village-Creek,
Province of XX”) and attach your file.
This question asks if any Indigenous peoples, including Indigenous
peoples, First Nations, Metis, and Inuit were provided with
information about the project. Were any groups contacted through
email, letter, or phone calls? Were there any meetings held with
Indigenous peoples to get their feedback and opinions on the project? 
46
Band Council Resolution, or
consultation transcripts.*
If yes, please provide all related documentation. Attach any records of
consultation, meeting minutes, letters of support, a Band Council
Resolution, or consultation transcripts.
Have concerns been raised by
Indigenous groups about the
project?*
Please select “Yes”, or “No”.
This question asks if any Indigenous peoples (First Nations, Metis, or
Inuit) indicated they had concerns or were opposed to the project or
to any of the project components or activities. Provide context to the
issues raised.
Does the recipient attest that
all concerns have been
addressed?*
Please select “Yes”, or “No” or “N/A”
If “Yes” is selected, using the
text box justify how concerns
have been addressed (refer
to where concerns have been
addressed in consultation
record).*
This question asks to confirm that all concerns raised by Indigenous
peoples were resolved in some way. If all concerns were addressed,
please answer “Yes”. If there are outstanding concerns, please answer
“No”. If the applicant is unsure if concerns are addressed, please
answer “No”.
 
47
Annex C: How to Create a KML File
You can use Google Earth to create a Keyhole Markup Language (KML) file to visualize and share
geographic information about your project. Create your KML using either Google Earth Web (requires a
Google account), or Google Earth Pro for desktop (requires download and installation). Both sets of
instructions are included below.
REVIEW THESE KEY POINTS BEFORE BEGINNING
• The KML should provide an accurate and detailed representation of the project’s location, size,
components and works; placemarks and/or pins alone are insufficient.
• Use multiple line and/or polygon features for different project components. To differentiate
components, title each feature with a descriptive name.
• If the project includes multiple phases, identify each phase in the feature’s title. If HICC is only
funding specific phases of the project, note that information in the feature’s description.
• For most projects, all features should be submitted in a single KML file.
Geospatial Feature Examples of Project Components
Line • Roads
• Sewer lines
• Trails
• Railways
• Pipelines
• Transmission lines
Polygon • Building footprints
• Vegetation cuts
• Sewer/wastewater lagoons
• Culverts/bridges
INSTRUCTIONS FOR GOOGLE EARTH WEB:
Step 1: Create a new project
Click ‘Launch Earth’. Click ‘+ New’. Choose whether to save as a Drive project (to Google drive) or as a
Local KML file (stored on the browser or device). Click ‘Create’. In the left-hand panel, hover over file
name, click the three dots, select rename and enter a title for your project. Avoid using special characters
like \ / ( ) { } [ ] : * ? " <>| # % $ & !
Step 2: Navigate to the desired location
Navigate to the desired location on the map by typing an address in the search bar (top-left corner)
and/or by dragging the map to the correct location. Zoom in and out using the minus and plus signs in the
bottom-right corner.
Step 3: Create a KML to display project components
Add a Placemark: To optionally mark the project’s central location, click the ‘Add placemark’ icon or
‘Add’ > ‘Placemark’ in the menu. A placemark will appear on the map, and you can move it to the desired 
48
location or specify coordinates. In the right-hand panel, enter a title for the placemark and optional
description.
Draw Lines (Paths) or Polygons: To draw a line or polygon, click ‘Add path or polygon’ icon or ‘Add’ >
‘Path or polygon’ in the menu. Select a starting point and click on the map to add points that will
delineate the path or polygon. For lines, click ‘Done’. For both lines and polygons, click ‘Save to project’.
In the right-hand panel, enter the title of the component and an optional description. Click the paint
bucket icon to adjust the colour.
Step 4: Adjust the features
Click on any placemark, line, or polygon in the left-hand panel to adjust the feature. A right-hand panel
will appear. Click ‘Edit’. You can edit the title, description, and colour. You can also adjust the points on
the map that make up the feature. Below is an example project:
Step 5: Save your KML file
To export your project as a KML file, click on the three dots next to your project title in the left-hand
panel, then choose ‘Export as KML file’. The KML file will appear in your Downloads folder. It can be
shared or uploaded to other mapping platforms.
49
INSTRUCTIONS FOR GOOGLE EARTH DESKTOP:
Step 1: Create a project folder
Right-click on ‘My Places’ or ‘Temporary Places’ > ‘Add’ > ‘Folder’. Name the folder using the project title
and click ‘OK’.
Step 2: Navigate to the desired location
Navigate to the desired location on the map by typing an address in the search bar or by clicking and
dragging in the map viewer. Click ‘View’ > ‘Reset’ > ‘Tilt and Compass’. Zoom in and out using the minus
and plus signs in the top-right corner.
Step 3: Create a KML to display project components
Add a Placemark (Point): To optionally mark the project’s central location, click ‘Add Placemark’ in the
toolbar or ‘Add’ > ‘Placemark’ in the menu. Enter a name for the placemark and adjust its location on the
map by dragging the yellow pin or by specifying known latitude and longitude coordinates. In the
‘Properties’ panel, enter a title for the placemark and optional description. Adjust the style and colour.
Draw Lines (Paths) or Polygons: To draw project components, use the ‘Add Path’ and/or ‘Add Polygon’
buttons or ‘Add’ > ‘Path or polygon’ in the menu. Click on the map to create points. You can click and drag
any point to adjust its location, click on the map to add a new point, or select a point and press the
‘Delete’ key to remove it. In the ‘Properties’ panel, enter a title for the project component and optional
description. Adjust the style and colour. Click ‘OK’ when finished.
Step 4: Adjust the features
Right-click on any placemark, path, or polygon to open the ‘Properties’ window. You can edit the label,
description and colour, and check measurements. You can also adjust the points on the map that make
up the feature. Below is an example project:
50
Step 5: Save your KML file
Right-click the project folder, then ‘Save Place As’. Name your file. Avoid using special characters like \ / ( )
{ } [ ] : * ? " <>| # % $ & ! Select the KML format and save to your computer. Your KML file can be can be
shared or uploaded to other mapping platforms.
 '''

In [39]:
AHfTSNelYuhb8eq_text_1 = '''© 2023 International Monetary Fund
RESILIENCE AND SUSTAINABILITY TRUST—2022
CONTRIBUTION AGREEMENTS WITH AUSTRALIA,
CANADA, CHINA, GERMANY, JAPAN AND SPAIN
IMF staff regularly produces papers proposing new IMF policies, exploring options for
reform, or reviewing existing IMF policies and operations. The Report prepared by IMF
staff and completed on January 3, 2023, has been released.
The staff report was issued to the Executive Board for information. The report was
prepared by IMF staff. The views expressed in this paper are those of the IMF staff and
do not necessarily represent the views of the IMF's Executive Board.
The IMF’s transparency policy allows for the deletion of market-sensitive information
and premature disclosure of the authorities’ policy intentions in published staff reports
and other documents.
Electronic copies of IMF Policy Papers
are available to the public from
http://www.imf.org/external/pp/ppindex.aspx
International Monetary Fund
Washington, D.C.
February 2023
RESILIENCE AND SUSTAINABILITY TRUST―2022
CONTRIBUTION AGREEMENTS WITH AUSTRALIA, CANADA,
CHINA, GERMANY, JAPAN, AND SPAIN
Approved By
Craig Beaumont and
Bernhard Steinki
Prepared by the Finance and Legal departments. The staff team was
led by Linda Kaltani (FIN) and Jonathan Swanepoel (LEG) and
comprised Phil de Imus, Ivetta Hakobyan, and Izabela Rutkowska (FIN),
Egbiri Egbiri, Stephanie Fontana-Raina, Eva Kabundu, Ioana Luca, and
Audrey Yiadom (LEG); production assistance by Vera Lochan (FIN).
INTRODUCTION
1. This paper presents for the information of Executive Directors the six Resilience
and Sustainability Trust (RST) contribution agreements that had been finalized with
contributors by the time of RST operationalization. Shortly after the Executive Board’s
April 2022 approval of the establishment of the RST, the Managing Director wrote to
35 potential contributors with strong external positions, seeking their contributions in a total
amount of at least SDR 33 billion.1 To date, the Fund has received pledges amounting to
SDR 30.4 billion from 17 members, of which SDR 25.3 billion are pledges to all three RST
accounts (i.e., the Loan Account (LA), Deposit Account (DA) and the Reserve Account (RA)) that
count toward the SDR 33 billion target, corresponding to 77 percent of the targeted
contributions (see Table 1). By the date of RST operationalization on October 12, 2022,
agreements had been finalized with six contributors.2, 3 Five members (or their institutions)
contributed through a Contribution Package to all three of the RST’s accounts in proportion to
the agreed ratios, while one member concluded a ‘standalone’ contribution to the DA and RA.4
1 See Decision No. 17231-(22/37), adopted April 13, 2022, and with effect from May 1, 2022.
2 Pursuant to Section III, Paragraph 2 of the Instrument to Establish the RST (the “RST Instrument”),
the Managing Director is authorized to enter into contribution agreements and to make the
necessary arrangements.
3 A few additional contribution agreements have since been finalized which will be circulated to the
Executive Board in a follow-up paper once a critical mass of further contributions have been
completed.
4 As specified in Section III, Paragraph 1(b) of the RST Instrument, each LA contributor is required to sign a
Contribution Package with three parts—the Loan Account contribution (borrowing agreement), a Reserve
Account contribution, and a Deposit Account contribution. Contributions to the RA and DA would be a
minimum of 2 percent and 20 percent, respectively, of a contributor’s commitment to the LA. The Trust may
also receive ‘standalone’ contributions to the RA and/or DA that are not unrelated to the LA contribution (see
paragraphs 88-89 in the Proposal to Establish A Resilience and Sustainability Trust).
January 3, 2023
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
2 INTERNATIONAL MONETARY FUND
Table 1. Status of Pledges to the Resilience and Sustainability Trust
(In SDR billions as of December 14, 2022)
2. The six concluded agreements provide for contributions in a total amount of about
SDR 15.3 billion across the three RST accounts. Altogether, the agreements provide approximately
SDR 8,613 million to the LA, approximately SDR 6,079 million to the DA and approximately SDR 631
million to the RA.5 Agreements were finalized as follows:
• a contribution agreement with the Government of Australia totaling SDR 927.2 million to all
three accounts of the RST, effective October 11, 2022;
• a contribution agreement with Canada, as represented by the Minister of Finance, totaling
SDR 1,387.6 million to all three accounts of the RST, effective September 27, 2022;
5 Finalized borrowing agreements represent ‘committed’ resources ready to be called for on-lending under Resilience
and Sustainability Facility (RSF) arrangements. Resources provided under finalized RA and DA agreements have
already been disbursed to the RST’s accounts, with the exception of Germany whose RA and DA contributions are
expected to be received by end-February 2023.
Country RST contribution1 SDR allocation
in 2021
 Contribution as
share of 2021 SDR
allocation
Status
1 Australia 0.9 6.3 15% Finalized agreement
2 Canada 1.4 10.6 13% Finalized agreement
3 China 6.0 29.2 21% Finalized agreement
4 France 3.0 19.3 16% Finalized agreement
5 Italy 1.9 14.4 13%
6 Japan 4.9 29.5 17% Finalized agreement for the initial 0.8 billion
7 Korea 0.9 8.2 11% Finalized agreement
8 Lithuania 0.085 0.4 20%
9 Luxembourg 0.253 1.3 20%
10 Malta 0.023 0.2 14%
11 Netherlands 1.2 8.4 15%
12 Oman 0.039 0.5 7%
13 Singapore 0.7 3.7 20%
14 Spain 1.4 9.1 16% Finalized agreement
15 United Kingdom 2.5 19.3 13%
Subtotal 25.3 15%
Standalone contributions2
16 Estonia 0.025 0.2 11%
17 Germany 5.0 25.5 20% Finalized agreement
Grand total 30.4 15%
Source: Country authorities; IMF staff estimates.
1
 The table reports the amounts pledged or contributed. When pledges are reported, this table shows the amount as pledged by each country. In
most of such cases, the pledge amount excludes the reserve account contribution, but at 2 percent of the loan contribution, its impact on the total
amount is small. For some countries, pledges are subject to domestic procedures, including budgetary approvals.
2
 Standalone contributions to the Reserve and/or Deposit Accounts, without a contribution to the Loan Account.
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 3
• a contribution agreement with the People’s Bank of China totaling SDR 6,000 million to all
three accounts of the RST, effective September 27, 2022;
• an agreement with the Federal Republic of Germany for contributions of the SDR equivalent of
Euro 5,700 million to the DA and Euro 600 million to the RA, effective October 12, 2022. This
agreement constitutes a ‘standalone’ contribution as no loan resources are being provided;
• a contribution agreement with the Government of Japan totaling SDR 798.0 million to all
three accounts of the RST, effective October 6, 2022, with the LA and DA agreements in the form
of a Note Purchase Agreement (NPA); and
• agreements with Spain, the first member to make its RST pledge effective with contributions to
the LA and DA by the Bank of Spain and a contribution to the RA by the Ministry for the
Economy and Digitalization—all effective September 8, 2022, totaling SDR 1,417.1 million.
TERMS OF CONTRIBUTION AGREEMENTS
3. Contribution agreements are all subject to uniform substantive terms. As envisaged in the
RST policy paper, agreements with contributors are subject to uniform key substantive terms. Annex I
summarizes these key financial terms as provided for in the RST Instrument. Key uniform terms include
denomination of all transactions in SDRs, the requirement to meet the minimum threshold relative to
committed loan resources of 20 and 2 percent, respectively, for the deposit and reserve contributions,6
the right to seek encashment of outstanding loan account and deposit contributions in case of a
contributor’s balance of payments needs or due to developments in a contributor’s reserves,7
outstanding loan and deposit account claims are remunerated at an interest rate that cannot exceed
the SDRi, and the investment of DA and RA resources under a uniform investment strategy. With the
exception of the standalone contribution from Germany, which features a 10-year maturity for the
contributions to both the DA and RA, all other agreements include the same terms for the timing of
drawings, repayments, and financial arrangements in case of a termination event. Individual
agreements are based on template agreements reflecting these key substantive terms.
4. Deviations from the standardized template are very minor. Deviations from the templates
reflect legal and operational requirements of contributors with no implications for the uniformity of
key substantive terms. For example, as contemplated in the RST Board paper, the three components of
the contribution package could be signed for the member by one counterparty or broken up into
separate agreements to be signed by different institutions. In the agreements with Canada, Germany,
Japan, and Spain, the SDR denominated contributions to the RA (and DA in case of Germany) are
6 This requirement is not applicable to standalone contributions of deposits and/or reserves without loan resources.
7 Where the contributor is a not the member, this refers to the balance of payments and reserves position of the
relevant member per se. 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
4 INTERNATIONAL MONETARY FUND
calculated in reference to an underlying commitment expressed in currency, and contributions are
disbursed in currency, while under the agreements with Australia and China, the commitments are
expressed in SDRs, and contributions are also disbursed in SDRs. The number of days for the
notification period of the intention to draw on loan commitments varies modestly between five and
seven days. In the case of Japan, the loan and deposit contributions are in the form of a note purchase
agreement, with terms consistent with the key substantive terms of template agreements. Agreements
with Australia and Canada include language regarding their treatment under domestic law.8
5. Table 2 summarizes the key financial terms of the finalized agreements and Attachments
1-8 present the full text of the agreements as executed.
Table 2. Key Financial Terms of RST Contribution Agreements by Contributing Country
(In SDR million)
8 See Attachment I Section IV.1 and Attachment II Section IV.1. Both agreements also use “will” instead of “shall”
(used in the templates) with no impact on the substance of the provisions.
Australia Canada China Germany1 Japan Spain
Effectiveness date2 10/11/2022 9/27/2022 9/27/2022 10/12/2022 10/6/2022 9/8/2022
Amount
Loan Account 760.0 1,137.4 4,900.0 n/a 654.1 1,161.6
Deposit Account 152.0 227.5 1,000.0 4,336.1 130.8 232.3
Reserve Account 15.2 22.8 100.0 456.4 13.1 23.2
Media3
Loan Account SDR SDR SDR n/a SDR SDR
Deposit Account SDR or other SDR or other SDR or other EUR SDR or other SDR
Reserve Account SDR or other SDR or other SDR or other EUR SDR or other EUR
Remuneration
Loan Account SDRi SDRi SDRi n/a SDRi SDRi
Deposit Account SDRi SDRi SDRi SDRi SDRi SDRi
Reserve Account Not remunerated Not remunerated Not remunerated Not remunerated Not remunerated Not remunerated
Type of Agreement
Loan Account Loan Agreement Loan Agreement Loan Agreement n/a
Note Purchase
Agreement Loan Agreement
Deposit Account Deposit Agreement Deposit Agreement Deposit Agreement Deposit Agreement Note Agreement Deposit Agreement
Reserve Account Reserve Agreement Reserve Agreement Reserve Agreement Reserve Agreement Reserve Agreement Reserve Agreement
Encashment4 Yes Yes Yes5 Yes Yes Yes
Source: IMF Finance Department.
4 The right to encash applies to loan and deposit resources. For the LA, to fund a request for encashment of an outstanding loan the Trustee will call on resources
committed by other contributors to the RST's Loan Account. Contributors will restore an amount equivalent to the encashed resources as soon as practicable in light of
favorable developments in its balance of payments and reserve position.
5 On November 7, 2022, China waived its encashment right under paragraph 6 of the borrowing agreement to early repayment of any outstanding claims on the RST's
Loan Account until the total amount of committed RST loan resources reaches SDR 11.8 billion.
1 Germany committed to provide stand-alone contributions to the RST's Deposit and Reserve accounts. The deposit of EUR 5.7 billion and the reserve contribution of
EUR 600 million will be delivered by end-February, 2023, and their SDR value is estimated at the exchange rate of the agreement's effectiveness date of October 12,
2022. The actual SDR value of these contributions will be determined two Fund business days prior to the value date of the contributions.
2 The five LA agreements became effective when the Managing Director notified the Executive Board that the RST became operational on October 12, 2022.
3 SDR or freely usable currency in which the contribution is to be disbursed as laid out in the agreement. All contributions and transactions are denominated in SDR.
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 5
CONCLUSION
6. These six finalized agreements deliver critical resources for the operationalization of
the RST and represent an important step towards the adequate funding of the RST. Additional
contributions are needed to ensure that the RST is adequately resourced to meet demand from
eligible members in the medium term. Crucially, the uniformity of key substantive terms allows for a
streamlined process in finalizing contribution agreements that promises to translate into greater
operational efficiency during the discussion of contribution agreements. Discussions with other
contributors are ongoing, and staff will keep the Executive Board updated on new developments.
Staff will periodically circulate to the Executive Board additional contribution agreements as they are
finalized. 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
6 INTERNATIONAL MONETARY FUND
Annex I. Key Terms of RST Agreements
Loan Account (LA)
Denomination All transactions denominated in Special Drawing Rights (SDR).
Media SDR or freely usable currencies. For payments by the Trust, other media can be
agreed between the Trustee and contributors.
Drawdown
period for loan
account
November 30, 2030 (can be shorter on an exceptional basis). Drawing after
November 30, 2030 is permitted to cover encashment calls for drawings that
finance RST lending prior to November 30, 2030.
Loan account
drawings
Each drawing will have its own financial profile, i.e., maturity and repayment
schedule, consistent with the repayment schedule for RSF loan disbursements.
Maturity Maximum maturity for each drawing is 20 years from the date the claim was
originated (i.e., when resources were drawn).
Principal
repayments
For each drawing, repayment will follow the same schedule as that for RSF loans
(i.e., 20 equal semi-annual instalments beginning 10.5 years from the date of
drawing).
Interest
payments
Interest on each outstanding drawing will be calculated and accrued daily at the
applicable SDR interest rate (SDRi), or lower if agreed with the contributor, during
the Fund’s financial quarter and paid promptly after each financial quarter. For
example, for the first financial quarter, interest on each drawing would accrue
from May 1 to July 31, and the interest payment would be made on promptly
after July 31.1
Encashment Loan claims are encashable upon a representation by the contributor of a balance
of payments (BoP) or reserves need. At the time of the BoP or reserves need
representation, any undrawn loan commitment will become unavailable for
drawings. The repayment of all outstanding drawn amounts will be made from
resources provided from calls under other contributors’ LA contribution
agreements. These calls would take on the same repayment terms as those of
each encashed drawing. As soon as the contributor’s BoP/reserve problem is
resolved/mitigated as determined under conditions set out in the RST Instrument,
the undrawn loan commitment amount under the LA agreement will become
available for drawing.
Transfer of
claims
A contributor’s claims can be transferred to other RST contributors, other IMF
members, and prescribed holders of SDRs.
Early
repayment
The Trustee may repay part or all of any drawing at any time prior to the maturity
of such drawing in case of early repayment by RST borrowers.
1 This aligns the timing of LA interest payments with that for interest and charges on SDR holdings.
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 7
Deposit Account (DA)
Denomination All transactions denominated in Special Drawing Rights (SDR).
Media SDR or freely usable currencies. For payments by the Trust, other media can be
agreed between the Trustee and contributors.
Due date The deposit shall be made promptly on or after the effectiveness of the
Contribution Agreement.
Maturity Maturity of DA contributions (excluding stand-alone DA contributions) is
November 30, 2050. For stand-alone contributions, the maturity date is agreed
between the parties where a maturity of 10 years would be preferable.
Principal
repayments
At the time of maturity, the principal amount of the original deposit will be
returned to a contributor, net of any attributed losses, and the remaining net
investment amount attributed to this specific contributor will be transferred to
their RA claim.
Interest
payments
Interest is calculated and accrued daily on the DA’s principal amount at the SDRi
(or lower if agreed with the contributor) and paid on a quarterly basis promptly
after April 30, July 31, October 31, and January 31 of each year (similar to the
interest payments on drawings under the LA). Interest would be paid regularly
from accumulated balances in the DA.
Investment
strategy
Resources will be invested in liquid, high-quality instruments based on a robust
investment strategy that minimizes investment risk, approved and regularly
reviewed by the Executive Board of the IMF.
Encashment Contributors can encash their DA claims before maturity in case of representation
of BoP and reserve needs. The encashment of DA claims would be funded from
liquidation proceeds from the contributor’s relative share in the market value of
the pooled invested DA assets (which could be lower than the principal of the
investment depending on market value of the portfolio at the time of
encashment). The encashing contributor commits to reconstitute its DA deposit as
soon as it no longer has a BoP/reserve need.
Transfer of
claims
A contributor’s claims can be transferred to other RST contributors, other IMF
members, and prescribed holders of SDRs.
Early
repayment
The Trustee may repay part or all of the DA contributions any time prior to their
maturity as agreed between the Trustee and the contributor.
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
8 INTERNATIONAL MONETARY FUND
Reserve Account (RA)
Denomination All transactions denominated in Special Drawing Rights (SDR).
Media SDR or freely usable currencies. For payments by the Trust, other media can be
agreed between the Trustee and contributors.
Due date The reserve contribution shall be made promptly on or after the effectiveness of
the Contribution Agreement.
Maturity No fixed maturity date for contributions to RA. For stand-alone contributions, the
maturity date may be agreed between the parties where a maturity of 10 years
would be preferable.
Principal
repayments
Upon the liquidation of the Trust, the Trustee will distribute to contributors their
shares in the resources of the Reserve Account. The Trustee can decide on early
distribution.
Interest
payments
None.
Investment
strategy
Resources will be invested in liquid, high-quality instruments based on a robust
investment strategy that minimizes investment risk, approved and regularly
reviewed by the Executive Board of the IMF.
Encashment No right to seek encashment.
Transfer of
claims
A contributor’s claims can be transferred to other RST contributors, other IMF
members, and prescribed holders of SDRs.
Early
repayment
The Trustee may repay part of the RA following the Trustee’s decision to distribute
a portion of the balances in the Reserve Account of the RST. Contributors will
receive part of this distribution in proportion to their shares in the Reserve
Account.
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 9
Attachment I. Contribution Agreement between the
Government of Australia and the International Monetary Fund,
as Trustee of the Resilience and Sustainability Trust
WHEREAS the International Monetary Fund (the “Fund”) has established the Resilience and
Sustainability Trust (the “Trust”) to assist in fulfilling the purposes of the Fund by providing loans
under the Resilience and Sustainability Facility of the Trust (the “RSF”) to eligible Fund members to
enhance their economic resilience and sustainability;
WHEREAS, Australia has committed to contribute financial resources to the loan account (“Loan
Account”), the deposit account (the “Deposit Account”) and the reserve account (the “Reserve
Account”) of the Trust in accordance with the provisions of the Instrument to Establish the Resilience
and Sustainability Trust (the “RST Instrument”);
WHEREAS, to fulfill Australia’s commitment, the Government of Australia (the “Contributor”) and the
Fund, as Trustee (the “Trustee”) of the Trust, have reached understandings on a borrowing
agreement (the “Borrowing Agreement”), a deposit agreement (the “Deposit Agreement”), and a
reserve contribution agreement (the “Reserve Contribution Agreement”), jointly referred to as the
Contribution Agreement;
NOW, THEREFORE, the Contributor and the Fund, as Trustee of the Trust, mutually decide on the
terms and conditions of these contributions as follows:
I. BORROWING AGREEMENT
1. Amount. The Contributor decides to lend an amount up to the equivalent of SDR 760
million to the Loan Account of the Trust.
2. Drawings.
(a) The Trustee may make drawings under this Borrowing Agreement at any time during the
period from the effective date of this Borrowing Agreement, as set forth in paragraph 3 of the Final
Provisions below, through November 30, 2030 (the “drawdown period”) to finance loan
disbursements under the RSF.
(b) The Trustee may make drawings under this Borrowing Agreement to finance early
repayment of outstanding Trust borrowing under other borrowing agreements with the Loan
Account pursuant to Section IV, Paragraph 3(b) of the RST Instrument for as long as claims under
such borrowing agreements remain outstanding.
(c) The Trustee’s drawings under this Borrowing Agreement will be suspended following the
Contributor’s request for early repayment under paragraph 6 of this Borrowing Agreement or when
Australia is no longer included for transfers in the Fund’s Financial Transactions Plan, in accordance
with Section IV, paragraph 4 of the RST Instrument. 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
10 INTERNATIONAL MONETARY FUND
(d) If Australia does not provide its consent to a proposed amendment to the RST Instrument
pursuant to Section X, paragraph 2(a) of the RST Instrument, further drawings under this Borrowing
Agreement to finance any new commitment of loans under the RSF that is made after the
effectiveness of the proposed amendment will be suspended.
(e) If any installment of principal or interest is not paid to the Contributor within a period of ten
Fund business days after its due date, the Trustee will not make further drawings under this
Borrowing Agreement pending consultations with the Contributor on the matter. However, the
Trustee may resume drawings under this Borrowing Agreement once all arrears to the Contributor
have been discharged.
(f) Unless otherwise mutually decided between the Contributor and the Trustee, the Trustee will
give the Contributor at least five (Canberra and Sydney) business days’ notice of its intention to
draw and will provide payment instructions at least two (Fund) business days prior to the value date
of the transaction by SWIFT.
3. Denomination, Media, and Claim Certificate.
(a) The amount of each drawing will be denominated in SDRs. The amount will be paid by the
Contributor on the value date specified in the Trustee’s notice by transfer of SDRs to an account
specified by the Trustee.
(b) Upon request, the Trustee will issue to the Contributor a non-negotiable certificate evidencing
its claim on the Trust resulting from a drawing outstanding under this Borrowing Agreement.
4. Interest.
(a) Each drawing will bear interest at the SDR interest rate established by the Fund pursuant to
Article XX, Section 3 of the Fund’s Articles of Agreement.
(b) The amount of interest payable in respect of each drawing will be calculated on an actual
day basis and will be paid on all outstanding drawings under this Borrowing Agreement on a
quarterly basis promptly after January 31, April 30, July 31, and October 31 of each year.
5. Maturity and Repayment. Except as otherwise provided in this Borrowing Agreement, each
drawing will be repaid in accordance with the repayment schedule for the loan disbursement under
the RSF for which it was drawn, and at the time of each drawing, the Trustee will provide information
to the Contributor on the specific repayment schedule for the amount drawn.
6. Encashment. The Contributor will be permitted to seek early repayment of all or part of the
principal amount of its outstanding claims on the Loan Account of the Trust pursuant to Section IV,
paragraph 3(b) of the RST Instrument if the Contributor represents that the balance of payments and
reserve position of Australia justifies the early repayment, and the Trustee, having given this
representation the overwhelming benefit of any doubt, concurs. Early repayment is in all cases
subject to the availability of resources under borrowing agreements to the Loan Account of the Trust.
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 11
7. Early Repayment. The Trustee may repay any drawing or part thereof at any time in
advance of maturity upon its receipt of early repayment of the corresponding loan disbursement
under the RSF, provided that the Trustee will give notice to the Contributor at least five (5) (Fund)
business days prior to such advance repayment.
8. Modality of Principal Repayment and Interest Payments.
(a) Unless otherwise mutually decided between the Trustee and the Contributor, payments of
principal and interest by the Trustee will be made in SDRs.
(b) Payments in SDRs by the Trustee will be made by crediting Australia’s holdings account in
the Special Drawing Rights Department. If the due date is not a business day of the Fund, the
payment will be made on the following business day of the Fund.
(c) Payments by the Trustee in currencies will be made to an account as mutually decided
between the Trustee and the Contributor. If the due date is not a business day of the Fund or the
Target system or the domestic markets of the non-euro currencies included in the SDR basket, the
payment will be made on the following business day for the Fund, the Target system and the
domestic markets of the non-euro currencies included in the SDR basket.
9. Transfer of Claims.
(a) The Contributor may transfer at any time all or part of any claim to any member of the Fund,
to the central bank or other fiscal agency designated by such a member for purposes of Article V,
Section 1 of the Fund’s Articles of Agreement or to any official entity that has been prescribed as a
holder of SDRs pursuant to Article XVII, Section 3 of the Fund’s Articles of Agreement.
(b) The transferee will acquire all the benefits of the Contributor under this Borrowing Agreement
with respect to repayment of principal and interest on the transferred claim, except that the transferee
will be permitted to request early repayment under paragraph 6 of this Borrowing Agreement only if
it is a member or the central bank or other fiscal agency of a member and, at the time of transfer, the
member’s balance of payments and reserve position was considered sufficiently strong in the opinion
of the Fund for its currency to be usable in transfers under the Fund’s Financial Transactions Plan.
10. Exchange Rates. Unless otherwise mutually decided between the Trustee and the
Contributor, all transfers, exchanges, and payments of principal and interest in currencies will be
made at the exchange rates for the relevant currencies in terms of the SDR established by the Fund
pursuant to Article XIX, Section 7(a) of the Fund’s Articles of Agreement and the rules and
regulations of the Fund thereunder for the second (Fund) business day before the value date of the
transfer, exchange or payment.
11. SDR Basket. If the Fund changes the currency composition or currency weights of the SDR
basket, all transfers, exchanges, and payments of principal and interest made two or more (Fund)
business days after the effective date of the change will be made on the basis of the new SDR
basket.
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
12 INTERNATIONAL MONETARY FUND
***************************
II. DEPOSIT AGREEMENT
1. Amount of the Deposit. The Contributor will make a deposit (the “Deposit”) equivalent to
SDR 152 million with the Deposit Account of the Trust (the “Original Amount of the Deposit
2. Denomination and Media.
(a) The Deposit will be denominated in SDRs.
(b) The Deposit will be made in either SDRs or a freely usable currency, as determined by the
Contributor.
(c) The Deposit will be made promptly on or after the effectiveness of the Contribution
Agreement.
3. Maturity. The Deposit will mature on November 30, 2050 (the “Maturity Date”).
4. Investment Strategy and Risk Disclosure.
(a) The resources from the Deposit will be invested by the Trustee in accordance with the
guidelines adopted by the Trustee pursuant to Section VI, paragraph 4(a) of the RST Instrument.
(b) The assets of the Deposit Account are exposed to market and credit risk which could result
in potential losses to the Original Amount of the Deposit.
5. Notional Value of the Deposit. For the purposes set out in paragraphs 6, 7, 8, and 9 of this
Deposit Agreement, the Trustee will calculate the notional value of the Deposit based on the
Original Amount of the Deposit and subsequently any attributed investment earnings or losses,
interest payments made to the Contributor pursuant to paragraph 6 of this Deposit Agreement, if
any, and transfers pursuant to paragraphs 8, 9, 10 or 11 of this Deposit Agreement or pursuant to
Section VI, paragraphs 1(c) and 4(b)(2) of the RST Instrument (“Notional Value”).
6. Interest.
(a) The Trustee will pay the Contributor interest on the Original Amount of the Deposit less any
amounts repaid to the Contributor under paragraphs 8 or 9 or transferred by the Contributor under
paragraph 11 of this Deposit Agreement at the SDR interest rate established by the Fund pursuant
to Article XX, Section 3 of the Fund’s Articles of Agreement. Interest will accrue daily and will be paid
promptly by the Trustee on a quarterly basis [after January 31, April 30, July 31 and October 31 of
each year]. Any payment of interest will be limited to the Notional Value of the Deposit.
(b) Following any full repayment of the Deposit in accordance with paragraphs 7 and 9 of this
Deposit Agreement, all remaining interest under paragraph 6(a) above, will be paid to the
Contributor at the end of the Fund financial quarter in which repayment was made.
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 13
7. Repayment upon Maturity. As soon as practicable following the Maturity Date, the Trustee
will repay the Deposit to the Contributor. The repayment amount will be the lower of: (a) the
Original Amount of the Deposit less any amount(s) repaid to the Contributor under paragraphs 8 or
9 or transferred by the Contributor under paragraph 11 of this Deposit Agreement, and (b) the
Notional Value of the Deposit. To the extent any transfers are received in the Deposit Account
pursuant to Section VI, paragraph 1(c) of the RST Instrument after the Maturity Date, the Trustee will
promptly repay to the Contributor any amounts received that are attributable to the Deposit.
8. Encashment. The Contributor will be permitted to seek early repayment of all or part of the
Deposit (the “Encashed Deposit”), pursuant to Section IV, paragraph 5(e) of the RST Instrument, if
the Contributor represents that the balance of payments and reserve position of Australia justifies
the early repayment in advance of the Maturity Date and the Trustee, having given the
representation the overwhelming benefit of any doubt, concurs. The Encashed Deposit amount will
not exceed the Original Amount of the Deposit less any amount(s) repaid to the Contributor under
this paragraph or paragraph 9 or transferred by the Contributor under paragraph 11 of this Deposit
Agreement, or the Notional Value of the Deposit, whichever is lower. The Contributor will restore an
amount equivalent to the Encashed Deposit to the Deposit Account as soon as practicable in light of
favorable developments in Australia’s balance of payments and reserve position, as evidenced by
the inclusion of the Australian currency in the Fund’s Financial Transactions Plan.
9. Early Repayment.
(a) The Trustee may repay all or part of the Deposit at any time in advance of the Maturity Date;
provided that the repayment amount will not exceed the Original Amount of the Deposit less any
amount(s) repaid to the Contributor under this paragraph or paragraph 8 or transferred by the
Contributor under paragraph 11 of this Deposit Agreement, or the Notional Value of the Deposit,
whichever is lower; and provided further that the Trustee will give notice to the Contributor at least
five (5) Fund business days prior to any advance repayment.
(b) If Australia does not provide its consent to a proposed amendment to the RST Instrument as
required under Section X, paragraph 2(a) of the RST Instrument, the Contributor may request the return
of a portion of the Original Amount of the Deposit less any amount(s) repaid to the Contributor under
this paragraph and paragraph 8 or transferred by the Contributor under paragraph 11 of this Deposit
Agreement, or the Notional Value of the Deposit, whichever is lower, with that portion based on the ratio
of remaining uncommitted loan resources to total loan commitments under the Contributor’s Borrowing
Agreement in accordance with Section X, paragraph 2(b) of the RST Instrument.
10. Liquidation. If the Trustee decides to wind up the operations of the Trust, the Trust will
repay the Deposit to the Contributor, at the Original Amount of the Deposit less any amount(s)
repaid to the Contributor under paragraphs 8 or 9 or transferred by the Contributor under
paragraph 11 of this Deposit Agreement, or the Notional Value of the Deposit, whichever is lower,
and in accordance with liquidation procedures under the RST Instrument, provided that the Trustee
will notify the Contributor in advance of the repayment. Payment will be made to an account
specified by the Contributor, which can include an account or trust administered by the Fund. 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
14 INTERNATIONAL MONETARY FUND
11. Transfer of Claims.
(a) The Contributor may transfer at any time all or part of any claim on the Deposit Account to
any member of the Fund, to the central bank or other fiscal agency designated by such a member
for purposes of Article V, Section 1 of the Fund’s Articles of Agreement, or to any official entity that
has been prescribed as a holder of SDRs pursuant to Article XVII, Section 3 of the Fund’s Articles of
Agreement.
(b) The transferee will acquire all the benefits of the Contributor under this Deposit Agreement
with respect to repayment of and interest on the transferred claim, except that the transferee will
acquire the benefit to request early repayment under paragraph 8 of this Deposit Agreement only if
it is a member or the central bank or other fiscal agency of a member and, at the time of transfer,
the member’s balance of payments and reserve position was considered sufficiently strong in the
opinion of the Fund for its currency to be usable in transfers under the Fund’s Financial Transactions
Plan.
12. Modality of Interest Payment and Repayment of the Deposit.
(a) Unless otherwise mutually decided between the Contributor and the Trustee, payment of
interest will be made in SDRs and repayments of the Deposit will be made in SDRs.
(b) Payment of interest and repayment of the Deposit in SDRs will be made by crediting the
amount due to Australia’s holdings account in the Special Drawing Rights Department. If the due
date is not a business day of the Fund, the payment will be made on the following business day for
the Fund.
(c) Repayment of the Deposit by the Trustee in currencies will be made to an account as
mutually decided between the Trustee and the Contributor. If the due date is not a business day of
the Fund or the Target system or the domestic markets of the non-euro currencies included in the
SDR basket, the payment will be made on the following business day for the Fund, the Target
system and the domestic markets of the non-euro currencies included in the SDR basket.]
13. Exchange Rate. Unless otherwise mutually decided between the Contributor and the
Trustee, all transfers, exchanges, payments of interest, or repayments of the Deposit under this
Deposit Agreement in currencies will be made at the exchange rates for the relevant currencies in
terms of the SDR established by the Fund pursuant to Article XIX, Section 7(a) of the Fund’s Articles
of Agreement and the rules and regulations of the Fund thereunder for the second (Fund) business
day before the value date of the transfer, exchange[, payment of interest,] or repayment.
14. SDR Basket. If the Fund changes the currency composition or currency weights of the SDR
basket, all transfers, exchanges, payments of interest, and repayments of the Deposit two or more
(Fund) business days after the effective date of the change will be made on the basis of the new SDR
basket.
*********************************** 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 15
III. RESERVE CONTRIBUTION AGREEMENT
1. Amount. The Contributor will provide a contribution equivalent to SDR 15.2 million to the
Reserve Account of the Trust (the “Contribution
2. Denomination and Media.
(a) The Contribution will be denominated in SDRs.
(b) The Contribution will be made in either SDRs or a freely usable currency, as determined by
the Contributor.
(c) The Contributor will transfer the Contribution to an account specified by the Trustee on or
promptly after the effectiveness of the Contribution Agreement.
3. Investment Strategy and Risk Disclosure.
(a) The resources in the Reserve Account will be invested by the Trustee in accordance with the
guidelines adopted by the Trustee pursuant to Section V, paragraph 3(a) of the RST Instrument.
(b) The assets of the Reserve Account are exposed to the market and credit risk which could
result in potential losses to the Contributor’s share in the resources of the Reserve Account.
4. Remuneration. No remuneration will be paid to the Contributor in respect of the
Contribution.
5. Contributor’s Share in the Reserve Account. The Contributor will have a proportional
share in the Reserve Account, based on its Contribution, any resources that are transferred from the
Deposit Account and attributed to the Contributor’s contribution to the Deposit Account pursuant
to Section VI, Paragraph 5(d) of the RST Instrument, and attributed investment earnings or losses in
the Reserve Account.
6. Transfer of the Contributor’s Share. The Contributor may transfer at any time all or part of
its share in the Reserve Account to any member of the Fund, to any member’s central bank or other
fiscal agency designated by such a member for purposes of Article V, Section 1 of the Fund’s Articles
of Agreement, or to any official entity that has been prescribed as a holder of SDRs pursuant to
Article XVII, Section 3 of the Fund’s Articles of Agreement. The transferee will acquire all the benefits
of the Contributor under this Reserve Contribution Agreement with respect to the transferred
amount.
7. Distribution of Resources in the Reserve Account.
(a) The Contribution will have no fixed maturity. 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
16 INTERNATIONAL MONETARY FUND
(b) Upon the liquidation of the Trust, the Trustee will distribute to the Contributor its share in
the resources of the Reserve Account of the Trust in accordance with Section V, paragraph 4(c) of
the RST Instrument.
(c) Prior to the liquidation of the Trust and following the Trustee’s decision to distribute a
portion of the balances in the Reserve Account of the RST in accordance with Section V, paragraph
4(d) of the RST Instrument, the Contributor will receive part of this distribution in proportion to its
share in the Reserve Account.
(d) If Australia does not provide its consent to a proposed amendment to the RST Instrument as
required under Section X, paragraph 2(a) of the RST Instrument, the Contributor may request the
return of a portion of its share in the Reserve Account based on the ratio of remaining uncommitted
loan resources to total loan commitments under the Contributor’s Borrowing Agreement in
accordance with Section X, paragraph 2(b) of the RST Instrument.
8. Modality of Distribution.
(a) Unless otherwise mutually decided between the Trustee and the Contributor, any
distribution to the Contributor under this Reserve Contribution Agreement will be made in SDRs.
(b) Distribution in SDRs will be made by crediting Australia’s holdings account in the Special
Drawing Rights Department on a business day of the Fund.
(c) Distribution in currencies will be made to an account as mutually decided between the
Trustee and the Contributor and on the business day of the Fund, the Target system and the
domestic markets of the non-euro currencies included in the SDR basket.
9. Exchange Rates. Unless otherwise mutually decided between the Trustee and the
Contributor, all transfers, exchanges, and distribution in currencies will be made at the exchange
rates for the relevant currencies in terms of the SDR established by the Fund pursuant to Article XIX,
Section 7(a) of the Fund’s Articles of Agreement and the rules and regulations of the Fund
thereunder for the second (Fund) business day before the value date of the transfer, exchange or
payment.
10. SDR Basket. If the Fund changes the currency composition or currency weights of the SDR
basket, all transfers, exchanges, and distribution made two or more (Fund) business days after the
effective date of the change will be made on the basis of the new SDR basket.
***********************
IV. FINAL PROVISIONS
1. Dispute Resolution. This agreement is of an operational and financial nature. Any question
arising under this Contribution Agreement will be settled mutually between the Contributor and the
Trustee. 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 17
2. Variation. This Contribution Agreement may be amended by mutual agreement in writing
between the Contributor and the Trustee.
3. Signature. This Contribution Agreement may be signed in one or more duplicate
counterparts, each of which will be deemed an original, and which together will constitute one and
the same instrument.
4. Effectiveness. This Contribution Agreement will become effective on the last date written
below; provided, however, that, Section I. (Borrowing Agreement) will become effective on the last
date written below or the date the Managing Director of the Fund has notified the Executive Board
of the Fund that the RST has become operational, whichever is later.
[Signature page follows] 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
18 INTERNATIONAL MONETARY FUND
The Government of Australia and the International Monetary Fund, as Trustee of the Resilience and
Sustainability Trust, have signed this agreement.
For the Government of Australia:
October 7, 2022
______________________________ ______________________________
Jim Chalmers /s Date
Treasurer of the Commonwealth of Australia
For the INTERNATIONAL MONETARY FUND as Trustee:
October 11, 2022
______________________________ ______________________________
Kristalina Georgieva /s Date
Managing Director
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 19
Attachment II. Contribution Agreement between
Canada and the International Monetary Fund,
as Trustee of the Resilience and Sustainability Trust
WHEREAS the International Monetary Fund (the “Fund”) has established the Resilience and
Sustainability Trust (the “Trust”) to assist in fulfilling the purposes of the Fund by providing loans
under the Resilience and Sustainability Facility of the Trust (the “RSF”) to eligible Fund members to
enhance their economic resilience and sustainability;
WHEREAS, Canada has committed to contribute financial resources to the loan account (“Loan
Account”), the deposit account (the “Deposit Account”) and the reserve account (the “Reserve
Account”) of the Trust in accordance with the provisions of the Instrument to Establish the Resilience
and Sustainability Trust (the “RST Instrument”); and
WHEREAS, to fulfill Canada’s commitment, Canada (the “Contributor”) and the Fund, as Trustee (the
“Trustee”) of the Trust, have reached understandings on a borrowing agreement (the “Borrowing
Agreement”), a deposit agreement (the “Deposit Agreement”), and a reserve contribution agreement
(the “Reserve Contribution Agreement”), jointly referred to as the Contribution Agreement;
NOW, THEREFORE, the Contributor and the Fund, as Trustee of the Trust, agree on the terms and
conditions of these contributions as follows:
I. BORROWING AGREEMENT
1. Amount. The Contributor agrees to lend an SDR-denominated amount up to the equivalent
of C$2,000,000,000 to the Loan Account of the Trust. This SDR-denominated amount will be
determined at the exchange rate for the Canadian dollar in terms of the SDR established by the
Fund pursuant to Article XIX, Section 7(a) of the Fund’s Articles of Agreement and the rules and
regulations of the Fund thereunder as of the second (Fund) business day before the value date of
the transfer by the Contributor of its contribution to the Reserve Account under the Reserve
Contribution Agreement.
2. Drawings.
(a) The Trustee may make drawings under this Borrowing Agreement at any time during the
period from the effective date of this Borrowing Agreement, as set forth in paragraph 3 of the Final
Provisions below, through November 30, 2030 (the “drawdown period”) to finance loan
disbursements under the RSF.
(b) The Trustee may make drawings under this Borrowing Agreement to finance early
repayment of outstanding Trust borrowing under other borrowing agreements with the Loan
Account pursuant to Section IV, Paragraph 3(b) of the RST Instrument for as long as claims under
such borrowing agreements remain outstanding. 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
20 INTERNATIONAL MONETARY FUND
(c) The Trustee’s drawings under this Borrowing Agreement will be suspended following the
Contributor’s request for early repayment under paragraph 6 of this Borrowing Agreement or when
Canada is no longer included for transfers in the Fund’s Financial Transactions Plan, in accordance
with Section IV, paragraph 4 of the RST Instrument.
(d) If Canada does not provide its consent to a proposed amendment to the RST Instrument
pursuant to Section X, paragraph 2(a) of the RST Instrument, further drawings under this Borrowing
Agreement to finance any new commitment of loans under the RSF that is made after the
effectiveness of the proposed amendment will be suspended.
(e) If any installment of principal or interest is not paid to the Contributor within a period of ten
Fund business days after its due date, the Trustee will not make further drawings under this
Borrowing Agreement pending consultations with the Contributor on the matter. However, the
Trustee may resume drawings under this Borrowing Agreement once all arrears to the Contributor
have been discharged.
(f) Unless otherwise agreed between the Contributor and the Trustee, the Trustee will give the
Contributor at least five (Ottawa) business days’ notice of its intention to draw and will provide payment
instructions at least two (Fund) business days prior to the value date of the transaction by SWIFT.
3. Denomination, Media, and Claim Certificate.
(a) The amount of each drawing will be denominated in SDRs. The amount will be paid by the
Contributor on the value date specified in the Trustee’s notice by transfer of SDRs to an account
specified by the Trustee.
(b) Upon request, the Trustee will issue to the Contributor a non-negotiable certificate evidencing
its claim on the Trust resulting from a drawing outstanding under this Borrowing Agreement.
4. Interest.
(a) Each drawing will bear interest at the SDR interest rate established by the Fund pursuant to
Article XX, Section 3 of the Fund’s Articles of Agreement.
(b) The amount of interest payable in respect of each drawing will be calculated on an actual
day basis and will be paid on all outstanding drawings under this Borrowing Agreement on a
quarterly basis promptly after January 31, April 30, July 31, and October 31 of each year.
5. Maturity and Repayment. Except as otherwise provided in this Borrowing Agreement, each
drawing will be repaid in accordance with the repayment schedule for the loan disbursement under
the RSF for which it was drawn, and at the time of each drawing, the Trustee will provide information
to the Contributor on the specific repayment schedule for the amount drawn.
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 21
6. Encashment. The Contributor will have the right to seek early repayment of all or part of the
principal amount of its outstanding claims on the Loan Account of the Trust pursuant to Section IV,
paragraph 3(b) of the RST Instrument if the Contributor represents that the balance of payments
and reserve position of Canada justifies the early repayment, and the Trustee, having given this
representation the overwhelming benefit of any doubt, agrees. Early repayment is in all cases subject
to the availability of resources under borrowing agreements to the Loan Account of the Trust.
7. Early Repayment. The Trustee may repay any drawing or part thereof at any time in
advance of maturity upon its receipt of early repayment of the corresponding loan disbursement
under the RSF, provided that the Trustee will give notice to the Contributor at least five (5) (Fund)
business days prior to such advance repayment.
8. Modality of Principal Repayment and Interest Payments.
(a) Unless otherwise agreed between the Trustee and the Contributor, payments of principal
and interest by the Trustee will be made in SDRs.
(b) Payments in SDRs by the Trustee will be made by crediting Canada’s holdings account in the
Special Drawing Rights Department. If the due date is not a business day of the Fund, the payment
will be made on the following business day of the Fund.
(c) Payments by the Trustee in currencies will be made to an account as agreed between the
Trustee and the Contributor. If the due date is not a business day of the Fund or the Target system
or the domestic markets of the non-euro currencies included in the SDR basket, the payment will be
made on the following business day for the Fund, the Target system and the domestic markets of
the non-euro currencies included in the SDR basket.
9. Transfer of Claims.
(a) The Contributor will have the right to transfer at any time all or part of any claim to any
member of the Fund, to the central bank or other fiscal agency designated by such a member for
purposes of Article V, Section 1 of the Fund’s Articles of Agreement or to any official entity that has
been prescribed as a holder of SDRs pursuant to Article XVII, Section 3 of the Fund’s Articles of
Agreement.
(b) The transferee will acquire all the rights of the Contributor under this Borrowing Agreement
with respect to repayment of principal and interest on the transferred claim, except that the
transferee will acquire the right to request early repayment under paragraph 6 of this Borrowing
Agreement only if it is a member or the central bank or other fiscal agency of a member and, at the
time of transfer, the member’s balance of payments and reserve position was considered sufficiently
strong in the opinion of the Fund for its currency to be usable in transfers under the Fund’s Financial
Transactions Plan.
10. Exchange Rates. Unless otherwise agreed between the Trustee and the Contributor, all
transfers, exchanges, and payments of principal and interest in currencies will be made at the
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
22 INTERNATIONAL MONETARY FUND
exchange rates for the relevant currencies in terms of the SDR established by the Fund pursuant to
Article XIX, Section 7(a) of the Fund’s Articles of Agreement and the rules and regulations of the
Fund thereunder for the second (Fund) business day before the value date of the transfer, exchange
or payment.
11. SDR Basket. If the Fund changes the currency composition or currency weights of the SDR
basket, all transfers, exchanges, and payments of principal and interest made two or more (Fund)
business days after the effective date of the change will be made on the basis of the new SDR
basket.
***************************
II. DEPOSIT AGREEMENT
1. Amount of the Deposit. The Contributor will make a deposit (the “Deposit”) of an SDR
denominated amount equivalent to C$400,000,000 with the Deposit Account of the Trust (the
“Original Amount of the Deposit”). This SDR-denominated amount will be determined at the
exchange rate for the Canadian dollar in terms of the SDR established by the Fund pursuant to
Article XIX, Section 7(a) of the Fund’s Articles of Agreement and the rules and regulations of the
Fund thereunder as of the second (Fund) business day before the value date of the transfer by the
Contributor of its contribution to the Reserve Account under the Reserve Contribution Agreement.
2. Denomination and Media.
(a) The Deposit will be denominated in SDRs.
(b) The Deposit will be made in either SDRs or a freely usable currency, as determined by the
Contributor.
(c) The Deposit will be made promptly on or after the effectiveness of the Contribution
Agreement.
3. Maturity. The Deposit will mature on November 30, 2050 (the “Maturity Date”).
4. Investment Strategy and Risk Disclosure.
(a) The resources from the Deposit will be invested by the Trustee in accordance with the
guidelines adopted by the Trustee pursuant to Section VI, paragraph 4(a) of the RST Instrument.
(b) The assets of the Deposit Account are exposed to market and credit risk which could result in
potential losses to the Original Amount of the Deposit.
5. Notional Value of the Deposit. For the purposes set out in paragraphs 6, 7, 8, and 9 of this
Deposit Agreement, the Trustee will calculate the notional value of the Deposit based on the
Original Amount of the Deposit and subsequently any attributed investment earnings or losses, 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 23
interest payments made to the Contributor pursuant to paragraph 6 of this Deposit Agreement, if
any, and transfers pursuant to paragraphs 8, 9, 10 or 11 of this Deposit Agreement or pursuant to
Section VI, paragraphs 1(c) and 4(b)(2) of the RST Instrument (“Notional Value”).
6. Interest.
(a) The Trustee will pay the Contributor interest on the Original Amount of the Deposit less any
amounts repaid to the Contributor under paragraphs 8 or 9 or transferred by the Contributor under
paragraph 11 of this Deposit Agreement, at the SDR interest rate established by the Fund pursuant
to Article XX, Section 3 of the Fund’s Articles of Agreement. Interest will accrue daily and will be paid
promptly by the Trustee on a quarterly basis after January 31, April 30, July 31 and October 31 of
each year. Any payment of interest will be limited to the Notional Value of the Deposit.
(b) Following any full repayment of the Deposit in accordance with paragraphs 7 and 9 of this
Deposit Agreement, all remaining interest under paragraph 6(a) above, will be paid to the Contributor
at the end of the Fund financial quarter in which repayment was made.
7. Repayment upon Maturity. As soon as practicable following the Maturity Date, the Trustee
will repay the Deposit to the Contributor. The repayment amount will be the lower of: (a) the
Original Amount of the Deposit less any amount(s) repaid to the Contributor under paragraphs 8 or
9 or transferred by the Contributor under paragraph 11 of this Deposit Agreement, and (b) the
Notional Value of the Deposit. To the extent any transfers are received in the Deposit Account
pursuant to Section VI, paragraph 1(c) of the RST Instrument after the Maturity Date, the Trustee will
promptly repay to the Contributor any amounts received that are attributable to the Deposit.
8. Encashment. The Contributor will have the right to seek early repayment of all or part of the
Deposit (the “Encashed Deposit”), pursuant to Section VI, paragraph 5(e) of the RST Instrument, if
the Contributor represents that the balance of payments and reserve position of Canada justifies the
early repayment in advance of the Maturity Date and the Trustee, having given the representation
the overwhelming benefit of any doubt, agrees. The Encashed Deposit amount may not exceed the
Original Amount of the Deposit less any amount(s) repaid to the Contributor under this paragraph
or paragraph 9 or transferred by the Contributor under paragraph 11 of this Deposit Agreement, or
the Notional Value of the Deposit, whichever is lower. The Contributor agrees to restore an amount
equivalent to the Encashed Deposit to the Deposit Account as soon as practicable in light of
favorable developments in Canada’s balance of payments and reserve position, as evidenced by the
inclusion of the Canada’s currency in the Fund’s Financial Transactions Plan.
9. Early Repayment.
(a) The Trustee may repay all or part of the Deposit at any time in advance of the Maturity Date;
provided that the repayment amount may not exceed the Original Amount of the Deposit less any
amount(s) repaid to the Contributor under this paragraph or paragraph 8 or transferred by the
Contributor under paragraph 11 of this Deposit Agreement, or the Notional Value of the Deposit, 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
24 INTERNATIONAL MONETARY FUND
whichever is lower; and provided further that the Trustee will give notice to the Contributor at least
five (5) Fund business days prior to any advance repayment.
(b) If Canada does not provide its consent to a proposed amendment to the RST Instrument as
required under Section X, paragraph 2(a) of the RST Instrument, the Contributor may request the
return of a portion of the Original Amount of the Deposit less any amount(s) repaid to the
Contributor under this paragraph and paragraph 8 or transferred by the Contributor under
paragraph 11 of this Deposit Agreement, or the Notional Value of the Deposit, whichever is lower,
with that portion based on the ratio of remaining uncommitted loan resources to total loan
commitments under the Contributor’s Borrowing Agreement in accordance with Section X,
paragraph 2(b) of the RST Instrument.
10. Liquidation. If the Trustee decides to wind up the operations of the Trust, the Trust will
repay the Deposit to the Contributor, at the Original Amount of the Deposit less any amount(s)
repaid to the Contributor under paragraphs 8 or 9 or transferred by the Contributor under
paragraph 11 of this Deposit Agreement, or the Notional Value of the Deposit, whichever is lower,
and in accordance with liquidation procedures under the RST Instrument, provided that the Trustee
will notify the Contributor in advance of the repayment. Payment will be made to an account
specified by the Contributor, which can include an account or trust administered by the Fund.
11. Transfer of Claims.
(a) The Contributor will have the right to transfer at any time all or part of any claim on the
Deposit Account to any member of the Fund, to the central bank or other fiscal agency designated
by such a member for purposes of Article V, Section 1 of the Fund’s Articles of Agreement, or to any
official entity that has been prescribed as a holder of SDRs pursuant to Article XVII, Section 3 of the
Fund’s Articles of Agreement.
(b) The transferee will acquire all the rights of the Contributor under this Deposit Agreement
with respect to repayment of, and interest on, the transferred claim, except that the transferee will
acquire the right to request early repayment under paragraph 8 of this Deposit Agreement only if it
is a member or the central bank or other fiscal agency of a member and, at the time of transfer, the
member’s balance of payments and reserve position was considered sufficiently strong in the
opinion of the Fund for its currency to be usable in transfers under the Fund’s Financial Transactions
Plan.
12. Modality of Interest Payment and Repayment of the Deposit.
(a) Unless otherwise agreed between the Contributor and the Trustee, payment of interest and
repayments of the Deposit will be made in SDRs.
(b) Payment of interest and repayment of the Deposit in SDRs will be made by crediting the
amount due to Canada’s holdings account in the Special Drawing Rights Department. If the due 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 25
date is not a business day of the Fund, the payment will be made on the following business day for
the Fund.
(c) Payment of interest and repayment of the Deposit by the Trustee in currencies will be made
to an account as agreed between the Trustee and the Contributor. If the due date is not a business
day of the Fund or the Target system or the domestic markets of the non-euro currencies included in
the SDR basket, the payment will be made on the following business day for the Fund, the Target
system and the domestic markets of the non-euro currencies included in the SDR basket.
13. Exchange Rate. Unless otherwise agreed between the Contributor and the Trustee, all
transfers, exchanges, payments of interest, or repayments of the Deposit under this Deposit
Agreement in currencies will be made at the exchange rates for the relevant currencies in terms of
the SDR established by the Fund pursuant to Article XIX, Section 7(a) of the Fund’s Articles of
Agreement and the rules and regulations of the Fund thereunder for the second (Fund) business day
before the value date of the transfer, exchange, payment of interest, or repayment.
14. SDR Basket. If the Fund changes the currency composition or currency weights of the SDR
basket, all transfers, exchanges, payments of interest, and repayments of the Deposit two or more
(Fund) business days after the effective date of the change will be made on the basis of the new SDR
basket.
***********************************
III. RESERVE CONTRIBUTION AGREEMENT
1. Amount. The Contributor will provide a contribution of an SDR-denominated amount
equivalent to C$40,000,000 (the “Contribution”), with the SDR-denominated amount to be
determined pursuant to paragraph 9 of this Reserve Contribution Agreement as of the second
(Fund) business day before the value date of the transfer of the Contribution to the Reserve Account
of the Trust.
2. Denomination and Media.
(a) The Contribution will be denominated in SDRs.
(b) The Contribution will be made in either SDRs or a freely usable currency, as determined by
the Contributor.
(c) The Contributor will transfer the Contribution to an account specified by the Trustee on or
promptly after the effectiveness of the Contribution Agreement.
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
26 INTERNATIONAL MONETARY FUND
3. Investment Strategy and Risk Disclosure.
(a) The resources in the Reserve Account will be invested by the Trustee in accordance with the
guidelines adopted by the Trustee pursuant to Section V, paragraph 3(a) of the RST Instrument.
(b) The assets of the Reserve Account are exposed to the market and credit risk which could
result in potential losses to the Contributor’s share in the resources of the Reserve Account.
4. Remuneration. No remuneration will be paid to the Contributor in respect of the
Contribution.
5. Contributor’s Share in the Reserve Account. The Contributor will have a proportional
share in the Reserve Account, based on its Contribution, any resources that are transferred from the
Deposit Account and attributed to the Contributor’s contribution to the Deposit Account pursuant
to Section VI, Paragraph 5(d) of the RST Instrument, and attributed investment earnings or losses in
the Reserve Account.
6. Transfer of the Contributor’s Share. The Contributor will have the right to transfer at any
time all or part of its share in the Reserve Account to any member of the Fund, to any member’s
central bank or other fiscal agency designated by such a member for purposes of Article V, Section 1
of the Fund’s Articles of Agreement, or to any official entity that has been prescribed as a holder of
SDRs pursuant to Article XVII, Section 3 of the Fund’s Articles of Agreement. The transferee will
acquire all the rights of the Contributor under this Reserve Contribution Agreement with respect to
the transferred amount.
7. Distribution of Resources in the Reserve Account.
(a) The Contribution will have no fixed maturity.
(b) Upon the liquidation of the Trust, the Trustee will distribute to the Contributor its share in
the resources of the Reserve Account of the Trust in accordance with Section V, paragraph 4(c) of
the RST Instrument.
(c) Prior to the liquidation of the Trust and following the Trustee’s decision to distribute a
portion of the balances in the Reserve Account of the RST in accordance with Section V, paragraph
4(d) of the RST Instrument, the Contributor will receive part of this distribution in proportion to its
share in the Reserve Account.
(d) If Canada does not provide its consent to a proposed amendment to the RST Instrument as
required under Section X, paragraph 2(a) of the RST Instrument, the Contributor may request the
return of a portion of its share in the Reserve Account based on the ratio of remaining uncommitted
loan resources to total loan commitments under the Contributor’s Borrowing Agreement in
accordance with Section X, paragraph 2(b) of the RST Instrument. 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 27
8. Modality of Distribution.
(a) Unless otherwise agreed between the Trustee and the Contributor, any distribution to the
Contributor under this Reserve Contribution Agreement will be made in SDRs.
(b) Distribution in SDRs will be made by crediting Canada’s holdings account in the Special
Drawing Rights Department on a business day of the Fund.
(c) Distribution in currencies will be made to an account as agreed between the Trustee and the
Contributor and on a business day of the Fund, the Target system and the domestic markets of the
non-euro currencies included in the SDR basket.
9. Exchange Rates. Unless otherwise agreed between the Trustee and the Contributor, all
transfers, exchanges, and distribution in currencies will be made at the exchange rates for the
relevant currencies in terms of the SDR established by the Fund pursuant to Article XIX, Section 7(a)
of the Fund’s Articles of Agreement and the rules and regulations of the Fund thereunder for the
second (Fund) business day before the value date of the transfer, exchange or payment.
10. SDR Basket. If the Fund changes the currency composition or currency weights of the SDR
basket, all transfers, exchanges, and distribution made two or more (Fund) business days after the
effective date of the change will be made on the basis of the new SDR basket.
***********************
IV. FINAL PROVISIONS
1. Dispute Resolution. This Agreement is of an operational and financial nature. Any question
arising under this Contribution Agreement will be settled by mutual agreement between the
Contributor and the Trustee.
2. Execution. This Contribution Agreement may be executed in one or more duplicate
counterparts, each of which will be deemed an original, and which together will constitute one and
the same instrument.
3. Effectiveness. This Contribution Agreement will become effective on the last date written
below; provided, however, that, Section I. (Borrowing Agreement) will become effective on the last
date written below or the date the Managing Director of the Fund has notified the Executive Board
of the Fund that the RST has become operational, whichever is later.
[Signature page follows] 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
28 INTERNATIONAL MONETARY FUND
IN WITNESS WHEREOF, Canada and the International Monetary Fund, as Trustee of the Resilience and
Sustainability Trust, have executed this agreement.
For HER MAJESTY THE QUEEN IN RIGHT OF CANADA, as represented by the MINISTER OF FINANCE:
September 23, 2022
______________________________ ______________________________
Chrystia Freeland /s Date
Minister of Finance
For the INTERNATIONAL MONETARY FUND as Trustee:
September 27, 2022
______________________________ ______________________________
Kristalina Georgieva /s Date
Managing Director
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 29
Attachment III. Contribution Agreement between
People’s Bank of China and the International Monetary Fund,
as Trustee of the Resilience and Sustainability Trust
WHEREAS the International Monetary Fund (the “Fund”) has established the Resilience and
Sustainability Trust (the “Trust”) to assist in fulfilling the purposes of the Fund by providing loans
under the Resilience and Sustainability Facility of the Trust (the “RSF”) to eligible Fund members to
enhance their economic resilience and sustainability;
WHEREAS, China has committed to contribute financial resources to the loan account (“Loan
Account”), the deposit account (the “Deposit Account”) and the reserve account (the “Reserve
Account”) of the Trust in accordance with the provisions of the Instrument to Establish the Resilience
and Sustainability Trust (the “RST Instrument”); and
WHEREAS, to fulfill China’s commitment, the People’s Bank of China (the “Contributor”) and the
Fund, as Trustee (the “Trustee”) of the Trust, have reached understandings on a borrowing
agreement (the “Borrowing Agreement”), a deposit agreement (the “Deposit Agreement”), and a
reserve contribution agreement (the “Reserve Contribution Agreement”), jointly referred to as the
Contribution Agreement;
NOW, THEREFORE, the Contributor and the Fund, as Trustee of the Trust, agree on the terms and
conditions of these contributions as follows:
I. BORROWING AGREEMENT
1. Amount. The Contributor agrees to lend an amount up to the equivalent of SDR 4.9 billion
to the Loan Account of the Trust.
2. Drawings.
(a) The Trustee may make drawings under this Borrowing Agreement at any time during the
period from the effective date of this Borrowing Agreement, as set forth in paragraph 3 of the Final
Provisions below, through November 30, 2030 (the “drawdown period”) to finance loan
disbursements under the RSF.
(b) The Trustee may make drawings under this Borrowing Agreement to finance early repayment
of outstanding Trust borrowing under other borrowing agreements with the Loan Account pursuant
to Section IV, Paragraph 3(b) of the RST Instrument for as long as claims under such borrowing
agreements remain outstanding.
(c) The Trustee’s drawings under this Borrowing Agreement shall be suspended following the
Contributor’s request for early repayment under paragraph 6 of this Borrowing Agreement or when
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
30 INTERNATIONAL MONETARY FUND
China is no longer included for transfers in the Fund’s Financial Transactions Plan, in accordance with
Section IV, paragraph 4 of the RST Instrument.
(d) If China does not provide its consent to a proposed amendment to the RST Instrument
pursuant to Section X, paragraph 2(a) of the RST Instrument, further drawings under this Borrowing
Agreement to finance any new commitment of loans under the RSF that is made after the
effectiveness of the proposed amendment shall be suspended.
(e) If any installment of principal or interest is not paid to the Contributor within a period of ten
Fund business days after its due date, the Trustee shall not make further drawings under this
Borrowing Agreement pending consultations with the Contributor on the matter. However, the
Trustee may resume drawings under this Borrowing Agreement once all arrears to the Contributor
have been discharged.
(f) Unless otherwise agreed between the Contributor and the Trustee, the Trustee shall give the
Contributor at least five (Beijing) business days’ notice of its intention to draw and shall provide
payment instructions at least two (Fund) business days prior to the value date of the transaction by
SWIFT.
3. Denomination, Media, and Claim Certificate.
(a) The amount of each drawing shall be denominated in SDRs. The amount shall be paid by the
Contributor on the value date specified in the Trustee’s notice by transfer of SDRs to an account
specified by the Trustee.
(b) Upon request, the Trustee shall issue to the Contributor a non-negotiable certificate
evidencing its claim on the Trust resulting from a drawing outstanding under this Borrowing
Agreement.
4. Interest.
(a) Each drawing shall bear interest at the SDR interest rate established by the Fund pursuant to
Article XX, Section 3 of the Fund’s Articles of Agreement.
(b) The amount of interest payable in respect of each drawing shall be calculated on an actual
day basis and shall be paid on all outstanding drawings under this Borrowing Agreement on a
quarterly basis promptly after January 31, April 30, July 31, and October 31 of each year.
5. Maturity and Repayment. Except as otherwise provided in this Borrowing Agreement, each
drawing shall be repaid in accordance with the repayment schedule for the loan disbursement under
the RSF for which it was drawn, and at the time of each drawing, the Trustee shall provide
information to the Contributor on the specific repayment schedule for the amount drawn.
6. Encashment. The Contributor shall have the right to seek early repayment of all or part of
the principal amount of its outstanding claims on the Loan Account of the Trust pursuant to Section
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 31
IV, paragraph 3(b) of the RST Instrument if the Contributor represents that the balance of payments
and reserve position of China justifies the early repayment, and the Trustee, having given this
representation the overwhelming benefit of any doubt, agrees. Early repayment is in all cases subject
to the availability of resources under borrowing agreements to the Loan Account of the Trust.
7. Early Repayment. The Trustee may repay any drawing or part thereof at any time in advance
of maturity upon its receipt of early repayment of the corresponding loan disbursement under the
RSF, provided that the Trustee will give notice to the Contributor at least five (5) (Fund) business days
prior to such advance repayment.
8. Modality of Principal Repayment and Interest Payments.
(a) Unless otherwise agreed between the Trustee and the Contributor, payments of principal and
interest by the Trustee shall be made in SDRs.
(b) Payments in SDRs by the Trustee shall be made by crediting China’s holdings account in the
Special Drawing Rights Department. If the due date is not a business day of the Fund, the payment
shall be made on the following business day of the Fund.
(c) Payments by the Trustee in currencies shall be made to an account as agreed between the
Trustee and the Contributor. If the due date is not a business day of the Fund or the Target system or
the domestic markets of the non-euro currencies included in the SDR basket, the payment shall be
made on the following business day for the Fund, the Target system and the domestic markets of the
non-euro currencies included in the SDR basket.
9. Transfer of Claims.
(a) The Contributor shall have the right to transfer at any time all or part of any claim to any
member of the Fund, to the central bank or other fiscal agency designated by such a member for
purposes of Article V, Section 1 of the Fund’s Articles of Agreement or to any official entity that has
been prescribed as a holder of SDRs pursuant to Article XVII, Section 3 of the Fund’s Articles of
Agreement.
(b) The transferee shall acquire all the rights of the Contributor under this Borrowing Agreement
with respect to repayment of principal and interest on the transferred claim, except that the
transferee shall acquire the right to request early repayment under paragraph 6 of this Borrowing
Agreement only if it is a member or the central bank or other fiscal agency of a member and, at the
time of transfer, the member’s balance of payments and reserve position was considered sufficiently
strong in the opinion of the Fund for its currency to be usable in transfers under the Fund’s Financial
Transactions Plan.
10. Exchange Rates. Unless otherwise agreed between the Trustee and the Contributor, all
transfers, exchanges, and payments of principal and interest in currencies shall be made at the
exchange rates for the relevant currencies in terms of the SDR established by the Fund pursuant to
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
32 INTERNATIONAL MONETARY FUND
Article XIX, Section 7(a) of the Fund’s Articles of Agreement and the rules and regulations of the
Fund thereunder for the second (Fund) business day before the value date of the transfer, exchange
or payment.
11. SDR Basket. If the Fund changes the currency composition or currency weights of the SDR
basket, all transfers, exchanges, and payments of principal and interest made two or more (Fund)
business days after the effective date of the change shall be made on the basis of the new SDR
basket.
***************************
II. DEPOSIT AGREEMENT
1. Amount of the Deposit. The Contributor shall make a deposit (the “Deposit”) equivalent to
SDR 1 billion with the Deposit Account of the Trust (the “Original Amount of the Deposit”).
2. Denomination and Media.
(a) The Deposit shall be denominated in SDRs.
(b) The Deposit shall be made in either SDRs or a freely usable currency, as determined by the
Contributor.
(c) The Deposit shall be made promptly on or after the effectiveness of the Contribution
Agreement.
3. Maturity. The Deposit shall mature on November 30, 2050 (the “Maturity Date”).
4. Investment Strategy and Risk Disclosure.
(a) The resources from the Deposit shall be invested by the Trustee in accordance with the
guidelines adopted by the Trustee pursuant to Section VI, paragraph 4(a) of the RST Instrument.
(b) The assets of the Deposit Account are exposed to market and credit risk which could result in
potential losses to the Original Amount of the Deposit.
5. Notional Value of the Deposit. For the purposes set out in paragraphs 6, 7, 8, and 9 of this
Deposit Agreement, the Trustee shall calculate the notional value of the Deposit based on the
Original Amount of the Deposit and subsequently any attributed investment earnings or losses,
interest payments made to the Contributor pursuant to paragraph 6 of this Deposit Agreement, if
any, and transfers pursuant to paragraphs 8, 9, 10 or 11 of this Deposit Agreement or pursuant to
Section VI, paragraphs 1(c) and 4(b)(2) of the RST Instrument (“Notional Value”). 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 33
6. Interest.
(a) The Trustee shall pay the Contributor interest on the Original Amount of the Deposit less any
amounts repaid to the Contributor under paragraphs 8 or 9 or transferred by the Contributor under
paragraph 11 of this Deposit Agreement at the SDR interest rate established by the Fund pursuant to
Article XX, Section 3 of the Fund’s Articles of Agreement. Interest shall accrue daily and shall be paid
promptly by the Trustee on a quarterly basis after January 31, April 30, July 31 and October 31 of
each year. Any payment of interest shall be limited to the Notional Value of the Deposit.
(b) Following any full repayment of the Deposit in accordance with paragraphs 7 and 9 of this
Deposit Agreement, all remaining interest under paragraph 6(a) above, shall be paid to the
Contributor at the end of the Fund financial quarter in which repayment was made.
7. Repayment upon Maturity. As soon as practicable following the Maturity Date, the Trustee
shall repay the Deposit to the Contributor. The repayment amount shall be the lower of: (a) the
Original Amount of the Deposit less any amount(s) repaid to the Contributor under paragraphs 8 or
9 or transferred by the Contributor under paragraph 11 of this Deposit Agreement, and (b) the
Notional Value of the Deposit. To the extent any transfers are received in the Deposit Account
pursuant to Section VI, paragraph 1(c) of the RST Instrument after the Maturity Date, the Trustee
shall promptly repay to the Contributor any amounts received that are attributable to the Deposit.
8. Encashment. The Contributor shall have the right to seek early repayment of all or part of
the Deposit (the “Encashed Deposit”), pursuant to Section VI, paragraph 5(e) of the RST Instrument, if
the Contributor represents that the balance of payments and reserve position of China justifies the
early repayment in advance of the Maturity Date and the Trustee, having given the representation
the overwhelming benefit of any doubt, agrees. The Encashed Deposit amount may not exceed the
Original Amount of the Deposit less any amount(s) repaid to the Contributor under this paragraph or
paragraph 9 or transferred by the Contributor under paragraph 11 of this Deposit Agreement, or the
Notional Value of the Deposit, whichever is lower. The Contributor agrees to restore an amount
equivalent to the Encashed Deposit to the Deposit Account as soon as practicable in light of
favorable developments in China’s balance of payments and reserve position, as evidenced by the
inclusion of China’s currency in the Fund’s Financial Transactions Plan.
9. Early Repayment.
(a) The Trustee may repay all or part of the Deposit at any time in advance of the Maturity Date;
provided that the repayment amount may not exceed the Original Amount of the Deposit less any
amount(s) repaid to the Contributor under this paragraph or paragraph 8 or transferred by the
Contributor under paragraph 11 of this Deposit Agreement, or the Notional Value of the Deposit,
whichever is lower; and provided further that the Trustee will give notice to the Contributor at least
five (5) Fund business days prior to any advance repayment.
(b) If China does not provide its consent to a proposed amendment to the RST Instrument as
required under Section X, paragraph 2(a) of the RST Instrument, the Contributor may request the 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
34 INTERNATIONAL MONETARY FUND
return of a portion of the Original Amount of the Deposit less any amount(s) repaid to the
Contributor under this paragraph and paragraph 8 or transferred by the Contributor under
paragraph 11 of this Deposit Agreement, or the Notional Value of the Deposit, whichever is lower,
with that portion based on the ratio of remaining uncommitted loan resources to total loan
commitments under the Contributor’s Borrowing Agreement in accordance with Section X,
paragraph 2(b) of the RST Instrument.
10. Liquidation. If the Trustee decides to wind up the operations of the Trust, the Trust shall
repay the Deposit to the Contributor, at the Original Amount of the Deposit less any amount(s)
repaid to the Contributor under paragraphs 8 or 9 or transferred by the Contributor under paragraph
11 of this Deposit Agreement, or the Notional Value of the Deposit, whichever is lower, and in
accordance with liquidation procedures under the RST Instrument, provided that the Trustee shall
notify the Contributor in advance of the repayment. Payment will be made to an account specified by
the Contributor, which can include an account or trust administered by the Fund.
11. Transfer of Claims.
(a) The Contributor shall have the right to transfer at any time all or part of any claim on the Deposit
Account to any member of the Fund, to the central bank or other fiscal agency designated by such a member
for purposes of Article V, Section 1 of the Fund’s Articles of Agreement, or to any official entity that has been
prescribed as a holder of SDRs pursuant to Article XVII, Section 3 of the Fund’s Articles of Agreement.
(b) The transferee shall acquire all the rights of the Contributor under this Deposit Agreement with
respect to repayment of and interest on the transferred claim, except that the transferee shall acquire
the right to request early repayment under paragraph 8 of this Deposit Agreement only if it is a
member or the central bank or other fiscal agency of a member and, at the time of transfer, the
member’s balance of payments and reserve position was considered sufficiently strong in the opinion
of the Fund for its currency to be usable in transfers under the Fund’s Financial Transactions Plan.
12. Modality of Interest Payment and Repayment of the Deposit.
(a) Unless otherwise agreed between the Contributor and the Trustee, payment of interest shall
be made in SDRs and repayment of the Deposit shall be made in SDRs.
(b) Payment of interest and repayment of the Deposit in SDRs shall be made by crediting the
amount due to China’s holdings account in the Special Drawing Rights Department. If the due date is
not a business day of the Fund, the payment shall be made on the following business day for the
Fund.
(c) Payment of interest and repayment of the Deposit by the Trustee in currencies shall be made
to an account as agreed between the Trustee and the Contributor. If the due date is not a business
day of the Fund or the Target system or the domestic markets of the non-euro currencies included in
the SDR basket, the payment shall be made on the following business day for the Fund, the Target
system and the domestic markets of the non-euro currencies included in the SDR basket.
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 35
13. Exchange Rate. Unless otherwise agreed between the Contributor and the Trustee, all
transfers, exchanges, payments of interest, or repayments of the Deposit under this Deposit
Agreement in currencies shall be made at the exchange rates for the relevant currencies in terms of
the SDR established by the Fund pursuant to Article XIX, Section 7(a) of the Fund’s Articles of
Agreement and the rules and regulations of the Fund thereunder for the second (Fund) business day
before the value date of the transfer, exchange, payment of interest, or repayment.
14. SDR Basket. If the Fund changes the currency composition or currency weights of the SDR
basket, all transfers, exchanges, payments of interest, and repayments of the Deposit two or more
(Fund) business days after the effective date of the change shall be made on the basis of the new
SDR basket.
***********************************
III. RESERVE CONTRIBUTION AGREEMENT
1. Amount. The Contributor shall provide a contribution equivalent to SDR 100 million to the
Reserve Account of the Trust (the “Contribution”).
2. Denomination and Media.
(a) The Contribution shall be denominated in SDRs.
(b) The Contribution shall be made in either SDRs or a freely usable currency, as determined by
the Contributor.
(c) The Contributor shall transfer the Contribution to an account specified by the Trustee on or
promptly after the effectiveness of the Contribution Agreement.
3. Investment Strategy and Risk Disclosure.
(a) The resources in the Reserve Account shall be invested by the Trustee in accordance with the
guidelines adopted by the Trustee pursuant to Section V, paragraph 3(a) of the RST Instrument.
(b) The assets of the Reserve Account are exposed to the market and credit risk which could
result in potential losses to the Contributor’s share in the resources of the Reserve Account.
4. Remuneration. No remuneration shall be paid to the Contributor in respect of the
Contribution.
5. Contributor’s Share in the Reserve Account. The Contributor shall have a proportional share in
the Reserve Account, based on its Contribution, any resources that are transferred from the Deposit
Account and attributed to the Contributor’s contribution to the Deposit Account pursuant to Section VI,
Paragraph 5(d) of the RST Instrument, and attributed investment earnings or losses in the Reserve Account. 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
36 INTERNATIONAL MONETARY FUND
6. Transfer of the Contributor’s Share. The Contributor shall have the right to transfer at any
time all or part of its share in the Reserve Account to any member of the Fund, to any member’s
central bank or other fiscal agency designated by such a member for purposes of Article V, Section 1
of the Fund’s Articles of Agreement, or to any official entity that has been prescribed as a holder of
SDRs pursuant to Article XVII, Section 3 of the Fund’s Articles of Agreement. The transferee shall
acquire all the rights of the Contributor under this Reserve Contribution Agreement with respect to
the transferred amount.
7. Distribution of Resources in the Reserve Account.
(a) The Contribution shall have no fixed maturity.
(b) Upon the liquidation of the Trust, the Trustee shall distribute to the Contributor its share in
the resources of the Reserve Account of the Trust in accordance with Section V, paragraph 4(c) of the
RST Instrument.
(c) Prior to the liquidation of the Trust and following the Trustee’s decision to distribute a
portion of the balances in the Reserve Account of the RST in accordance with Section V, paragraph
4(d) of the RST Instrument, the Contributor shall receive part of this distribution in proportion to its
share in the Reserve Account.
(d) If China does not provide its consent to a proposed amendment to the RST Instrument as
required under Section X, paragraph 2(a) of the RST Instrument, the Contributor may request the
return of a portion of its share in the Reserve Account based on the ratio of remaining uncommitted
loan resources to total loan commitments under the Contributor’s Borrowing Agreement in
accordance with Section X, paragraph 2(b) of the RST Instrument.
8. Modality of Distribution.
(a) Unless otherwise agreed between the Trustee and the Contributor, any distribution to the
Contributor under this Reserve Contribution Agreement shall be made in SDRs.
(b) Distribution in SDRs shall be made by crediting China’s holdings account in the Special
Drawing Rights Department on a business day of the Fund.
(c) Distribution in currencies shall be made to an account as agreed between the Trustee and
the Contributor and on a business day of the Fund, the Target system and the domestic markets of
the non-euro currencies included in the SDR basket.
9. Exchange Rates. Unless otherwise agreed between the Trustee and the Contributor, all
transfers, exchanges, and distribution in currencies shall be made at the exchange rates for the
relevant currencies in terms of the SDR established by the Fund pursuant to Article XIX, Section 7(a)
of the Fund’s Articles of Agreement and the rules and regulations of the Fund thereunder for the
second (Fund) business day before the value date of the transfer, exchange or payment. 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 37
10. SDR Basket. If the Fund changes the currency composition or currency weights of the SDR
basket, all transfers, exchanges, and distribution made two or more (Fund) business days after the
effective date of the change shall be made on the basis of the new SDR basket.
***********************
IV. FINAL PROVISIONS
1. Dispute Resolution. Any question arising under this Contribution Agreement shall be settled
by mutual agreement between the Contributor and the Trustee.
2. Execution. This Contribution Agreement may be executed in one or more duplicate
counterparts, each of which shall be deemed an original, and which together shall constitute one and
the same instrument.
3. Effectiveness. This Contribution Agreement shall become effective on the last date written
below; provided, however, that Section I. (Borrowing Agreement) shall become effective on the last
date written below or the date the Managing Director of the Fund has notified the Executive Board
of the Fund that the RST has become operational, whichever is later.
[Signature page follows] 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
38 INTERNATIONAL MONETARY FUND
IN WITNESS WHEREOF, the People’s Bank of China and the International Monetary Fund, as Trustee
of the Resilience and Sustainability Trust, have executed this agreement.
For the PEOPLE’S BANK of CHINA:
September 26, 2022
______________________________ ______________________________
Yi Gang /s Date
Governor
For the INTERNATIONAL MONETARY FUND as Trustee:
September 27, 2022
______________________________ ______________________________
Kristalina Georgieva /s Date
Managing Director
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 39
Attachment IV. Contribution Agreement between the
Federal Republic of Germany and the International Monetary
Fund, as Trustee of the Resilience and Sustainability Trust
WHEREAS the International Monetary Fund (the “Fund”) has established the Resilience and
Sustainability Trust (the “Trust”) to assist in fulfilling the purposes of the Fund by providing loans
under the Resilience and Sustainability Facility of the Trust (the “RSF”) to eligible Fund members to
enhance their economic resilience and sustainability;
WHEREAS, the Federal Republic of Germany (“Germany”) has committed to contribute financial
resources to the deposit account (the “Deposit Account”) and the reserve account (the “Reserve
Account”) of the Trust in accordance with the provisions of the Instrument to Establish the Resilience
and Sustainability Trust (the “RST Instrument”); and
WHEREAS, to fulfill Germany’s commitment, Germany (the “Contributor”) and the Fund, as Trustee
(the “Trustee”) of the Trust, have reached understandings on a deposit agreement (the “Deposit
Agreement”) and a reserve contribution agreement (“Reserve Contribution Agreement”), jointly
referred to as the Contribution Agreement;
NOW, THEREFORE, the Contributor(s) and the Trustee, agree on the terms and conditions of the
Contribution Agreement as follows:
I. DEPOSIT AGREEMENT
1. Amount of the Deposit. The Contributor shall make a deposit (the “Deposit”) equivalent to
5.7 billion euro with the Deposit Account of the Trust (the “Original Amount of the Deposit”).
2. Denomination and Media.
(a) The Deposit shall be denominated in SDRs.
(b) The Deposit shall be made in euros.
(c) The Deposit shall be made by February 28, 2023.
3. Maturity. The Deposit shall mature on the tenth anniversary of the date on which the
Deposit is received (the “Maturity Date”).
4. Investment Strategy and Risk Disclosure.
(a) The resources from the Deposit shall be invested by the Trustee in accordance with the
guidelines adopted by the Trustee pursuant to Section VI, paragraph 4(a) of the RST Instrument.
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
40 INTERNATIONAL MONETARY FUND
(b) The assets of the Deposit Account are exposed to market and credit risk which could result
in potential losses to the Original Amount of the Deposit.
5. Notional Value of the Deposit. For the purposes set out in paragraphs 6, 7, 8, and 9 of this
Deposit Agreement, the Trustee shall calculate the notional value of the Deposit based on the
Original Amount of the Deposit and subsequently any attributed investment earnings or losses,
interest payments made to the Contributor pursuant to paragraph 6 of this Deposit Agreement, if
any, and transfers pursuant to paragraphs 8, 9, 10 or 11 of this Deposit Agreement or pursuant to
Section VI, paragraphs 1(c) and 4(b)(2) of the RST Instrument (“Notional Value”).
6. Interest.
(a) The Trustee shall pay the Contributor interest on the Original Amount of the Deposit less any
amounts repaid to the Contributor under paragraphs 8 or 9 or transferred by the Contributor under
paragraph 11 of this Deposit Agreement, at the SDR interest rate established by the Fund pursuant to
Article XX, Section 3 of the Fund’s Articles of Agreement. Interest shall accrue daily and shall be paid
promptly by the Trustee on a quarterly basis after January 31, April 30, July 31 and October 31 of each
year. Any payment of interest shall be limited to the Notional Value of the Deposit.
(b) Following any full repayment of the Deposit in accordance with paragraphs 7 and 9 of this
Deposit Agreement, all remaining interest under paragraph 6(a) above, shall be paid to the
Contributor at the end of the Fund financial quarter in which repayment was made.
7. Repayment upon Maturity. As soon as practicable but not later than 60 days following the
Maturity Date, the Trustee shall repay the Deposit to the Contributor. The repayment amount shall be
the lower of: (a) the Original Amount of the Deposit less any amount(s) repaid to the Contributor under
paragraphs 8 or 9 or transferred by the Contributor under paragraph 11 of this Deposit Agreement, and
(b) the Notional Value of the Deposit. To the extent any transfers are received in the Deposit Account
pursuant to Section VI, paragraph 1(c) of the RST Instrument after the Maturity Date, the Trustee shall
promptly repay to the Contributor any amounts received that are attributable to the Deposit.
8. Encashment. The Contributor shall have the right to seek early repayment of all or part of
the Deposit (the “Encashed Deposit”), pursuant to Section VI, paragraph 5(e) of the RST Instrument,
if the Contributor represents that the balance of payments and reserve position of Germany justifies
the early repayment in advance of the Maturity Date and the Trustee, having given the
representation the overwhelming benefit of any doubt, agrees. The Encashed Deposit amount may
not exceed the Original Amount of the Deposit less any amount(s) repaid to the Contributor under
this paragraph or paragraph 9 or transferred by the Contributor under paragraph 11 of this Deposit
Agreement, or the Notional Value of the Deposit, whichever is lower. The Contributor agrees to
restore an amount equivalent to the Encashed Deposit to the Deposit Account as soon as
practicable in light of favorable developments in Germany’s balance of payments and reserve
position, as evidenced by the inclusion of Germany’s currency in the Fund’s Financial Transactions
Plan.
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 41
9. Early Repayment.
(a) The Trustee may repay all or part of the Deposit at any time in advance of the Maturity Date;
provided that the repayment amount may not exceed the Original Amount of the Deposit less any
amount(s) repaid to the Contributor under this paragraph or paragraph 8 or transferred by the
Contributor under paragraph 11 of this Deposit Agreement, or the Notional Value of the Deposit,
whichever is lower; and provided further that the Trustee will give notice to the Contributor at least
five (5) Fund business days prior to any advance repayment.
(b) If Germany does not provide its consent to a proposed amendment to the RST Instrument as
required under Section X, paragraph 2(a) of the RST Instrument, the Contributor may request the
return of the Original Amount of the Deposit less any amount(s) repaid to the Contributor under this
paragraph and paragraph 8 or transferred by the Contributor under paragraph 11 of this Deposit
Agreement, or the Notional Value of the Deposit, whichever is lower, in accordance with Section X,
paragraph 2(c) of the RST Instrument.
10. Liquidation. If the Trustee decides to wind up the operations of the Trust, the Trust shall
repay the Deposit to the Contributor, at the Original Amount of the Deposit less any amount(s)
repaid to the Contributor under paragraphs 8 or 9 or transferred by the Contributor under
paragraph 11 of this Deposit Agreement, or the Notional Value of the Deposit, whichever is lower,
and in accordance with liquidation procedures under the RST Instrument, provided that the Trustee
shall notify the Contributor in advance of the repayment. Payment will be made to an account
specified by the Contributor, which can include an account or trust administered by the Fund.
11. Transfer of Claims.
(a) The Contributor shall have the right to transfer at any time all or part of any claim on the
Deposit Account to any member of the Fund, to the central bank or other fiscal agency designated
by such a member for purposes of Article V, Section 1 of the Fund’s Articles of Agreement, or to any
official entity that has been prescribed as a holder of SDRs pursuant to Article XVII, Section 3 of the
Fund’s Articles of Agreement.
(b) The transferee shall acquire all the rights of the Contributor under this Deposit Agreement
with respect to repayment of, and interest on, the transferred claim, except that the transferee shall
acquire the right to request early repayment under paragraph 8 of this Deposit Agreement only if it
is a member or the central bank or other fiscal agency of a member and, at the time of transfer, the
member’s balance of payments and reserve position was considered sufficiently strong in the
opinion of the Fund for its currency to be usable in transfers under the Fund’s Financial Transactions
Plan.
12. Modality of Interest Payment and Repayment of the Deposit.
(a) Payment of interest and repayments of the Deposit shall be made in euros. 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
42 INTERNATIONAL MONETARY FUND
(b) Payment of interest and repayment of the Deposit by the Trustee shall be made to an
account as agreed between the Trustee and the Contributor. If the due date is not a business day of
the Fund or the Target system or the domestic markets of the non-euro currencies included in the
SDR basket, the payment shall be made on the following business day for the Fund, the Target
system and the domestic markets of the non-euro currencies included in the SDR basket.
13. Exchange Rate. Unless otherwise agreed between the Contributor and the Trustee, all
transfers, exchanges, payments of interest, or repayments of the Deposit under this Deposit
Agreement in currencies shall be made at the exchange rates for the relevant currencies in terms of
the SDR established by the Fund pursuant to Article XIX, Section 7(a) of the Fund’s Articles of
Agreement and the rules and regulations of the Fund thereunder for the second (Fund) business day
before the value date of the transfer, exchange, payment of interest, or repayment.
14. SDR Basket. If the Fund changes the currency composition or currency weights of the SDR
basket, all transfers, exchanges, payments of interest, and repayments of the Deposit two or more
(Fund) business days after the effective date of the change shall be made on the basis of the new SDR
basket.
***********************************
II. RESERVE CONTRIBUTION AGREEMENT
1. Amount. The Contributor shall provide a contribution equivalent to 600 million euro to the
Reserve Account of the Trust (the “Contribution”).
2. Denomination and Media.
(a) The Contribution shall be denominated in SDRs.
(b) The Contribution shall be made in euros.
(c) The Contributor shall transfer the Contribution to an account specified by the Trustee by
February 28, 2023.
3. Investment Strategy and Risk Disclosure.
(a) The resources in the Reserve Account shall be invested by the Trustee in accordance with the
guidelines adopted by the Trustee pursuant to Section V, paragraph 3(a) of the RST Instrument.
(b) The assets of the Reserve Account are exposed to the market and credit risk which could
result in potential losses to the Contributor’s share in the resources of the Reserve Account.
4. Remuneration. No remuneration shall be paid to the Contributor in respect of the
Contribution. 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 43
5. Contributor’s Share in the Reserve Account. The Contributor shall have a proportional
share in the Reserve Account, based on its Contribution, any resources that are transferred from the
Deposit Account and attributed to the Contributor’s contribution to the Deposit Account pursuant
to Section VI, Paragraph 5(d) of the RST Instrument, and attributed investment earnings or losses in
the Reserve Account.
6. Transfer of the Contributor’s Share. The Contributor shall have the right to transfer at any
time all or part of its share in the Reserve Account to any member of the Fund, to any member’s
central bank or other fiscal agency designated by such a member for purposes of Article V, Section 1
of the Fund’s Articles of Agreement, or to any official entity that has been prescribed as a holder of
SDRs pursuant to Article XVII, Section 3 of the Fund’s Articles of Agreement. The transferee shall
acquire all the rights of the Contributor under this Reserve Contribution Agreement with respect to
the transferred amount.
7. Distribution of Resources in the Reserve Account.
(a) The Contribution shall have a maturity of 10 years from the date on which it is received.
(b) Upon the liquidation of the Trust prior to maturity, the Trustee shall distribute to the
Contributor its share in the resources of the Reserve Account of the Trust in accordance with Section
V, paragraph 4(c) of the RST Instrument.
(c) Prior to maturity and the liquidation of the Trust, and following the Trustee’s decision to
distribute a portion of the balances in the Reserve Account of the RST in accordance with Section V,
paragraph 4(d) of the RST Instrument, the Contributor shall receive part of this distribution in
proportion to its share in the Reserve Account.
(d) If Germany does not provide its consent to a proposed amendment to the RST Instrument as
required under Section X, paragraph 2(a) of the RST Instrument, the Contributor may request the
return of its share in the Reserve Account in accordance with Section X, paragraph 2(c) of the RST
Instrument.
8. Modality of Distribution.
(a) Unless otherwise agreed between the Trustee and the Contributor, any distribution to the
Contributor under this Reserve Contribution Agreement shall be made in euros.
(b) Distributions shall be made to an account as agreed between the Trustee and the
Contributor and on a business day of the Fund, the Target system and the domestic markets of the
non-euro currencies included in the SDR basket.
9. Exchange Rates. Unless otherwise agreed between the Trustee and the Contributor, all
transfers, exchanges, and distribution in currencies shall be made at the exchange rates for the
relevant currencies in terms of the SDR established by the Fund pursuant to Article XIX, Section 7(a)
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
44 INTERNATIONAL MONETARY FUND
of the Fund’s Articles of Agreement and the rules and regulations of the Fund thereunder for the
second (Fund) business day before the value date of the transfer, exchange or payment.
10. SDR Basket. If the Fund changes the currency composition or currency weights of the SDR
basket, all transfers, exchanges, and distribution made two or more (Fund) business days after the
effective date of the change shall be made on the basis of the new SDR basket.
***********************
III. FINAL PROVISIONS
1. Dispute Resolution. Any question arising under this Contribution Agreement shall be
settled by mutual agreement between the Contributor and the Trustee.
2. Execution. This Contribution Agreement may be executed in one or more duplicate
counterparts, each of which shall be deemed an original, and which together shall constitute one
and the same instrument.
3. Effectiveness. This Contribution Agreement shall become effective on the last date written
below.
[Signature page follows] 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 45
IN WITNESS WHEREOF, the Contributor and the International Monetary Fund, as Trustee of the
Resilience and Sustainability Trust, have executed this agreement.
For the Federal Republic of Germany:
October 11, 2022
______________________________ ______________________________
Christian Lindner /s Date
Federal Minister of Finance Germany

For the INTERNATIONAL MONETARY FUND as Trustee:
October 12, 2022
______________________________ ______________________________
Kristalina Georgieva /s Date
Managing Director
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
46 INTERNATIONAL MONETARY FUND
Attachment V. Contribution Agreement between the Government
of Japan and the International Monetary Fund,
as Trustee of the Resilience and Sustainability Trust
WHEREAS the International Monetary Fund (the “Fund”) has established the Resilience and
Sustainability Trust (the “Trust”) to assist in fulfilling the purposes of the Fund by providing loans
under the Resilience and Sustainability Facility of the Trust (the “RSF”) to eligible Fund members to
enhance their economic resilience and sustainability;
WHEREAS, Japan has committed to contribute financial resources to the loan account (the “Loan
Account”), the deposit account (the “Deposit Account”) and the reserve account (the “Reserve
Account”) of the Trust in accordance with the provisions of the Instrument to Establish the Resilience
and Sustainability Trust (the “RST Instrument”); and
WHEREAS, to fulfill Japan’s commitment, the Government of Japan (the “Contributor”) and the Fund,
as Trustee of the Trust, have reached understandings on a loan account note purchase agreement
(the “Loan Account Note Purchase Agreement”), a deposit account note agreement (the “Deposit
Account Note Agreement”), and a reserve contribution agreement (the “Reserve Contribution
Agreement”), jointly referred to as the “Contribution Agreement”;
NOW, THEREFORE, the Contributor and the Fund, as Trustee of the Trust, agree on the terms and
conditions of these contributions as follows:
I. Loan Account Note Purchase Agreement
1. Promissory Notes. The Contributor agrees to purchase from the Trustee promissory notes,
which shall be issued in accordance with the RST Instrument, the terms of this Loan Account Note
Purchase Agreement and the General Terms and Conditions attached as Annex 1 to the Contribution
Agreement (the “Loan Account Notes”).
2. Amount. Loan Account Notes in a total, SDR-denominated, principal amount up to the
equivalent of United States Dollars (USD) 835 million may be issued under this Loan Account Note
Purchase Agreement. This SDR-denominated amount will be determined at the exchange rate for
USD in terms of the SDR established by the Fund pursuant to Article XIX, Section 7(a) of the Fund’s
Articles of Agreement and the rules and regulations of the Fund thereunder as of the second (Fund)
business day before the value date of the transfer by the Contributor of its contribution to the
Reserve Account under the Reserve Contribution Agreement.
3. Issuance and Purchase of Loan Account Notes.
(a) The Trustee may issue, and the Contributor shall purchase, Loan Account Notes at
any time from the effective date of this Loan Account Note Purchase Agreement as set forth in 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 47
paragraph 3 of the Final Provisions below, through November 30, 2030 to finance loan
disbursements under the RSF.
(b) The Trustee may issue, and the Contributor shall purchase, Loan Account Notes to
finance early repayment of outstanding Trust borrowing under other borrowing agreements with
the Loan Account pursuant to Section IV, Paragraph 3(b) of the RST Instrument for as long as claims
under such borrowing agreements remain outstanding.
(c) The issuance of Loan Account Notes for purchase under this Loan Account Note
Purchase Agreement shall be suspended following the Contributor’s request for early repayment
because of Japan’s balance of payments and reserve position or when the currency of Japan is no
longer included for transfers in the Fund’s Financial Transactions Plan, in accordance with Section
IV, paragraph 4 of the RST Instrument.
(d) If Japan does not provide its consent to a proposed amendment to the RST
Instrument pursuant to Section X, paragraph 2(a) of the RST Instrument, further issuance of Loan
Account Notes for purchase under this Loan Account Note Purchase Agreement to finance any new
commitment of loans under the RSF that is made after the effectiveness of the proposed
amendment shall be suspended.
(e) If principal or interest under a Loan Account Note is not paid fully within a period of
ten Fund business days after its due date, the Trustee shall not issue further Loan Account Notes
under this Loan Account Note Purchase Agreement pending consultations with the Contributor on
the matter. However, the Trustee may resume issuances of Loan Account Notes under this Loan
Account Note Purchase Agreement once all arrears under the note have been discharged.
(f) Unless otherwise agreed between the Contributor and the Trustee, the Trustee shall
give the Contributor at least seven (Tokyo) business days’ notice of the Trustee’s intention to issue
Loan Account Notes for purchase under this Loan Account Note Purchase Agreement, including the
amount of the Loan Account Notes to be purchased, and shall provide payment instructions at least
two (Fund) business days prior to the value date of the transaction by SWIFT.
4. Denomination and Purchase Price. Loan Account Notes shall be denominated in special
drawing rights (SDRs). The purchase price for each Loan Account Note shall be 100 percent of the
principal amount thereof.
5. Payment for Purchase of Loan Account Notes, Exchange Rate, and SDR Basket.
(a) Unless otherwise agreed between the Trustee and the Contributor, the purchase
price of each Loan Account Note shall be paid by the Contributor on the value date specified in the
Trustee’s notice under paragraph 3(f) of this Loan Account Note Purchase Agreement by transfer of
SDRs to an account specified by the Trustee. 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
48 INTERNATIONAL MONETARY FUND
(b) Unless otherwise agreed between the Trustee and the Contributor, all purchases of
Loan Account Notes where the purchase price is paid in currency and all exchanges of currency
pursuant to this Loan Account Note Purchase Agreement shall be made at the exchange rate for
the relevant currency in terms of the SDR established by the Fund pursuant to Article XIX, Section
7(a) of the Fund’s Articles of Agreement and the rules and regulations of the Fund thereunder for
the second (Fund) business day before the value date of the purchase or exchange. If this exchange
rate determination date is not a business day in Tokyo, then such date shall be the last preceding
business day of the Fund that is also a business day in Tokyo.
(c) If the Fund changes the currency composition or currency weights of the SDR basket,
all payment for purchase of Loan Account Notes made two or more (Fund) business days after the
effective date of the change shall be made on the basis of the new SDR basket.
6. Transfer of Loan Account Notes. The Contributor shall have the right to transfer all or part of
a Loan Account Note held by it in accordance with Section VII of the RST Instrument and the General
Terms and Conditions for Loan Account Notes attached as Annex 1 to the Contribution Agreement.
II. Deposit Account Note Agreement
1. Promissory Note. The Contributor agrees to make a contribution to the Deposit Account
through the purchase from the Trustee of a promissory note with a SDR-denominated principal
amount equivalent to USD 167 million (the “Deposit Account Note”), which shall be issued in
accordance with the RST Instrument and this Deposit Account Note Agreement, and be subject to
the Terms and Conditions attached as Annex 3 to the Contribution Agreement. This SDRdenominated amount will be determined at the exchange rate for USD in terms of the SDR
established by the Fund pursuant to Article XIX, Section 7(a) of the Fund’s Articles of Agreement and
the rules and regulations of the Fund thereunder as of the second (Fund) business day before the
value date of the transfer by the Contributor of its contribution to the Reserve Account under the
Reserve Contribution Agreement.
2. Denomination, Purchase Price and Issuance and Purchase of Deposit Account Note.
(a) The Deposit Account Note shall be denominated in SDRs.
(b) The purchase price for the Deposit Account Note shall be 100 percent of the
principal amount thereof.
(c) The Trustee shall issue, and the Contributor shall purchase, the Deposit Account
Note promptly on or after the effectiveness of this Deposit Account Note Agreement.
(d) Unless otherwise agreed between the Contributor and the Trustee, the Trustee shall
give the Contributor at least seven (Tokyo) business days’ notice of the Trustee’s intention to issue
the Deposit Account Note for purchase under this Deposit Account Note Agreement, including the
amount of the Deposit Account Note to be purchased, and shall provide payment instructions at
least two (Fund) business days prior to the value date of the transaction by SWIFT. 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 49
(e) The purchase price for the Deposit Account Note shall be paid in either SDRs or a
freely usable currency, as determined by the Contributor.
(f) Unless otherwise agreed between the Trustee and the Contributor, all transfers,
exchange, and payments of principal and interest in currencies pursuant to this Deposit Account
Note Agreement shall be made at the exchange rate for the relevant currency in terms of the SDR
established by the Fund pursuant to Article XIX, Section 7(a) of the Fund’s Articles of Agreement and
the rules and regulations of the Fund thereunder for the second (Fund) business day before the
value date of the purchase or exchange. If this exchange rate determination date is not a business
day in Tokyo, then such date shall be the last preceding business day of the Fund that is also a
business day in Tokyo.
3. Investment Strategy and Risk Disclosure.
(a) The resources in the Deposit Account shall be invested by the Trustee in accordance with the
guidelines adopted by the Trustee pursuant to Section VI, paragraph 4(a) of the RST Instrument.
(b) The assets of the Deposit Account are exposed to market and credit risk which could result
in potential losses to the value of the Deposit Account Note.
III. Reserve Contribution Agreement
1. Amount. The Contributor shall provide a contribution of an SDR-denominated amount
equivalent to USD 16.7 million (the “Contribution”), with the SDR-denominated amount to be
determined pursuant to paragraph 9 of this Reserve Contribution Agreement as of the second
(Fund) business day before the value date of the transfer of the Contribution to the Reserve Account
of the Trust.
2. Denomination and Media.
(a) The Contribution shall be denominated in SDRs.
(b) The Contribution shall be made in either SDRs or a freely usable currency, as determined by
the Contributor.
(c) The Contributor shall transfer the Contribution to an account specified by the Trustee on or
promptly after the effectiveness of this Reserve Contribution Agreement.
3. Investment Strategy and Risk Disclosure.
(a) The resources in the Reserve Account shall be invested by the Trustee in accordance with the
guidelines adopted by the Trustee pursuant to Section V, paragraph 3(a) of the RST Instrument.
(b) The assets of the Reserve Account are exposed to market and credit risk which could result
in potential losses to the Contributor’s share in the resources of the Reserve Account. 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
50 INTERNATIONAL MONETARY FUND
4. Remuneration. No remuneration shall be paid to the Contributor in respect of the
Contribution.
5. Contributor’s Share in the Reserve Account. The Contributor shall have a proportional
share in the Reserve Account, based on its Contribution, any resources that are transferred from the
Deposit Account and attributed to the Contributor’s contribution to the Deposit Account pursuant
to Section VI, Paragraph 5(d) of the RST Instrument, and attributed investment earnings or losses in
the Reserve Account.
6. Transfer of the Contributor’s Share. The Contributor shall have the right to transfer at
anytime all or part of its share in the Reserve Account to any member of the Fund, to any member’s
central bank or other fiscal agency designated by such a member for purposes of Article V, Section 1
of the Fund’s Articles of Agreement, or to any official entity that has been prescribed as a holder of
SDRs pursuant to Article XVII, Section 3 of the Fund’s Articles of Agreement. The transferee shall
acquire all the rights of the Contributor under this Reserve Contribution Agreement with respect to
the transferred amount.
7. Distribution of Resources in the Reserve Account.
(a) The Contribution shall have no fixed maturity.
(b) Upon the liquidation of the Trust, the Trustee shall distribute to the Contributor its share in
the resources of the Reserve Account of the Trust in accordance with Section V, paragraph 4(c) of
the RST Instrument.
(c) Prior to the liquidation of the Trust and following the Trustee’s decision to distribute a
portion of the balances in the Reserve Account of the RST in accordance with Section V, paragraph
4(d) of the RST Instrument, the Contributor shall receive part of this distribution in proportion to its
share in the Reserve Account.
(d) If Japan does not provide its consent to a proposed amendment to the RST Instrument as
required under Section X, paragraph 2(a) of the RST Instrument, the Contributor may request the
return of a portion of its share in the Reserve Account based on the ratio of remaining uncommitted
loan resources to total loan commitments to the Trust under the Loan Account Note Purchase
Agreement in accordance with Section X, paragraph 2(b) of the RST Instrument.
8. Modality of Distribution.
(a) Unless otherwise agreed between the Trustee and the Contributor, any distribution to the
Contributor under this Reserve Contribution Agreement shall be made in SDRs.
(b) Distribution in SDRs shall be made by crediting Japan’s holdings account in the Special
Drawing Rights Department on a business day of the Fund. 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 51
(c) Distribution in currencies shall be made to an account as agreed between the Trustee and
the Contributor and on the business day of the Fund, the Target system and the domestic markets
of the non-euro currencies included in the SDR basket.
9. Exchange Rates. Unless otherwise agreed between the Trustee and the Contributor, all
transfers, exchanges, and distribution in currencies shall be made at the exchange rates for the
relevant currencies in terms of the SDR established by the Fund pursuant to Article XIX, Section 7(a)
of the Fund’s Articles of Agreement and the rules and regulations of the Fund thereunder for the
second (Fund) business day before the value date of the transfer, exchange, or payment.
10. SDR Basket. If the Fund changes the currency composition or currency weights of the SDR
basket, all transfers, exchanges, and distribution made two or more (Fund) business days after the
effective date of the change shall be made on the basis of the new SDR basket.
IV. Final Provisions
1. Dispute Resolution: Any question arising under this Contribution Agreement shall be
settled by mutual agreement between the Contributor and the Trustee.
2. Execution: This Contribution Agreement may be executed in one or more duplicate
counterparts, each of which shall be deemed an original, and which together shall constitute one
and the same instrument.
3. Effectiveness: This Contribution Agreement shall become effective on the last date written
below; provided, however, that Section I (Loan Account Note Purchase Agreement) shall become
effective on the last date written below or the date the Managing Director of the Fund has notified
the Executive Board of the Fund that the RST has become operational, whichever is later.
[Signature page follows]
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
52 INTERNATIONAL MONETARY FUND
IN WITNESS WHEREOF, the Contributor and the Trustee have executed this Contribution
Agreement.
For the GOVERNMENT OF JAPAN:
 October 4, 2022
 Shunichi Suzuki /s Date
Minister of Finance

For the INTERNATIONAL MONETARY FUND, as TRUSTEE OF THE RESILIENCE AND SUSTAINABILITY
TRUST:

 October 6, 2022
 Kristalina Georgieva /s Date
Managing Director 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 53
Annex 1
General Terms and Conditions
for Notes Issued Under the Loan Account Note Purchase Agreement Between the
International Monetary Fund as Trustee of the Resilience and Sustainability Trust and
the Government of Japan
These General Terms and Conditions are issued in accordance with the Instrument to Establish the
Resilience and Sustainability Trust (the “Trust”), adopted by Executive Board Decision No.
17231(22/37), adopted April 13, 2022, and the Loan Account Note Purchase Agreement, which is a
constituent part of the Contribution Agreement between the International Monetary Fund (the
“Fund”) as Trustee (the “Trustee”) of the Trust and the Government of Japan (the “Contributor”), to
which they are attached.
1. Definitions. The following definitions apply for purposes of these General Terms and
Conditions:
(a) “Permitted Holder” means (i) the Contributor, (ii) a member of the Fund, (iii) the
central bank or other fiscal agency designated by a member of the Fund for purposes of Article V,
Section 1 of the Fund’s Articles of Agreement (“fiscal agency of a member”), and (iv) an official entity
that has been prescribed as a holder of special drawing rights (SDRs) pursuant to Article XVII,
Section 3 of the Fund’s Articles of Agreement (“Prescribed SDR holder”). Permitted Holders shall be
the only authorized holders of Notes.
(b) “Notes” mean the promissory notes that are subject to these General Terms and
Conditions.
(c) “Relevant Member” means, in the case of a Permitted Holder that is the central bank,
fiscal agency or other official institution of a member of the Fund, that member of the Fund.
2. Form, Delivery and Custody of Notes.
(a) Notes will only be issued in book entry form. The Trustee will establish and maintain
in its records a book entry account in the name of each Permitted Holder recording pertinent details
of all Notes issued, including the number, issue date, principal amount, and maturity date. As of the
value date of each purchase or transfer of a Note, the Trustee will make an appropriate entry in its
records regarding details of the Note purchased or transferred. The making of such an entry in the
records of the Trustee shall constitute a taking of delivery of the Note by the Contributor or
transferee.
(b) Upon the request of a Permitted Holder, the Trust will issue to the Permitted Holder
a registered Note substantially in the form set out in Annex 2 to the Contribution Agreement,
including without limitation the legend regarding restrictions on transfer of Notes. Each such
registered Note will bear as its issue date the value date of the purchase of the Note and shall be 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
54 INTERNATIONAL MONETARY FUND
issued in the name of the relevant Permitted Holder. Unless otherwise agreed between a Permitted
Holder and the Trustee, the Trustee will keep registered Notes in custody for the Permitted Holder,
and acceptance of custody by the Trustee shall constitute delivery of Notes to the Permitted Holder.
3. Maturity
(a) Except as otherwise provided in this paragraph 3 and in paragraph 6(b) of these
General Terms and Conditions, each Note shall have a maturity date that is six months from its issue
date. The Trustee may in its sole discretion elect to extend the maturity date of any Note or of any
portion thereof by additional periods of up to six months after the initial maturity date, which
extension the Trustee shall automatically be deemed to have elected with respect to the maturity
dates then in effect for all Notes unless, at least five (Fund) business days before a maturity date, the
Trustee notifies a Permitted Holder in writing that the Trustee does not elect to extend the maturity
date of the particular Note or portion thereof; provided however that, in no event, shall the maturity
date of any Note be extended to a date that is later than the maturity dates for the loan
disbursement under the Resilience and Sustainability Facility of the Trust for which it was issued. For
purposes of the preceding sentence, Notes issued in connection with the early repayment of other
Trust borrowing pursuant to Section IV, paragraph 3(b) of the RST Instrument, shall have the
maximum maturity dates associated with such other borrowing. At the time of issuance of a Note,
the Trustee shall provide information on the specific repayment schedule for the Note issued.
(b) The Trustee shall pay the principal amount of each Note on the maturity date that is
applicable to that Note in accordance with subparagraph (a) of this paragraph 3.
(c) The Trustee may make an early repayment in part or in full of the principal amount
of any Note at any time in advance of maturity upon its receipt of early repayment of the
corresponding loan disbursement under the RSF, provided that the Trustee notifies a Permitted
Holder at least five (Fund) business days before any such early repayment.
(d) The Trustee will cancel a Note upon repayment of the principal amount of the Note
and all accrued interest. If the Trustee makes early repayment of less than the principal amount of a
Note, the Trustee will cancel the Note and issue a new Note for the remainder of the amount, with
the same maturity date as the cancelled Note. If the maturity date of a Note is extended with
respect to less than the principal amount of the Note, the Trustee will cancel the Note and issue a
new Note for the remainder of the amount, with the extended maturity date.
(e) Any Note to be cancelled by the Trustee that is not already in the custody of the
Trustee shall be surrendered by the Permitted Holder to the Trustee for cancellation.
4. Rate of Interest.
(a) Interest on each Note shall be computed at the SDR interest rate established by the
Fund pursuant to Article XX, Section 3 of the Fund’s Articles of Agreement. 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 55
(b) The amount of interest payable on each Note shall be calculated on an actual day
basis and shall be paid on a quarterly basis promptly after July 31, October 31, January 31, and April
30 of each year.
5. Transferability of Notes.
(a) A Permitted Holder has the right to transfer at any time all or part of any Note to
any member of the Fund, to the central bank or other fiscal agency of such a member, or to a
Prescribed SDR holder.
(b) The transferee shall, as a condition of the transfer under this paragraph 5, assume all
the obligations of the transferor with respect to extensions of the maturity of the Note pursuant to
paragraph 3(a) of these General Terms and Conditions. Any Note or part thereof transferred
pursuant to this paragraph 5 shall be held by the transferee on the same terms and conditions as
the Note was held by the transferor, except as provided in paragraph 6 of these General Terms and
Conditions with respect to the right to early payment at the request of a Permitted Holder.
(c) The price of a Note transferred pursuant to this paragraph 5 will be as agreed
between the transferor and the transferee.
(d) The transferor of a Note shall inform the Trustee promptly of all or part of the Note
that is being transferred, the name of the transferee, the agreed price for the transfer, and the value
date of the transfer. The transfer shall be recorded by the Trustee and the transferee shall become
the holder of the Note only if such transfer is in accordance with this paragraph 5. Subject to the
foregoing, upon recording, the transfer shall be effective as of the value date agreed between the
transferee and the transferor.
(e) For all transfers under this paragraph 5, the Trustee will cancel the Note that has
been transferred in whole or in part and, if the Note is a registered Note, the transferor shall, as a
condition for the transfer, surrender for cancellation any such registered Note that is not already in
the custody of the Trustee. Upon cancellation of the relevant Note, the Trustee will issue a new Note
in the name of the transferee for the principal amount transferred and, where appropriate, a new
Note in the name of the transferor for any part of the principal amount retained by it. The issue date
of each new Note will be the issue date of the cancelled Note, and the new Notes will have the same
maturity date (including any maturity date resulting from extensions of a previous maturity date)
that is applicable to the cancelled Note pursuant to paragraph 3 of these General Terms and
Conditions. The form and delivery of each new Note will be as specified in paragraph 2 of these
General Terms and Conditions.
(f) If all or part of a Note is transferred, the Trustee will pay the full interest payment for
that period to the transferee.
(g) Derivative transactions in respect of any Note, and transfers of participation interests
in any Note, are prohibited. 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
56 INTERNATIONAL MONETARY FUND
6. Early Payment by the Trust at Request of Certain Permitted Holders.
(a) The rights provided in subparagraph (b) of this paragraph 6 shall apply only to a
Permitted Holder that is: (A) the Contributor, or (B) a member of the Fund or the central bank or
other fiscal agency of a member of the Fund that has acquired the Notes by transfer if, at the time
of such transfer, the balance of payments and reserve position of the member or Relevant Member,
as the case may be, was considered sufficiently strong in the opinion of the Fund for its currency to
be usable in transfers under the Financial Transactions Plan.
(b) If a Permitted Holder covered under subparagraph (a) of this paragraph 6 represents
that its balance of payments and reserve position (the balance of payments and reserve position of
the Relevant Member if the Permitted Holder is the central bank or other fiscal agency of a member
or other official institution of a member) justifies early payment of all or part of the principal amount
of the Permitted Holder’s Notes covered by subparagraph (a) of this paragraph 6, and if the Trustee,
having given this representation the overwhelming benefit of any doubt, agrees, then the Trust shall
pursuant to Section IV, paragraph 3(b) of the RST Instrument pay the principal amount of the
Permitted Holder’s Notes covered by subparagraph (a) of this paragraph 6 that is then outstanding,
up to the amount for which early payment has been requested, and subject to the availability or
resources under borrowing agreements and note purchase agreements to finance the Loan Account
of the Trust.
7. Media and Modalities of Payments on the Notes.
(a) Unless otherwise agreed between the Trustee and the Permitted Holder, payments of
the principal amount of the Notes and of interest on the Notes by the Trustee shall be made in SDRs.
(b) Payments in SDRs by the Trustee shall be made by crediting the holdings account in
the Special Drawing Rights Department of the Permitted Holder or of the Relevant Member, as the
case may be. If the due date is not a business day of the Fund, the payment shall be made on the
following business day of the Fund.
(c) Payments by the Trustee in currencies shall be made to an account as agreed
between the Trustee and the Permitted Holder. If the due date is not a business day of the Fund or
the Target system or the domestic markets of the non-euro currencies included in the SDR basket,
the payment shall be made on the following business day for the Fund, the Target system and the
domestic markets of the non-euro currencies included in the SDR basket.
8. Effective Exchange Rate for Payments.
Unless otherwise agreed between a Permitted Holder and the Trustee, all payments in currency of
principal and interest under these General Terms and Conditions will be made at the exchange rates
for the relevant currencies in terms of the SDR established pursuant to Article XIX, Section 7(a) of
the Fund’s Articles of Agreement and the rules and regulations of the Fund thereunder for the
second (Fund) business day before the value date of the payment. If this exchange rate 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 57
determination date is not a business day in the Permitted Holder’s principal location, then such date
will be the last preceding business day of the Fund that is also a business day in the Permitted
Holder’s principal location.
9. Changes in Method of Valuation of SDR.
If the Fund changes the currency composition or currency weights of the SDR basket, all transfers,
exchanges, and payments of principal and interest on the Notes made two or more (Fund) business
days after the effective date of the change will be made on the basis of the new SDR basket.
10. Settlement of Questions.
Any question arising under any Note shall be settled by mutual agreement between the relevant
Permitted Holder and the Trustee. 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
58 INTERNATIONAL MONETARY FUND
Annex 2
Form of Registered Loan Account Note
Number SDR
RESLIENCE AND SUSTAINABILITY TRUST REGISTERED LOAN
ACCOUNT NOTE
Issue Date:
Maturity Date: [to include description of automatic extensions of maturity]
The RESLIENCE AND SUSTAINABILITY (“the Trust”), for value received, hereby promises to pay
to
 , being the registered holder of this note an amount equivalent to
Special Drawing Rights (SDR)
on the maturity date specified above and to pay interest thereon as set forth below.
This Note is issued in accordance with the General Terms and Conditions for Loan
Account Notes issued under the Loan Account Note Purchase Agreement between the
International Monetary Fund (the “Fund”), as Trustee of the Resilience and Sustainability Trust
(the “Trustee”) and the Government of Japan dated [mm/dd/yy] (the “General Terms and
Conditions”). Each holder of this Note is deemed to have agreed to the General Terms and
Conditions and relevant terms of the Loan Account Note Purchase Agreement, including without
limitation the maturity date (including the terms on which it may be extended at the sole option of
the Trustee from time to time), the interest rate, the terms and conditions for early payment at the
request of the Trustee or the holder hereof, and the terms and conditions for transfer of this Note or
any part thereof.
THIS NOTE HAS NOT BEEN REGISTERED UNDER THE SECURITIES LAWS OF ANY JURISDICTION.
IN NO EVENT SHALL ANY HOLDER OF THIS NOTE SELL, ASSIGN, DISPOSE OF OR OTHERWISE
TRANSFER THIS NOTE, DIRECTLY OR INDIRECTLY, TO ANY ENTITY THAT IS NOT (I) A MEMBER
OF THE FUND (II) THE CENTRAL BANK OR OTHER FISCAL AGENCY DESIGNATED BY SUCH A
MEMBER OF THE FUND FOR PURPOSES OF ARTICLE V, SECTION 1 OF THE FUND’S ARTICLES OF
AGREEMENT, OR (III) AN OFFICIAL ENTITY THAT HAS BEEN PRESCRIBED AS A HOLDER OF SPECIAL
DRAWING RIGHTS PURSUANT TO ARTICLE XVII, SECTION 3 OF THE FUND’S ARTICLES OF
AGREEMENT.
ANY DERIVATIVE TRANSACTIONS IN RESPECT OF THIS NOTE, AND TRANSFERS OF PARTICIPATION
INTERESTS IN THIS NOTE, ARE PROHIBITED. 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 59
Interest shall accrue daily on the principal amount of this Note at the SDR interest rate
established by the Fund pursuant to Article XX, Section 3 of the Fund’s Articles of Agreement.
Interest shall be calculated on an actual day basis and shall be paid on a quarterly basis promptly
after July 31, October 31, January 31, and April 30 of each year. Payments of interest shall be made
in SDRs, or such currency as may be agreed upon between the registered holder and the Trustee.
Payments in SDRs shall be made by crediting the SDR Department account of the registered holder
(if any), or of the member for which the registered holder serves as central bank or other fiscal
agency or other official institution in the case of registered holders that are the central bank or other
fiscal agency or other official institution of a member of the Fund.
[Signatures]
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
60 INTERNATIONAL MONETARY FUND
Annex 3
Terms and Conditions for the Note issued pursuant to the Deposit Account Note
Agreement between the International Monetary Fund, as Trustee of the Resilience
and Sustainability Trust and the Government of Japan
These Terms and Conditions are issued in accordance with the Instrument to Establish the Resilience
and Sustainability Trust (the “RST Instrument”), adopted by Executive Board Decision No.
17231-(22/37), adopted April 13, 2022 (as may be amended), and the Deposit Account Note
Agreement, which is a constituent part of the Contribution Agreement between the International
Monetary Fund (the “Fund”) as Trustee (the “Trustee”) of the Resilience and Sustainability Trust (the
“Trust”) and the Government of Japan (the “Contributor”).
1. Definitions. The following definitions apply for the purposes of these Terms and Conditions:
(a) “Holder” means the Contributor or the most recent transferee. The Holder must
be a Permitted Holder.
(b) “Maturity Date” means November 30, 2050.
(c) “Note” means any promissory note that is subject to these Terms and Conditions.
(d) “Notional Value” of a Note means the Principal Amount of that Note as adjusted
for any attributed investment earnings or losses, interest payments made under paragraph 3 of
these Terms and Conditions, and any attributed transfers pursuant to Section VI, paragraph 1(c) and
paragraph 4(b)(2) of the Instrument to Establish the Resilience and Sustainability Trust (the " Trust
Instrument").
(e) “Permitted Holder” means (i) the Contributor, (ii) a member of the Fund, (iii) the
central bank or other fiscal agency designated by a member of the Fund for purposes of Article V,
Section 1 of the Fund’s Articles of Agreement, and (iv) an official entity that has been prescribed as a
holder of special drawing rights (“SDRs”) pursuant to Article XVII, Section 3 of the Fund’s Articles of
Agreement. Permitted Holders shall be the only authorized holders of Note.
(f) “Principal Amount” means: (i) for the Note issued to the Contributor in accordance
with the Deposit Account Note Agreement between the Trustee and the Contributor, the original
principal amount of contribution made by the Contributor to the Deposit Account of the Trust under
that agreement, which shall be reduced upon any repayment by the Trustee to the Contributor
pursuant to subparagraph 4(b) and paragraph 5 of these Terms and Conditions or any transfers by
the Contributor to a transferee pursuant to paragraph 7 of these Terms and Conditions, and
increased by amounts restored to the Deposit Account of the Trust by the Contributor pursuant to
subparagraph 4(c) of these Terms and Conditions; and (ii) for a Note issued to any Holder as a result
of transfer under paragraph 7 of these Terms and Conditions, the transferred amount, which shall be
recorded as all or part of the Principal Amount of the Note that was held by the transferor prior to
the transfer, and which shall be reduced upon any repayment by the Trustee to the Holder pursuant 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 61
to subparagraph 4(b) and paragraph 5 of these Terms and Conditions or any transfers by the Holder
to a transferee pursuant to paragraph 7 of these Terms and Conditions, and increased by amounts
restored to the Deposit Account of the Trust by the Holder pursuant to subparagraph 4(c) of these
Terms and Conditions.
(g) “Relevant Member” means, in the case of a Holder that is the central bank, fiscal
agency or other official institution of a member of the Fund, that member of the Fund.
2. Form, Delivery and Custody of Notes.
(a) Notes will only be issued in book entry form. The Trustee will establish and
maintain in its records a book entry account in the name of each Holder recording pertinent details
of all Notes issued, including each Note’s number, issue date and Principal Amount. As of the value
date of the purchase or any transfer of a Note, the Trustee will make an appropriate entry in its
records regarding details of the Note purchased or transferred. The making of such an entry in the
records of the Trustee shall constitute a taking of delivery of the Note by the Contributor or
transferee.
(b) Upon the request of a Permitted Holder, the Trustee will issue to the Permitted
Holder a registered Note substantially in the form set out in Annex 4 to the Contribution
Agreement, including without limitation the legend regarding restrictions on transfer of Notes. Each
such registered Note will bear as its issue date the value date of the purchase of the Note and shall
be issued in the name of the relevant Holder. Unless otherwise agreed between a Holder and the
Trustee, the Trustee will keep registered Notes in custody for the Holder, and acceptance of custody
by the Trustee shall constitute delivery of Notes to the Holder.
3. Interest.
(a) Subject to subparagraph (c) of this paragraph 3, interest accrues on the Principal
Amount of the Note at the SDR interest rate established by the Fund pursuant to Article XX, Section
3 of the Fund’s Articles of Agreement.
(b) The amount of interest payable on the Note shall be calculated on an actual day basis and
shall be paid on a quarterly basis promptly after July 31, October 31, January 31, and April 30 of each year.
(c) Any payment of interest shall be limited to the Notional Value of the Note.
4. Encashment.
(a) The rights provided in subparagraph (b) of this paragraph 4 shall apply only to a
Holder that is: (i) the Contributor or (ii) a member of the Fund or the central bank or other fiscal
agency of a member of the Fund that has acquired the Notes by transfer if, at the time of such
transfer, the balance of payments and reserve position of the member or the Relevant Member, as
the case may be, was considered sufficiently strong in the opinion of the Fund for its currency to be
usable in transfers under the Financial Transactions Plan. 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
62 INTERNATIONAL MONETARY FUND
(b) A Holder covered under subparagraph (a) of this paragraph 4 shall have the right
to seek early repayment of all or part of the Principal Amount of a Note (the “Encashed Amount”),
pursuant to Section VI, paragraph 5(e) of the RST Instrument, if the Holder represents that its
balance of payments and reserve position (the balance of payments and reserve position of the
Relevant Member if the Holder is the central bank or other fiscal agency or other official institution
of a member) justifies the early payment in advance of the Maturity Date and the Trustee, having
given this representation the overwhelming benefit of any doubt, agrees. The Encashed Amount
may not exceed the Notional Value of the Note.
(c) By exercising the right described in subparagraph (b) of this paragraph 4, the
Holder agrees to restore an amount equivalent to the Encashed Amount to the Deposit Account of
the Trust as soon as practicable in light of favorable developments in its balance of payments and
reserve position (the balance of payments and reserve position of the Relevant Member if the
Holder is the central bank or other fiscal agency or other official institution), as evidenced by the
inclusion of its or the Relevant Member’s, as applicable, currency in the Fund’s Financial Transactions
Plan. When the Encashed Amount is restored to the Deposit Account of the Trust, the Principal
Amount of the Note will be increased accordingly.
(d) The Principal Amount and the Notional Value of a Note shall be adjusted for the
amount repaid upon early repayment under subparagraph (b) of this paragraph 4 or for the
Encashed Amount that is restored in accordance with subparagraph (c) of this paragraph 4.
(e) For all early repayments under subparagraph (b) of this paragraph 4, the Trustee
will cancel the Note that has been repaid in whole or in part and, if the Note is a registered Note,
the Holder shall, as a condition for the repayment, surrender for cancellation any such registered
Note that is not already in the custody of the Trustee. Upon cancellation of the relevant Note, the
Trustee will issue a new Note for any part of the Principal Amount not repaid. In the event an
Encashed Amount is restored to the Deposit Account of the Trust pursuant to subparagraph (c) of
this paragraph 4, the Trustee will cancel the Note, and if the canceled Note was a registered Note,
the Holder shall, as a condition for the issuance of the new Note, surrender for cancellation any such
registered Note that is not already in the custody of the Trustee. Upon cancellation of the relevant
Note, the Trustee will issue a new Note that reflects the restored amount. In either case, the issue
date of the new Note will be the issue date of the cancelled Note, and the form and delivery of each
new Note will be as specified in paragraph 2 of these Terms and Conditions.
5. Early Repayment.
(a) The Trustee may redeem any Note, in whole or in part, at any time in advance of
the Maturity Date; provided that the repayment amount may not exceed the Notional Value of the
Note; and provided further that the Trustee will give notice to the Contributor at least five (5) Fund
business days prior to any redemption.
(b) If Japan does not provide its consent to a proposed amendment to the RST
Instrument as required under Section X, paragraph 2(a) of the RST Instrument and the Contributor is 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 63
the Holder of a Note, a portion of such Note will be repayable at the option of the Contributor;
provided that the repayment amount may not exceed the Notional Value of the Note; and provided
further that the repayable portion of the Note will be based on the ratio of remaining unissued
notes to total note purchase commitments under the Contributor’s Loan Account Note Purchase
Agreement in accordance with Section X, paragraph 2(b) of the RST Instrument.
(c) Upon early repayment of any Note under this paragraph 5, the Principal Amount
and the Notional Value of the Note shall both be adjusted for the amount repaid.
(d) For all early repayments under this paragraph 5, the Trustee will cancel the Note
that has been repaid in whole or in part and, if the Note is a registered Note, the Holder shall, as a
condition for the repayment, surrender for cancellation any such registered Note that is not already
in the custody of the Trustee. Upon cancellation of the relevant Note, the Trustee will issue a new
Note for any part of the Principal Amount not repaid. The issue date of the new Note will be the
issue date of the cancelled Note. The form and delivery of each new Note will be as specified in
paragraph 2 of these Terms and Conditions.
6. Liquidation. If the Trustee decides to wind up the operations of the Trust, the Trustee shall
redeem 100 percent of the Principal Amount of any Note in accordance with the liquidation
procedures under the RST Instrument; provided that the repayment amount may not exceed the
Notional Value of any Note; and provided further that the Trustee shall notify the Holder in advance
of the redemption. Payment will be made to an account specified by the Holder, which can include
an account or trust administered by the Fund. Following a redemption under this paragraph 6, the
Notes will be cancelled by the Trustee.
7. Transferability of Notes.
(a) A Holder has the right to transfer at any time all or part of any Note to any other
Permitted Holder. The Trustee shall inform the Holder about the most recent Notional Value of the Note.
(b) The transferor of a Note shall inform the Trustee promptly of all or part of the Note that
is being transferred, the name of the Permitted Holder who is the transferee, the agreed price for the
transfer, and the value date of the transfer. The transfer shall be recorded by the Trustee. Upon recording,
the transfer shall be effective as of the value date agreed between the transferee and the transferor.
(c) For all transfers under this paragraph 7, the Trustee will cancel the Note that has
been transferred in whole or in part and, if the Note is a registered Note, the transferor shall, as a
condition for the transfer, surrender for cancellation any such registered Note that is not already in
the custody of the Trustee. Upon cancellation of the relevant Note, the Trustee will issue a new Note
in the name of the transferee for any part of the Principal Amount transferred and, where
appropriate, a new Note in the name of the transferor for any part of the Principal Amount retained
by it. The issue date of each new Note will be the issue date of the cancelled Note. The form and
delivery of each new Note will be as specified in paragraph 2 of these Terms and Conditions. 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
64 INTERNATIONAL MONETARY FUND
(d) If less than the full Principal Amount of the Note is transferred, the Principal
Amount of the new Note to be held by the transferor shall be reduced by the transferred amount,
and the Notional Value of that new Note shall be proportionally reduced from the Notional Value of
the original Note that was held by the transferor. The Principal Amount and the Notional Amount of
the new Note to be held by the transferee shall be the transferred amount and the amount equal to
the difference between the Notional Value of the original Note that was held by the transferor and
the Notional Value of the new Note to be held by the transferor, respectively.
(e) If all or part of a Note is transferred, the Trustee will pay interest for the full period
on the amount transferred to the transferee.
(f) Derivative transactions in respect of any Note, and transfers of participation
interests in any Note, are prohibited.
8. Payment upon Maturity. As soon as practicable following the Maturity Date, the Trustee shall
repay the Holder of a Note the Principal Amount or the Notional Value of that Note, whichever is
lower.
9. Media and Modalities of Payments; Cancellation of Note; and Payment after Cancellation.
(a) Unless otherwise agreed between the Holder and the Trustee, the Trustee shall pay
interest and make repayment of the Note in SDRs.
(b) Payment of interest and repayment of the Notes in SDRs shall be made by crediting
the amount due to the Holder’s or Relevant Member’s, as applicable, account in the Special
Drawing Rights Department. If the due date is not a business day of the Fund, the payment
shall be made on the following business day for the Fund.
(c) Payment of interest and repayment of the Notes in currencies shall be made to an
account agreed between the Trustee and the Holder. If the due date is not a business day of the
Fund or the Target system or the domestic markets of the non-euro currencies included in the SDR
basket, the payment shall be made on the following business day for the Fund, the Target system
and the domestic markets of the non-euro currencies included in the SDR basket.
(d) Upon the full repayment of the Principal Amount or the Notional Value of the Note,
whichever is lower, the relevant Note shall be cancelled, and any remaining interest accrued from
the last interest payment date to, but excluding, that repayment date shall be paid to the Holder at
the end of the Fund financial quarter in which the repayment was made.
(e) To the extent any transfers are received in the Deposit Account of the Trust pursuant
to Section VI, paragraph 1(c) of the RST Instrument after the cancellation of a Note under
subparagraph (d) of this paragraph 9, the Trustee shall promptly pay to the last Holder of that Note
any amounts received that are attributable to that Note. 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 65
10. Exchange Rates. Unless otherwise agreed between a Holder and the Trustee, all payments of
interest and repayments in currency under these Terms and Conditions will be made at the
exchange rates for the relevant currencies in terms of the SDR established pursuant to Article XIX,
Section 7(a) of the Fund’s Articles of Agreement and the rules and regulations of the Fund
thereunder for the second (Fund) business day before the value date of the payment. If this
exchange rate determination date is not a business day in the Holder’s principal location, then such
date will be the last preceding business day of the Fund that is also a business day in the Holder’s
principal location.
11. SDR Basket. If the Fund changes the currency composition or currency weights of the SDR
basket, all payments of interest on the Notes and repayments of the Notes two or more (Fund)
business days after the effective date of the change shall be made on the basis of the new SDR
basket.
12. Dispute Resolution. Any question arising under any Note shall be settled by mutual
agreement between the Holder and the Trustee. 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
66 INTERNATIONAL MONETARY FUND
Annex 4
RESLIENCE AND SUSTAINABILITY TRUST
JAPAN DEPOSIT ACCOUNT NOTE
Number 1
Amount: SDR ___________
Issue Date: _____________
Maturity Date: November 30, 2050
This Note is issued in accordance with the Terms and Conditions for the Note issued under
the Deposit Account Note Agreement between the International Monetary Fund (the “Fund”), as
Trustee of the Resilience and Sustainability Trust (the “Trustee”) and the Government of Japan (the
“Contributor”) dated [mm/dd/yy] (the “Terms and Conditions”). Each holder of this Note is deemed
to have agreed to the Terms and Conditions and relevant terms of the Deposit Account Note
Agreement, including without limitation the maturity date, the interest rate, the terms and
conditions for early repayment at the request of the Trustee or the holder hereof, and the terms and
conditions for transfer of this Note or any part thereof.
As soon as practicable following November 30, 2050 (the “Maturity Date”), the Fund, as
Trustee, promises to pay to the Contributor, or registered transferee the lower of: (i) the Principal
Amount of the Note or (ii) the Notional Value of the Note, as these terms are defined in the Terms
and Conditions.
THIS NOTE HAS NOT BEEN REGISTERED UNDER THE SECURITIES LAWS OF ANY JURISDICTION.
IN NO EVENT SHALL ANY HOLDER OF THIS NOTE SELL, ASSIGN, DISPOSE OF OR OTHERWISE
TRANSFER THIS NOTE, DIRECTLY OR INDIRECTLY, TO ANY ENTITY THAT IS NOT (I) A MEMBER
OF THE FUND, (II) THE CENTRAL BANK OR OTHER FISCAL AGENCY DESIGNATED BY A MEMBER
OF THE FUND FOR PURPOSES OF ARTICLE V, SECTION 1 OF THE FUND’S ARTICLES OF
AGREEMENT, OR (III) AN OFFICIAL ENTITY THAT HAS BEEN PRESCRIBED AS A HOLDER OF SPECIAL
DRAWING RIGHTS PURSUANT TO ARTICLE XVII, SECTION 3 OF THE FUND’S ARTICLES OF
AGREEMENT.
ANY DERIVATIVE TRANSACTIONS IN RESPECT OF THIS NOTE, AND TRANSFERS OF PARTICIPATION
INTERESTS IN THIS NOTE, ARE PROHIBITED.
Interest shall accrue daily on the Principal Amount (as this term is defined in the Terms and
Conditions) of this Note at the SDR interest rate established by the Fund pursuant to Article XX, Section 3
of the Fund’s Articles of Agreement. Interest shall be calculated on an actual day basis and shall be paid 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 67
on a quarterly basis promptly after July 31, October 31, January 31, and April 30 of each year. Payments of
interest shall be limited to the Notional Value of the Note, as this term is defined in the Terms and
Conditions, and shall be made in SDRs, or such currency as may be agreed upon between the registered
holder and the Trustee. Payments in SDRs shall be made by crediting the SDR Department account of the
registered holder (if any), or of the member for which the registered holder serves as central bank or other
fiscal agency or other official institution in the case of registered holders that are the central bank or other
fiscal agency or other official institution of a member of the Fund.
[Signatures] 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
68 INTERNATIONAL MONETARY FUND
Attachment VI. Borrowing Agreement between
Banco De España and the International Monetary Fund,
as Trustee of the Resilience and Sustainability Trust
WHEREAS the International Monetary Fund (the “Fund”) has established the Resilience and
Sustainability Trust (the “Trust”) to assist in fulfilling the purposes of the Fund by providing loans
under the Resilience and Sustainability Facility of the Trust (the “RSF”) to eligible Fund members to
enhance their economic resilience and sustainability;
WHEREAS, the Kingdom of Spain (“Spain”) has committed to contribute financial resources to the
loan account (“Loan Account”), the deposit account (the “Deposit Account”) and the reserve account
(the “Reserve Account”) of the Trust in accordance with the provisions of the Instrument to Establish
the Resilience and Sustainability Trust (the “RST Instrument”); and
WHEREAS, to fulfill Spain’s commitment, Banco de España (the “Contributor”) and the Fund, as
Trustee (the “Trustee”) of the Trust, have reached understandings on a borrowing agreement (the
“Borrowing Agreement”) and a deposit agreement (the “Deposit Agreement”) to contribute to the
Loan Account and the Deposit Account respectively, and the Ministry for Economy and Digitalization
and the Trustee have also reached understandings on a reserve contribution agreement (the
“Reserve Contribution Agreement”) to contribute to the Reserve Account;
NOW, THEREFORE, the Contributor and the Trustee agree on the terms and conditions of this
Borrowing Agreement as follows:
1. Amount. The Contributor agrees to lend an SDR-denominated amount up to the equivalent
of Euros 1,525,000,000 to the Loan Account of the Trust. This SDR-denominated amount will be
determined at the exchange rate for the euro in terms of the SDR established by the Fund pursuant
to Article XIX, Section 7(a) of the Fund’s Articles of Agreement and the rules and regulations of the
Fund thereunder as of the second (Fund) business day before the value date of the transfer by the
Contributor of its contribution to the Reserve Account under the Reserve Contribution Agreement.
2. Drawings.
(a) The Trustee may make drawings under this Borrowing Agreement at any time during the
period from the effective date of this Borrowing Agreement, as set forth in paragraph 13(b) below,
through November 30, 2030 (the “drawdown period”) to finance loan disbursements under the RSF.
(b) The Trustee may make drawings under this Borrowing Agreement to finance early
repayment of outstanding Trust borrowing under other borrowing agreements with the Loan
Account pursuant to Section IV, Paragraph 3(b) of the RST Instrument for as long as claims under
such borrowing agreements remain outstanding. 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 69
(c) The Trustee’s drawings under this Borrowing Agreement shall be suspended following the
Contributor’s request for early repayment under paragraph 6 of this Borrowing Agreement or when
Spain is no longer included for transfers in the Fund’s Financial Transactions Plan, in accordance with
Section IV, paragraph 4 of the RST Instrument.
(d) If Spain does not provide its consent to a proposed amendment to the RST Instrument
pursuant to Section X, paragraph 2(a) of the RST Instrument, further drawings under this Borrowing
Agreement to finance any new commitment of loans under the RSF that is made after the effectiveness
of the proposed amendment shall be suspended.
(e) If any installment of principal or interest is not paid to the Contributor within a period of ten
Fund business days after its due date, the Trustee shall not make further drawings under this
Borrowing Agreement pending consultations with the Contributor on the matter. However, the
Trustee may resume drawings under this Borrowing Agreement once all arrears to the Contributor
have been discharged.
(f) Unless otherwise agreed between the Contributor and the Trustee, the Trustee shall give the
Contributor at least five (Madrid) business days’ notice of its intention to draw and shall provide
payment instructions at least two (Fund) business days prior to the value date of the transaction by
SWIFT.
3. Denomination, Media, and Claim Certificate.
(a) The amount of each drawing shall be denominated in SDRs. The amount shall be paid by the
Contributor on the value date specified in the Trustee’s notice by transfer of SDRs to an account
specified by the Trustee.
(b) Upon request, the Trustee shall issue to the Contributor a non-negotiable certificate
evidencing its claim on the Trust resulting from a drawing outstanding under this Borrowing
Agreement.
4. Interest.
(a) Each drawing shall bear interest at the SDR interest rate established by the Fund pursuant to
Article XX, Section 3 of the Fund’s Articles of Agreement.
(b) The amount of interest payable in respect of each drawing shall be calculated on an actual
day basis and shall be paid on all outstanding drawings under this Borrowing Agreement on a
quarterly basis promptly after January 31, April 30, July 31, and October 31 of each year.
5. Maturity and Repayment. Except as otherwise provided in this Borrowing Agreement, each
drawing shall be repaid in accordance with the repayment schedule for the loan disbursement under
the RSF for which it was drawn, and at the time of each drawing, the Trustee shall provide
information to the Contributor on the specific repayment schedule for the amount drawn.
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
70 INTERNATIONAL MONETARY FUND
6. Encashment. The Contributor shall have the right to seek early repayment of all or part of
the principal amount of its outstanding claims on the Loan Account of the Trust pursuant to Section
IV, paragraph 3(b) of the RST Instrument if the Contributor represents that the balance of payments
and reserve position of Spain justifies the early repayment, and the Trustee, having given this
representation the overwhelming benefit of any doubt, agrees. Early repayment is in all cases subject
to the availability of resources under borrowing agreements to the Loan Account of the Trust.
7. Early Repayment. The Trustee may repay any drawing or part thereof at any time in
advance of maturity upon its receipt of early repayment of the corresponding loan disbursement
under the RSF, provided that the Trustee will give notice to the Contributor at least five (5) (Fund)
business days prior to such advance repayment.
8. Modality of Principal Repayment and Interest Payments.
(a) Unless otherwise agreed between the Trustee and the Contributor, payments of principal and
interest by the Trustee shall be made in SDRs.
(b) Payments in SDRs by the Trustee shall be made by crediting Spain’s holdings account in the
Special Drawing Rights Department. If the due date is not a business day of the Fund, the payment
shall be made on the following business day of the Fund.
(c) Payments by the Trustee in currencies shall be made to an account as agreed between the
Trustee and the Contributor. If the due date is not a business day of the Fund or the Target system or
the domestic markets of the non-euro currencies included in the SDR basket, the payment shall be
made on the following business day for the Fund, the Target system and the domestic markets of the
non-euro currencies included in the SDR basket.
9. Transfer of Claims.
(a) The Contributor shall have the right to transfer at any time all or part of any claim to any
member of the Fund that is a participant in the Fund’s SDR Department, to the central bank or other
fiscal agency designated by such a member for purposes of Article V, Section 1 of the Fund’s Articles
of Agreement or to any official entity that has been prescribed as a holder of SDRs pursuant to Article
XVII, Section 3 of the Fund’s Articles of Agreement.
(b) The transferee shall acquire all the rights of the Contributor under this Borrowing Agreement
with respect to repayment of principal and interest on the transferred claim, except that the transferee
shall acquire the right to request early repayment under paragraph 6 of this Borrowing Agreement
only if it is a member or the central bank or other fiscal agency of a member and, at the time of
transfer, the member’s balance of payments and reserve position was considered sufficiently strong
in the opinion of the Fund for its currency to be usable in transfers under the Fund’s Financial
Transactions Plan.
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 71
10. Exchange Rates. Unless otherwise agreed between the Trustee and the Contributor, all
transfers, exchanges, and payments of principal and interest in currencies shall be made at the
exchange rates for the relevant currencies in terms of the SDR established by the Fund pursuant to
Article XIX, Section 7(a) of the Fund’s Articles of Agreement and the rules and regulations of the
Fund thereunder for the second (Fund) business day before the value date of the transfer, exchange
or payment.
11. SDR Basket. If the Fund changes the currency composition or currency weights of the
SDR basket, all transfers, exchanges, and payments of principal and interest made two or more
(Fund) business days after the effective date of the change shall be made on the basis of the new
SDR basket.
12. Dispute Resolution. Any question arising under this Borrowing Agreement shall be settled
by mutual agreement between the Contributor and the Trustee.
13. Execution and Effectiveness.
(a) This Borrowing Agreement may be executed in one or more duplicate counterparts, each of
which shall be deemed an original, and which together shall constitute one and the same instrument.
(b) This Borrowing Agreement shall become effective on the later of: (i) the last date written
below, or (ii) the date the Managing Director of the Fund has notified the Executive Board of the Fund
that the RST has become operational.
[Signature page follows]
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
72 INTERNATIONAL MONETARY FUND
IN WITNESS WHEREOF, the Banco de España and the International Monetary Fund, as Trustee of the
Resilience and Sustainability Trust, have executed this agreement.
For the Banco de España:
September 5, 2022
______________________________ ______________________________
Pablo Hernandez de Cos /s Date
Governor
For the INTERNATIONAL MONETARY FUND as Trustee:
September 8, 2022
______________________________ ______________________________
Kristalina Georgieva /s Date
Managing Director
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 73
Attachment VII. Deposit Agreement between
Banco De España and the International Monetary Fund,
as Trustee of the Resilience and Sustainability Trust
WHEREAS the International Monetary Fund (the “Fund”) has established the Resilience and
Sustainability Trust (the “Trust”) to assist in fulfilling the purposes of the Fund by providing loans
under the Resilience and Sustainability Facility of the Trust (the “RSF”) to eligible Fund members to
enhance their economic resilience and sustainability;
WHEREAS, the Kingdom of Spain (“Spain”) has committed to contribute financial resources to the
loan account (“Loan Account”), the deposit account (the “Deposit Account”) and the reserve account
(the “Reserve Account”) of the Trust in accordance with the provisions of the Instrument to Establish
the Resilience and Sustainability Trust (the “RST Instrument”); and
WHEREAS, to fulfill Spain’s commitment, Banco de España (the “Contributor”) and the Fund, as
Trustee (the “Trustee”) of the Trust, have reached understandings on a deposit agreement (the
“Deposit Agreement”) and a borrowing agreement (the “Borrowing Agreement”) to contribute to the
Deposit Account, and to the Loan Account respectively, and the Ministry for the Economy and
Digitalization and the Trustee have also reached understandings on a reserve contribution
agreement (the “Reserve Contribution Agreement”) to contribute to the Reserve Account;
NOW, THEREFORE, the Contributor and the Trustee agree on the terms and conditions of this
Deposit Agreement as follows:
1. Amount of the Deposit. The Contributor shall make a deposit (the “Deposit”) of an SDR
denominated amount equivalent to Euro 305,000,000 with the Deposit Account of the Trust (the
“Original Amount of the Deposit”). This SDR-denominated amount will be determined at the
exchange rate for the euro in terms of the SDR established by the Fund pursuant to Article XIX,
Section 7(a) of the Fund’s Articles of Agreement and the rules and regulations of the Fund
thereunder as of the second (Fund) business day before the value date of the transfer by the
Contributor of its contribution to the Reserve Account under the Reserve Contribution Agreement.
2. Denomination and Media.
(a) The Deposit shall be denominated in SDRs.
(b) The Deposit shall be made in SDRs.
(c) The Deposit shall be made promptly on or after the effectiveness of this Deposit Agreement.
3. Maturity. The Deposit shall mature on November 30, 2050 (the “Maturity Date”).
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
74 INTERNATIONAL MONETARY FUND
4. Investment Strategy and Risk Disclosure.
(a) The resources from the Deposit shall be invested by the Trustee in accordance with the
guidelines adopted by the Trustee pursuant to Section VI, paragraph 4(a) of the RST Instrument.
(b) The assets of the Deposit Account are exposed to market and credit risk which could result
in potential losses to the Original Amount of the Deposit.
5. Notional Value of the Deposit. For the purposes set out in paragraphs 6, 7, 8, and 9 of this
Deposit Agreement, the Trustee shall calculate the notional value of the Deposit based on the
Original Amount of the Deposit and subsequently any attributed investment earnings or losses,
interest payments made to the Contributor pursuant to paragraph 6 of this Deposit Agreement, if
any, and transfers pursuant to paragraphs 8, 9, 10 or 11 of this Deposit Agreement or pursuant to
Section VI, paragraphs 1(c) and 4(b)(2) of the RST Instrument (“Notional Value”).
6. Interest.
(a) The Trustee shall pay the Contributor interest on the Original Amount of the Deposit less
any amounts repaid to the Contributor under paragraphs 8 or 9 or transferred by the Contributor
under paragraph 11 of this Deposit Agreement, at the SDR interest rate established by the Fund
pursuant to Article XX, Section 3 of the Fund’s Articles of Agreement. Interest shall accrue daily and
shall be paid promptly by the Trustee on a quarterly basis after January 31, April 30, July 31 and
October 31 of each year. Any payment of interest shall be limited to the Notional Value of the Deposit.
(b) Following any full repayment of the Deposit in accordance with paragraphs 7 and 9 of this
Deposit Agreement, all remaining interest under paragraph 6(a) above, shall be paid to the
Contributor at the end of the Fund financial quarter in which repayment was made.
7. Repayment upon Maturity. As soon as practicable following the Maturity Date, the Trustee
shall repay the Deposit to the Contributor. The repayment amount shall be the lower of: (a) the
Original Amount of the Deposit less any amount(s) repaid to the Contributor under paragraphs 8 or
9 or transferred by the Contributor under paragraph 11 of this Deposit Agreement, and (b) the
Notional Value of the Deposit. To the extent any transfers are received in the Deposit Account
pursuant to Section VI, paragraph 1(c) of the RST Instrument after the Maturity Date, the Trustee
shall promptly repay to the Contributor any amounts received that are attributable to the Deposit.
8. Encashment. The Contributor shall have the right to seek early repayment of all or part of
the Deposit (the “Encashed Deposit”), pursuant to Section VI, paragraph 5(e) of the RST Instrument,
if the Contributor represents that the balance of payments and reserve position of Spain justifies the
early repayment in advance of the Maturity Date and the Trustee, having given the representation
the overwhelming benefit of any doubt, agrees. The Encashed Deposit amount may not exceed the
Original Amount of the Deposit less any amount(s) repaid to the Contributor under this paragraph
or paragraph 9 or transferred by the Contributor under paragraph 11 of this Deposit Agreement, or
the Notional Value of the Deposit, whichever is lower. The Contributor agrees to restore an amount
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 75
equivalent to the Encashed Deposit to the Deposit Account as soon as practicable in light of
favorable developments in Spain’s balance of payments and reserve position, as evidenced by the
inclusion of Spain’s currency in the Fund’s Financial Transactions Plan.
9. Early Repayment.
(a) The Trustee may repay all or part of the Deposit at any time in advance of the Maturity
Date; provided that the repayment amount may not exceed the Original Amount of the Deposit less
any amount(s) repaid to the Contributor under this paragraph or paragraph 8 or transferred by the
Contributor under paragraph 11 of this Deposit Agreement, or the Notional Value of the Deposit,
whichever is lower; and provided further that the Trustee will give notice to the Contributor at least
five (5) Fund business days prior to any advance repayment.
(b) If Spain does not provide its consent to a proposed amendment to the RST Instrument as
required under Section X, paragraph 2(a) of the RST Instrument, the Contributor may request the
return of a portion of the Original Amount of the Deposit less any amount(s) repaid to the
Contributor under this paragraph and paragraph 8 or transferred by the Contributor under
paragraph 11 of this Deposit Agreement, or the Notional Value of the Deposit, whichever is lower,
with that portion based on the ratio of remaining uncommitted loan resources to total loan
commitments to the Trust under the Borrowing Agreement in accordance with Section X, paragraph
2(b) of the RST Instrument.
10. Liquidation. If the Trustee decides to wind up the operations of the Trust, the Trust shall
repay the Deposit to the Contributor, at the Original Amount of the Deposit less any amount(s)
repaid to the Contributor under paragraphs 8 or 9 or transferred by the Contributor under
paragraph 11 of this Deposit Agreement, or the Notional Value of the Deposit, whichever is lower,
and in accordance with liquidation procedures under the RST Instrument, provided that the Trustee
shall notify the Contributor in advance of the repayment. Payment will be made to an account
specified by the Contributor, which can include an account or trust administered by the Fund.
11. Transfer of Claims.
(a) The Contributor shall have the right to transfer at any time all or part of any claim on the
Deposit Account to any member of the Fund that is a participant in the Fund’s SDR Department, to
the central bank or other fiscal agency designated by such a member for purposes of Article V, Section
1 of the Fund’s Articles of Agreement, or to any official entity that has been prescribed as a holder of
SDRs pursuant to Article XVII, Section 3 of the Fund’s Articles of Agreement.
(b) The transferee shall acquire all the rights of the Contributor under this Deposit Agreement
with respect to repayment of and interest on the transferred claim, except that the transferee shall
acquire the right to request early repayment under paragraph 8 of this Deposit Agreement only if it
is a member or the central bank or other fiscal agency of a member and, at the time of transfer, the
member’s balance of payments and reserve position was considered sufficiently strong in the opinion
of the Fund for its currency to be usable in transfers under the Fund’s Financial Transactions Plan.
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
76 INTERNATIONAL MONETARY FUND
12. Modality of Interest Payment and Repayment of the Deposit.
(a) Unless otherwise agreed between the Contributor and the Trustee, payment of interest and
repayments of the Deposit shall be made in SDRs.
(b) Payment of interest and repayment of the Deposit in SDRs shall be made by crediting the
amount due to Spain’s holdings account in the Special Drawing Rights Department. If the due date is
not a business day of the Fund, the payment shall be made on the following business day for the
Fund.
(c) Payment of interest and repayment of the Deposit by the Trustee in currencies shall be made
to an account as agreed between the Trustee and the Contributor. If the due date is not a business
day of the Fund or the Target system or the domestic markets of the non-euro currencies included in
the SDR basket, the payment shall be made on the following business day for the Fund, the Target
system and the domestic markets of the non-euro currencies included in the SDR basket.
13. Exchange Rate. Unless otherwise agreed between the Contributor and the Trustee, all
transfers, exchanges, payments of interest, or repayments of the Deposit under this Deposit
Agreement in currencies shall be made at the exchange rates for the relevant currencies in terms of
the SDR established by the Fund pursuant to Article XIX, Section 7(a) of the Fund’s Articles of
Agreement and the rules and regulations of the Fund thereunder for the second (Fund) business day
before the value date of the transfer, exchange, payment of interest, or repayment.
14. SDR Basket. If the Fund changes the currency composition or currency weights of the SDR
basket, all transfers, exchanges, payments of interest, and repayments of the Deposit two or more
(Fund) business days after the effective date of the change shall be made on the basis of the new
SDR basket.
15. Dispute Resolution. Any question arising under this Deposit Agreement shall be settled by
mutual agreement between the Contributor and the Trustee.
16. Execution and Effectiveness.
(a) This Deposit Agreement may be executed in one or more duplicate counterparts, each of
which shall be deemed an original, and which together shall constitute one and the same
instrument.
(b) This Deposit Agreement shall become effective on the last date written below.
[Signature page follows] 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 77
IN WITNESS WHEREOF, the Banco de España and the International Monetary Fund, as Trustee of the
Resilience and Sustainability Trust, have executed this agreement.
For the Banco de España:
September 5, 2022
______________________________ ______________________________
Pablo Hernandez de Cos /s Date
Governor
For the INTERNATIONAL MONETARY FUND as Trustee:
September 8, 2022
______________________________ ______________________________
Kristalina Georgieva /s Date
Managing Director
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
78 INTERNATIONAL MONETARY FUND
Attachment VIII. Contribution Agreement between the
Ministry for the Economy and Digitalization, and the
International Monetary Fund, as Trustee of the Resilience and
Sustainability Trust
WHEREAS the International Monetary Fund (the “Fund”) has established the Resilience and
Sustainability Trust (the “Trust”) to assist in fulfilling the purposes of the Fund by providing loans
under the Resilience and Sustainability Facility of the Trust (the “RSF”) to eligible Fund members to
enhance their economic resilience and sustainability;
WHEREAS, The Kingdom of Spain (“Spain”) has committed to contribute financial resources to the
loan account (“Loan Account”), the deposit account (the “Deposit Account”) and the reserve account
(“Reserve Account”) of the Trust in accordance with the provisions of the Instrument to Establish the
Resilience and Sustainability Trust (the “RST Instrument”); and
WHEREAS to fulfill Spain’s commitment, the Ministry for the Economy and Digitalization (the
“Contributor”) and the Fund, as Trustee (the “Trustee”) of the Trust, have reached understandings on
a reserve contribution agreement (the “Reserve Contribution Agreement”) to contribute to the
Reserve Account, and Banco de España and the Trustee have also reached understandings on a
borrowing agreement (the “Borrowing Agreement”) and a deposit agreement to contribute to the
Loan Account and the Deposit Account, respectively;
NOW, THEREFORE, the Contributor and the Trustee agree on the terms and conditions of this
Reserve Contribution Agreement as follows:
1. Amount. The Contributor shall provide a contribution of an SDR-denominated amount
equivalent to Euro 30,500,005 (the “Contribution”), with the SDR-denominated amount to be
determined pursuant to paragraph 9 of this Reserve Contribution Agreement as of the second
(Fund) business day before the value date of the transfer of the Contribution to the Reserve Account
of the Trust.
2. Denomination and Media.
(a) The Contribution shall be denominated in SDRs.
(b) The Contribution shall be made in euros.
(c) The Contributor shall transfer the Contribution to an account specified by the Trustee on or
promptly after the effectiveness of this Reserve Contribution Agreement.
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 79
3. Investment Strategy and Risk Disclosure.
(a) The resources in the Reserve Account shall be invested by the Trustee in accordance with the
guidelines adopted by the Trustee pursuant to Section V, paragraph 3(a) of the RST Instrument.
(b) The assets of the Reserve Account are exposed to the market and credit risk which could
result in potential losses to the Contributor’s share in the resources of the Reserve Account.
4. Remuneration. No remuneration shall be paid to the Contributor in respect of the
Contribution.
5. Contributor’s Share in the Reserve Account. The Contributor shall have a proportional
share in the Reserve Account, based on its Contribution, any resources that are transferred from the
Deposit Account and attributed to Banco de España’s contribution to the Deposit Account pursuant
to Section VI, Paragraph 5(d) of the RST Instrument, and attributed investment earnings or losses in
the Reserve Account.
6. Transfer of the Contributor’s Share. The Contributor shall have the right to transfer at any
time all or part of its share in the Reserve Account to any member of the Fund that is a participant in
the Fund’s SDR Department, to any member’s central bank or other fiscal agency designated by
such a member for purposes of Article V, Section 1 of the Fund’s Articles of Agreement, or to any
official entity that has been prescribed as a holder of SDRs pursuant to Article XVII, Section 3 of the
Fund’s Articles of Agreement. The transferee shall acquire all the rights of the Contributor under this
Reserve Contribution Agreement with respect to the transferred amount.
7. Distribution of Resources in the Reserve Account.
(a) The Contribution shall have no fixed maturity.
(b) Upon the liquidation of the Trust, the Trustee shall distribute to the Contributor its share in
the resources of the Reserve Account of the Trust in accordance with Section V, paragraph 4(c) of
the RST Instrument.
(c) Prior to the liquidation of the Trust and following the Trustee’s decision to distribute a
portion of the balances in the Reserve Account of the RST in accordance with Section V, paragraph
4(d) of the RST Instrument, the Contributor shall receive part of this distribution in proportion to its
share in the Reserve Account.
(d) If Spain does not provide its consent to a proposed amendment to the RST Instrument as
required under Section X, paragraph 2(a) of the RST Instrument, the Contributor may request the
return of a portion of its share in the Reserve Account based on the ratio of remaining uncommitted
loan resources to total loan commitments to the Trust under the Borrowing Agreement in
accordance with Section X, paragraph 2(b) of the RST Instrument. 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
80 INTERNATIONAL MONETARY FUND
8. Modality of Distribution.
(a) Unless otherwise agreed between the Trustee and the Contributor, any distribution to the
Contributor under this Reserve Contribution Agreement shall be made in SDRs.
(b) Distribution in SDRs shall be made by crediting Spain’s holdings account in the Special
Drawing Rights Department on a business day of the Fund.
(c) Distribution in currencies shall be made to an account as agreed between the Trustee and
the Contributor and on the business day of the Fund, the Target system and the domestic markets
of the non-euro currencies included in the SDR basket.
9. Exchange Rates. Unless otherwise agreed between the Trustee and the Contributor, all
transfers, exchanges, and distribution in currencies shall be made at the exchange rates for the
relevant currencies in terms of the SDR established by the Fund pursuant to Article XIX, Section 7(a)
of the Fund’s Articles of Agreement and the rules and regulations of the Fund thereunder for the
second (Fund) business day before the value date of the transfer, exchange, or payment.
10. SDR Basket. If the Fund changes the currency composition or currency weights of the SDR
basket, all transfers, exchanges, and distribution made two or more (Fund) business days after the
effective date of the change shall be made on the basis of the new SDR basket.
11. Dispute Resolution. Any question arising under this Reserve Contribution Agreement shall
be settled by mutual agreement between the Contributor and the Trustee.
12. Effectiveness.
(a) This Reserve Contribution Agreement may be executed in one or more duplicate
counterparts, each of which shall be deemed an original, and which together shall constitute one
and the same instrument.
(b) This Reserve Contribution Agreement shall become effective on the last date written below.
[Signature page follows] 
CONCESSIONAL FINANCING: RST BORROWING AGREEMENTS
INTERNATIONAL MONETARY FUND 81
IN WITNESS WHEREOF, the Ministry for the Economy and Digitalization of Spain and the
International Monetary Fund, as Trustee of the Resilience and Sustainability Trust, have executed this
agreement.
For the Ministry for the Economy and Digitalization of Spain:
August 24, 2022
______________________________ ______________________________
Carlos Cuerpo Caballero /s Date
Secretary General for the Treasury and International Finance
For the INTERNATIONAL MONETARY FUND as Trustee:
September 8, 2022
______________________________ ______________________________
Kristalina Georgieva /s Date
Managing Director'''

In [37]:
vswV5GcFju9btwKa_text = '''Canada - African Development Bank Climate Fund
cacf_banner-latest.jpg
The Canada – African Development Bank Climate Fund (CACF) is a Special Fund established in 2021 as a joint effort between the Government of Canada and the African Development Bank to fight climate change in the African continent.

The objective of CACF is to extend long-term concessional loans to climate mitigation and adaptation projects that embed strong gender equality principles throughout the various phases of the project cycle.

Structure



CACF is composed of an Investment Facility aimed at extending concessional loans to eligible climate change projects, and a Technical Assistance Facility seeking to address the increased complexity, coordination requirements and reporting obligations related to climate change projects and to mainstream gender and human rights in CACF projects.

CACF loans will be deployed in co-financing along with the African Development Bank’s own resources to help public and private sector projects offset costs hurdles and high-risk perception.

Sectors and Areas of Focus

The focus of CACF interventions is put on projects that can lead to significant reduction or avoidance of greenhouse gas emissions, and increased climate resilience for populations and businesses in Africa.

In alignment to Canada’s Feminist International Assistance Policy, and with the African Development Bank Group Gender Strategy 2021–2025, gender equality, and particularly the empowerment of women and girls, is at the core of CACF interventions.



Financing Terms

Long-term concessional loans are the only instrument available under the Investment Facility, to be deployed over a commitment period of 5 years from 2021 onwards.



Implementation Status as of End 2024

To date, approximately USD 71 million out of a total of USD 99 million have been approved under the Investment Facility and the Technical Assistance Facility of CACF. This represents 72% of the total funds available for commitment.

Under the CACF Investment Facility, around USD 70 million have been approved in co-financing with close to USD 453 million approved by the African Development Bank.

Distribution of CACF Investment Facility approved projects:





“In building back Africa, climate resilience is very important and these resources are very unique in helping us with adaptation. First, it is long-term financing. It will provide long-term capital to the private sector and the public sector. It also provides it at affordable levels for countries. What I like most about it is that it looks at the multi-sectoral use of this financing. All these things are very important to support Africa in climate adaptation and mitigation.”

Dr. Akinwumi A. Adesina, President of the African Development Bank



“This contribution, with its strong gender footprint, recognizes the critical role that women need to play in climate action, and supports their efforts to mitigate and adapt to the effects of climate change.”

Hon. Karina Gould, Canada’s Minister of International Development'''

In [38]:
Ui54HFehYuBl7ur_text_1 = '''Canadian Climate Fund for the Private Sector in Asia II
What is the fund?
Established in March 2017, the Canadian Climate Fund for the Private Sector in Asia II is a CAD200 million fund designed to support greater private sector participation in climate change mitigation and adaptation in low and lower middle income countries and upper middle-income small island developing states in Asia and the Pacific. The fund will also seek to promote gender equality and the empowerment of women and girls in projects supported by the fund.

The fund will aim to play a key role in helping the private sector overcome development risks (including market risks, technology risks, financing risks, regulatory and other risks) by offering financing on concessional terms and conditions to projects that would not proceed solely on a commercial basis. Concessional financing will be used to help offset cost hurdles and initiate, scale-up and replicate high impact projects in markets that show strong potential for growth. ADB will be judicious in its use of concessional financing resources so as to not distort local markets, and will be guided by a set of principles on the use of concessional finance.

The fund has been designed to provide concessional financing in the form of senior loans, subordinated loans and risk participations. Fund resources will not be invested as equity or any instruments convertible into equity. Similar to the first fund, this fund has provision for unhedged local currency financing in some markets subject to an overall ceiling.

The fund was established as a trust fund and will provide co-financing alongside ADB’s ordinary capital resources for non-sovereign projects. Fund resources will not be used to finance stand-alone transactions, meaning ADB participation will be necessary in all transactions. ADB is responsible for the administration and management of the fund, including the selection and approval of projects. ADB, acting as trustee of the fund will be responsible for the origination, processing, approval, and monitoring of transactions in accordance with the fund arrangement. ADB will be the lender of record, or guarantor of record, as applicable, for transactions supported by the fund.

What are the priorities?
The fund will invest in a combination of private sector led climate mitigation and adaptation activities including, but not limited to:

Clean and renewable energy generation;
Energy efficiency;
Transmission and distribution systems for renewable energy projects;
Process emissions;
Sustainable transport;
Waste management and wastewater projects;
Water supply and management;
Agriculture and forestry;
Land use management;
Natural resource management;
Resilient infrastructure;
Coastal protection; and
Disaster risk management.
Projects supported by the fund will contribute to increased gender-responsive economic development in eligible countries, and will integrate gender equality outputs.

To the extent applicable, projects supported by the fund will maximize development co-benefits. These may include efforts to:

Invigorate local economies;
Increase the number of incremental jobs in communities;
Benefit consumers;
Improve health benefits through reduced pollution; and
Improve countries’ competitiveness and sustainable development initiatives, including economic diversification.
Who is eligible to receive the fund?
The fund will support projects in Afghanistan, Armenia, Bangladesh, Bhutan, Cambodia, Fiji, India, Indonesia, Kiribati, Kyrgyz Republic, Lao People's Democratic Republic, Maldives, Marshall Islands, Federated States of Micronesia, Mongolia, Myanmar, Nepal, Pakistan, Palau, Papua New Guinea, Philippines, Samoa, Solomon Islands, Sri Lanka, Tajikistan, Timor-Leste, Tonga, Tuvalu, Uzbekistan, Vanuatu, and Viet Nam.

Who supports the fund?
The Government of Canada supports the fund.'''

In [40]:
AHfTSNelYuhb8eq_text_2 = '''Resilience and Sustainability Trust Frequently Asked Questions
Q1. What is the broader context of the RST?
Q2. Why is the RST needed?
Q3. What are the key features of IMF financing under the RST?
Q4. How is the purpose of the RST linked to the IMF’s mandate?
Q5. How is the RST financed?
Q6. Is the RST operational yet?
Q1. What is the broader context of the RST?
The IMF’s mandate is to promote members’ balance of payments stability, as well as foster global financial stability and economic growth. Today, low- and vulnerable middle-income countries confront the challenges of inflation, rising debt levels, and geopolitical uncertainty, as they also face longer-term, structural risks to macroeconomic and balance of payments stability, such as climate change and pandemic preparedness.
The IMF’s Resilience and Sustainability Trust (RST) helps low-income and vulnerable middle-income countries build resilience to external shocks and ensure sustainable growth, contributing to their longer-term balance of payments stability. It complements the IMF’s existing lending toolkit by providing longer-term affordable financing to address longer-term structural challenges, including climate change and pandemic preparedness.
Q2. Why is the RST needed?
Prior to the approval of the RST, the IMF’s lending instruments supported members facing short- and medium-term challenges through two key financing sources: the General Resources Account (GRA) which is accessible to all countries; and the Poverty Reduction and Growth Trust (PRGT) available to low-income countries on concessional terms (currently zero interest rates). The RST complements other lending instruments by providing affordable, longer-term financing to contribute to prospective balance of payments stability to enhance economic resilience and sustainability.
The RST is amplifying the effect of the US$650 billion general SDR allocation implemented in 2021 by channeling resources from economically stronger members to countries where needs are greatest.
This Trust is allowing the Fund to broaden its lending support within its mandate, while at the same time limiting risks to members’ financial positions at the Fund by permitting a subset of the membership—those with strong external positions—to pool financial resources that are separate from the Fund’s general resources for the benefit of another subset of the membership—low-income countries and vulnerable middle-income countries.
Q3. What are the key features of IMF financing under the RST?
The RST provides policy support and affordable longer maturity financing – with a 20-year maturity and a 10½ -year grace period – to help build resilience against long-term risks to balance of payments stability. About three-quarters of the IMF’s country membership is eligible for RST financing, including low-income members as well as most middle-income countries and all small developing states.
RST resources finance IMF-supported programs under the Resilience and Sustainability Facility (RSF); this Factsheet outlines the key features of this facility.
Q4. How is the purpose of the RST linked to the IMF’s mandate?
The focus of the RST on prospective balance of payments stability is well aligned with the Fund’s mandate and existing guidelines on the remit of Fund engagement with the membership (outlined in the 2012 Integrated Surveillance Decision). The RST aims to help low-income and vulnerable middle-income countries address longer-term structural challenges that pose macroeconomic risks, including climate change and pandemic preparedness.
An important role of the Fund is to help members address balance of payments issues through the provision of financial resources. Shoring up prospective balance of payments stability has become even more important as countries navigate multiple, persistent shocks that demand greater resilience and sustainability.
Q5. How is the RST financed?
The RST is a loan-based trust administered by the IMF, with a governance and financial structure broadly similar to that of the long-standing Poverty Reduction and Growth Trust (PRGT). RST resources are be mobilized based on voluntary contributions from IMF members with strong external positions, including those wishing to channel SDRs for the benefit of low-income and more vulnerable middle-income members.
The financial architecture is designed to ensure the safety and liquidity of contributors’ loan and deposit claims on the RST, which will retain their reserve asset character, while minimizing the need for budgetary contributions.
Information on the latest RST funding status can be found here.
 

Back to Top
Q6. Is the RST operational yet?
Yes, the RST is open for business since October 2022.
Our initial country cases focused on climate change and rely on broad principles for collaboration developed with the World Bank for climate change focused RSF arrangements. Similar principles for collaboration are under development for RSF arrangements focused on pandemic preparedness, with the World Bank and World Health Organization.'''

In [41]:
Ui54HFehYuBl7ur_text_2 = '''Canadian Climate Fund for the Private Sector in Asia
What is the fund?
The Canadian Climate Fund for the Private Sector in Asia was established in March 2013. The fund aims to catalyze greater private investment in climate change mitigation and adaptation in Asia and the Pacific. The fund will aim to play a key role in helping to overcome leading edge technology risks and cost hurdles in order to initiate and scale-up projects to reduce greenhouse gas emissions and increase climate resilience. The fund is ADB’s first concessional debt cofinancing facility specifically oriented to support private sector operations to combat climate change.

ADB will manage the fund resources under its Clean Energy Financing Partnership Facility umbrella.

What are the priorities and what kind of activities will be financed from the fund?
The fund will provide fund resources—Concessional financing, co-financed with ADB resources, to select private sector projects that do not benefit from a sovereign guarantee and require some financing on concessional terms to be viable or acceptable from a risk-reward perspective. These projects may include but are not limited to: clean energy (including renewable energy, energy efficiency and cleaner fuel) in energy sector and non-energy sector including agriculture, urban infrastructure and sustainable transportation projects.
The fund will also provide fund resources—Grants to selected private sector projects, or public sector projects that improve the enabling environment for private sector investments, to facilitate deployment of new technologies and support activities that enable policy and institutional reforms, as well as regulatory frameworks to encourage climate change mitigation and adaptation.
Who is eligible to receive the concessional financing or grant?
One or more of the following eligible ADB developing members can avail of the fund: Afghanistan, Armenia, Bangladesh, Bhutan, Cambodia, Cook Islands, Fiji, Georgia, India, Indonesia, Kiribati, Lao People's Democratic Republic, Maldives, Marshall Islands, Federated States of Micronesia, Mongolia, Myanmar, Nauru, Nepal, Pakistan, Palau, Papua New Guinea, Philippines, Samoa, Solomon Islands, Sri Lanka, Tajikistan, Timor-Leste, Tonga, Tuvalu, Uzbekistan, Vanuatu, and Viet Nam.

Who supports the fund?
The Government of Canada supports the fund.'''

In [53]:
AHfTSNelYuhb8eq_text = AHfTSNelYuhb8eq_text_1 + AHfTSNelYuhb8eq_text_2
Ui54HFehYuBl7ur_text = Ui54HFehYuBl7ur_text_1 + Ui54HFehYuBl7ur_text_2

In [54]:
def heal_row(results_df, unique_id, text):
    """
    Heal a row in the DataFrame by updating status, content, and char_count
    for the given unique_id with the provided text.

    Args:
        results_df: pandas DataFrame to update
        unique_id: string, the matched_unique_id for the row to heal
        text: string, the new content to set

    Returns:
        None. Modifies results_df in-place.
    """
    results_df.loc[
        results_df["matched_unique_id"] == unique_id,
        ["status", "content", "char_count"],
    ] = ["healed", text, len(text)]

heal_row(results_df, "JAYE9ES5gamFNfMr", JAYE9ES5gamFNfMr_text)
heal_row(results_df, "axxzOXaUnYi9W4yB", axxzOXaUnYi9W4yB_text)
heal_row(results_df, "3AHfTSNelYuhb8eq", AHfTSNelYuhb8eq_text)
heal_row(results_df, "vswV5GcFju9btwKa", vswV5GcFju9btwKa_text)
heal_row(results_df, "8Ui54HFehYuBl7ur", Ui54HFehYuBl7ur_text)


In [44]:
error_df = results_df[results_df["status"] == "error"]

In [45]:
error_df

,matched_unique_id,url,status,content_type,content,char_count,error


While the errors are gone, these are also not saving properly: 

Document 2XUgWVRI3YcBdecj not represented in vector store (name: agricultural clean technology program - research and innovation stream)

Document FfzBsvXzD9fkY2WQ not represented in vector store (name: climate finance access network)

Document Gvrfapf3xwftHMMZ not represented in vector store (name: climate finance support to address short-lived climate pollutant (slcp) emissions in the pacific alliance and west africa)

Document asMyDTzVOLLYV94k not represented in vector store (name: Energy Transition Program)

Document awpyR3n5mUyiQX8e not represented in vector store (name: net-zero challenge)

Document cU3N5G1KqrceLGLz not represented in vector store (name: Low Carbon Economy Fund - Indigenous Leadership Fund)

Document g2dTE7dek1zuS90I not represented in vector store (name: green shipping corridor program)

Document vDopz6hZTpPeboRn not represented in vector store (name: Incentives for Zero-Emission Vehicles (iZEV) program)

Document w0J7Xs9DoY23oeCt not represented in vector store (name: Low Carbon Economy Fund - Low Carbon Economy Leadership Fund)

In [56]:
XUgWVRI3YcBdecj_text = '''Agricultural Clean Technology Program: Research and Innovation Stream: 1. What this program offers
1. What this program offers
2. Who is eligible
3. How to apply
4. After you apply
Applicant guide
Contact information
1. What this program offers
Intake period: Open
The application intake period for the program is open.

As part of the Government of Canada’s strengthened climate plan, A Healthy Environment and a Healthy Economy, the new Agricultural Clean Technology (ACT) Program aims to create an enabling environment for the development and adoption of clean technology that will help drive the changes required to achieve a low-carbon economy and promote sustainable growth in Canada’s agriculture and agri-food sector.

The ACT Program offers support under 2 streams:

Research and Innovation Stream
Adoption Stream
Research and Innovation Stream
The Research and Innovation Stream will support pre-market innovation, including research, development, demonstration and commercialization activities, to develop transformative clean technologies and enable the expansion of current technologies, in 3 priority areas:

Green energy and energy efficiency
Precision agriculture
Bioeconomy
Eligible activities under the Research and Innovation Stream are of the following nature and type:

Applied research and development of clean technologies
Piloting and evaluating clean technologies
Demonstration and knowledge and technology transfer activities
Commercializing and scaling up clean technologies
Other activities that support the Research and Innovation Stream as determined by the program
Support is available in the form of repayable and non-repayable contributions from Agriculture and Agri-Food Canada.

Accelerator
The Accelerator is a program under the Research and Innovation Stream that will provide funding to eligible not-for-profit organizations that will, in turn, use their specific networks and expertise to further distribute these funds to other organizations in support of their research and innovation objectives.'''

In [64]:
FfzBsvXzD9fkY2WQ_text = '''Initiative details
Support to the Climate Finance Access Network (CFAN)

Canada’s Total Climate Finance Contribution: $9,500,000 CAD 
Targeted Regions:

Oceania
Funding Period: 2020 to 2021

Financial Instrument: Grant

Type of Support: Cross-cutting

Delivery Channel:

Rocky Mountain Institute
Description
This project aims to support the most climate vulnerable developing countries, notably Pacific Small Island Developing States (SIDS), in building their capacity to structure and secure public and private finance for priority climate mitigation and adaptation investments. The project will draw from a global talent pool to place dedicated and independent climate finance advisors in low capacity vulnerable countries, serving as facilitators and links between recipient countries and the various funding sources these countries need to achieve their national climate plans and related targets under the Sustainable Development Goals (SDGs). Although SIDS make negligible contributions to global greenhouse gas emissions rates, they find themselves on the frontline in the fight against climate change and are the most vulnerable to its impacts.

For more information, please visit Global Affairs Canada's Project Browser.

Results/Expected Outcomes
The expected outcomes for this project include: (1) strengthened local capacity of both embedded climate finance advisors and key national stakeholders of CFAN countries; (2)improved efficiency and coordination of the implementation of global climate finance initiatives in developing countries, including Pacific SIDS; (3) expanded pipeline of climate adaptation and mitigation projects in developing countries, including Pacific SIDS; and (4) improved integration of gender considerations into climate finance project proposals in developing countries, including Pacific SIDS.'''

In [63]:
Gvrfapf3xwftHMMZ_text = '''Grants and Contributions:
Agreement Number:-
Agreement Value:$4,253,000.00
Agreement Date:Sep 27, 2019 -
Description:Climate Finance Support to Address Short-Lived Climate Pollutant (SLCP) Emissions in the Pacific Alliance and West Africa
Organization:Environment and Climate Change Canada
Location:Ottawa, Ontario, CA
Reference Number:099-2019-2020-02-00125
Agreement Type:Contribution
Report Type:Grants and Contributions
Additional Information:
Multi-year

Recipient's Legal Name:Novasphere'''

In [65]:
asMyDTzVOLLYV94k_text = '''Energy Transition Program

Canada’s Total Climate Finance Contribution: $275,000,000 CAD 
Targeted Regions:

Global
Funding Period: 2019 to 2020, 2020 to 2021

Financial Instrument: Concessional Loan

Type of Support: Mitigation

Delivery Channel:

IBRD Trust Funds - World Bank
Description
This project aims to slow the expansion of coal in the electricity sector by developing low-carbon alternatives (including solar, wind and geothermal) and improving energy efficiency and grid upgrades in targeted countries. It aligns and complements Canada’s leadership in the Powering Past Coal Alliance. This program’s financing is aimed at strengthening key institutional actors’ capacity to design regulatory incentives to boost energy efficiency and renewable energy markets. The program will also promote the closer of existing coal generation or no planning of new coal generation. It provides funding alongside credit financing from the International Bank for Reconstruction and Development or the International Development Association. Eligible countries include South Africa, Indonesia, Philippines, Vietnam, India and Pakistan. The program will promote initiatives to install renewable energy and energy efficiency as well as creating new renewable energy jobs.

For more information, please visit Global Affairs Canada's Project Browser.

Results/Expected Outcomes
The expected outcome for this project is increased renewable energy and improved energy efficiency in countries with high-levels of coal-generated electricity, including increased share of electricity supplied by renewable sources, installing new renewable energy capacity, and creating new renewable energy jobs. Canada's contribution to the program expects to leverage more than US$400 million in private and public investment and reduce greenhouse gas emissions by more than 22 million tonnes.'''

In [71]:
awpyR3n5mUyiQX8e_text = '''What is the Net-Zero Challenge
The Net-Zero Challenge encourages businesses to develop and implement credible and effective plans to transition their facilities and operations to net-zero emissions by 2050. The Government of Canada is committed to reducing Canada's emissions by 40-45% from 2005 levels by 2030 and putting Canada on a path to reach net-zero emissions by 2050. The Canadian Net-Zero Emissions Accountability Act establishes in law Canada's emissions reduction target of net-zero emissions by 2050. To achieve these commitments, the federal government has published an emissions reduction roadmap, the 2030 Emissions Reduction Plan.

Getting Canada to net-zero emissions by 2050 will require collaboration and innovation across all parts of society, including the private sector. Unlocking further emissions reductions and planning now for a net-zero future is essential for Canada to remain competitive in an increasingly low-carbon global economy.

The Net-Zero Challenge takes into account that net-zero planning is an ongoing process and will be different for each participant. It also seeks to align with existing reporting requirements and minimize duplication. The design of the Challenge was informed by consultations with interested partners and stakeholders.

The Challenge aims to build a community of practice around net-zero planning in Canada, where ideas and best practices can be shared, and where participants can collaborate to further their own net-zero plans.'''

In [67]:
cU3N5G1KqrceLGLz_text = '''Indigenous Leadership Fund
Important notice
Due to high demand and limited funding, the First Nations and Designated funding streams are currently closed and not accepting new applications. If you have already submitted an Expression of Interest or project proposal, you will be contacted directly with an update on the status of your application.

On this page
About the program
Funding streams
Eligible projects
Contact us
Related links
Additional resources
About the program
The Indigenous Leadership Fund is a program under the Low Carbon Economy Fund. It provides up to $180 million by 2029 to support climate action by Indigenous peoples. The program funds Indigenous-owned and led renewable energy, energy efficiency, and low-carbon heating projects. These projects will help meet Canada’s 2030 emissions reduction target and net-zero emissions by 2050. This is a key part of the 2030 Emissions Reduction Plan.

Recognizing the unique rights, interests, and circumstances of First Nations, Inuit, and Métis governments, communities, and organizations, the Indigenous Leadership Fund uses a distinctions-based approach to deliver funding to support projects that:

are Indigenous-owned and led
advance Indigenous climate leadership
support the self-determined climate, social, and economic priorities of Indigenous peoples
As announced in Budget 2022, up to $32.2 million will go to the Atlin Hydro Expansion project, which will provide clean electricity to the Yukon and help reduce greenhouse gas emissions.

Funding streams
Environment and Climate Change Canada is delivering the program using a distinctions-based approach and is working and communicating directly with Indigenous governments, communities, and organizations to deliver funding to support projects. The three distinctions-based funding streams are:

First Nations
Inuit
Métis
A designated funding stream for eligible applicants not already included in the funding streams for First Nations, Inuit, and Métis is also available.

First Nations
Important notice
Due to high demand and limited funding, the First Nations funding stream is currently closed and not accepting new applications. If you have already submitted an Expression of Interest or project proposal, you will be contacted directly with an update on the status of your application.

The First Nations funding stream is spending up to $73.9 million. It was developed in collaboration with First Nation representatives and Indigenous clean energy experts. The stream is intended for First Nations governments, communities, and organizations, including First Nations tribal or band councils, Modern Treaty and Self-Governing First Nations, First Nations-owned businesses and corporations, First Nations not-for-profit organizations, etc.

The call for proposals can fund a variety of project types and sizes, with a maximum ECCC contribution of $6 million per project. The stream supports First Nation-owned and First Nation-led renewable energy, energy efficiency, or low-carbon heating projects that provide benefits to First Nations. Eligible projects must be past the concept stage and ready for deployment or construction, meaning that project planning and pre-feasibility activities have been completed, other funding has been secured or is close to being secured, and partnership agreements, impact or environmental assessments, permitting requirements, and power purchase agreements have been finalized or are near finalization.

Proposals that include non-First Nation partners must provide transparent and comprehensive information on the ownership and governance structure of the project, including details on decision-making processes.

Inuit
This funding stream was developed in collaboration with the Inuit Tapiriit Kanatami and four Inuit land claim organizations. It is delivered through a directed and continuous intake process over six years, starting in 2023. Each Inuit land claim organization is developing a portfolio of projects to access the funding they have been allocated.

Funding under the Inuit stream is available for the following Inuit land claim organizations:

Nunavut Tunngavik Incorporated
The Makivvik Corporation of Nunavik
The Nunatsiavut Government
The Inuvialuit Regional Corporation
Métis
This funding stream was developed in collaboration with the Métis National Council and five Métis governments. It is delivered through a directed and continuous intake process over six years, starting in 2023. Each Métis government is developing a portfolio of projects to access the funding they have been allocated. Funding allocations have been determined using the Métis National Council’s funding formula.

Funding under the Métis stream is available for the following Métis governments:

Métis Nation of Ontario
Métis Nation-Saskatchewan
Métis Nation of Alberta
Métis Nation British Columbia
Manitoba Métis Federation
Designated stream
Important notice
Due to high demand and limited funding, the Designated funding stream is currently closed and not accepting new applications. If you have already submitted an Expression of Interest or project proposal, you will be contacted directly with an update on the status of your application.

Eligible applicants for the Designated funding stream are Indigenous governments, communities, and organizations not already included in the funding streams for First Nations, Inuit, and Métis. This can include Indigenous-owned businesses and corporations, Indigenous not-for-profit organizations, Métis Settlements, and Indigenous research, academic or educational institutions.

The Designated funding stream is spending up to $7.39 million to support Indigenous-owned and Indigenous-led renewable energy, energy efficiency, or low-carbon heating projects that provide broad benefits to Indigenous peoples. Eligible projects must be past the concept stage and ready for deployment or construction, meaning that project planning and pre-feasibility activities have been completed, other funding has been secured or is close to being secured, and partnership agreements, impact or environmental assessments, permitting requirements, and power purchase agreements have been finalized or are near finalization.

Proposals that include non-Indigenous partners must provide transparent and comprehensive information on the ownership and governance structure of the project, including details on decision-making processes.

Eligible projects
The Indigenous Leadership Fund supports Indigenous-owned and Indigenous-led projects that meet the following criteria:

Incorporate or make use of renewable energy, energy efficiency, or low-carbon heating technologies and practices
Offer broader environmental, social, and economic benefits
Use proven technologies and practices (proven technologies usually correspond with a level 8 or above on the Technology Readiness Level Scale)
Applicants are also encouraged to submit projects that:

help reduce GHG emissions in 2030
align with Canada’s goals for net-zero emissions by 2050
Contact us
If you have questions on the Indigenous Leadership Fund or would like to learn more about the program, please contact us by email at LeadershipAutochtoneFEFEC-IndigenousLeadershipLCEF@ec.gc.ca.

Related links
Low Carbon Economy Leadership Fund
Indigenous climate and environmental funding'''

In [68]:
g2dTE7dek1zuS90I_text = '''The Green Shipping Corridor Program
The Green Shipping Corridor Program (GSCP) provides funding for projects that contribute to the establishment of green shipping corridors and the decarbonization of the marine sector along the Great Lakes, the St. Lawrence Seaway as well as Canada’s East and West Coasts by:

removing barriers to the adoption of emission reducing equipment and infrastructure
incentivizing industry-led partnerships and investments to accelerate the adoption of greenhouse gas (GHG) emission-reduction technologies and infrastructure
de-risking investments to increase the technology-readiness level of low carbon and zero-emission ship technology and marine fuels for the domestic vessel fleet
building capacity among Canadian vessel owner/operators with respect to their ability to identify, plan and implement next generation low carbon and zero-emission ship technology and marine fuels into their vessel operations
GSCP allocates funding through two streams that have been designed to contribute to the achievement of program objectives by supporting projects focused on the implementation of clean technologies along the shipping value chain on shore and on vessels.

Clean Ports Stream
The Clean Ports Stream will provide $127.2M in contribution funding over four years between 2024-25 to 2027-28 for projects targeting onshore equipment and infrastructure investments at ports and terminals, (for example, commercially available shore power, electrification of port equipment, vessel charging and clean fuel infrastructure), and funding to ports to establish/augment incentive programs to help attract clean vessels.

Clean Vessel Demonstration Stream
The Clean Vessel Demonstration Stream will provide $22.5M in grant and contribution funding over five years from 2023-24 to 2027-28 for projects with the goal of increasing the technology readiness of leading-edge low- and zero-emission vessel technologies; addressing potential barriers to the deployment of clean vessel innovations in Canada; and building Canadian industry's capacity to adopt decarbonization solutions for their vessels.'''

In [69]:
vDopz6hZTpPeboRn_text = '''Incentives for Zero-Emission Vehicles (iZEV)
The iZEV Program has now ended. The information on this page is provided for reference purposes only.

iZEV

Transportation is the second largest source of greenhouse gas (GHG) emissions in Canada, accounting for a quarter of Canada's total GHG emissions with almost half of these emissions coming from cars and light trucks. One way to reduce the amount of transportation-related GHG emissions is to put more zero-emission vehicles (ZEVs) on the road.

A ZEV is a vehicle that has the potential to produce no tailpipe emissions. They can still have a conventional internal combustion engine but must also be able to operate without using it.

We consider the following vehicles to be ZEVs:

battery-electric
plug-in hybrid electric
fuel cell electric
Find more incentives and programs to help fight climate change
 
In order to encourage the adoption of ZEVs by Canadians and Canadian businesses, the Government of Canada launched the Incentives for Zero-Emission Vehicles (iZEV) Program in May 2019. There are many different makes and models of eligible zero-emission vehicles for purchase or lease (12 months or longer) that offer eligible consumers with an incentive of up to $5,000. For more information on the iZEV Program, please consult the “Program overview” section below.

 
Displaying 1 to 7 of 7 items.View all
 
1

Program overview
This page contains information for Canadian residents and businesses about how to make a qualifying purchase or lease under the iZEV Program.

1

Eligible vehicles
If you are looking to lease or purchase a new ZEV and would like to know whether it is eligible under the iZEV Program, you can find that information right here.

1

Questions and answers
If you are a dealership representative, an individual or an organization thinking about purchasing or leasing a ZEV that qualifies under the iZEV Program, this page contains answers to the most frequently asked questions about the iZEV Program.

1

Program resources for dealerships
This page contains all program resources for dealerships and their representatives. It includes hyperlinks to the iZEV Program’s mandatory forms as well as the ZEV application link for dealerships to manage existing enrolment and incentive requests.

1

Zero-emission vehicle charging stations
Canada already has thousands of ZEV charging and refuelling stations and the number of new locations continues to grow. Find out how Natural Resources Canada’s Zero Emission Vehicle Infrastructure Program is helping to make your ZEV journey hassle-free.

1

Canada's Zero-Emission vehicle sales targets
Canada is committed to decarbonizing the country's transportation sector and becoming a global leader in ZEVs. As such, the Government of Canada is aiming for 100% of new light-duty sales to be zero-emission by 2035. You can learn more about Canada’s ZEV sales targets here.

1

iZEV program statistics
On the iZEV Program statistics page, you can submit a request to receive all ZEV incentives data collected to date.'''

In [70]:
w0J7Xs9DoY23oeCt_text = '''Low Carbon Economy Leadership Fund
The Low Carbon Economy Leadership Fund provides funding directly to provinces and territories. It helps them reduce carbon pollution and meet Canada’s 2030 climate target. This funding also supports our goal of net-zero emissions by 2050. Each province and territory focuses on the best way to deliver the funding available to reduce emissions given their unique climates and circumstances.

Overview
Each province and territory is allocated a funding envelope under the Low Carbon Economy Leadership Fund to support climate action priorities.

Eligible activity categories to reduce greenhouse gas (GHG) emissions and promote sustainability include energy efficiency, fuel switching and non-energy related actions.

Examples of eligible activity types include:

equipment retrofits for energy efficiency and fuel switching
organics diversion
industrial process emissions changes
industrial product use changes
These activities contribute to sustainable practices and emissions reductions across sectors.

Related links
Low Carbon Economy Challenge
Indigenous Leadership Fund
Implementation Readiness Fund
Low Carbon Economy Fund: visual image guidelines'''

In [72]:
heal_row(results_df, "2XUgWVRI3YcBdecj", XUgWVRI3YcBdecj_text)
heal_row(results_df, "FfzBsvXzD9fkY2WQ", FfzBsvXzD9fkY2WQ_text)
heal_row(results_df, "g2dTE7dek1zuS90I", g2dTE7dek1zuS90I_text)
heal_row(results_df, "vDopz6hZTpPeboRn", vDopz6hZTpPeboRn_text)
heal_row(results_df, "w0J7Xs9DoY23oeCt", w0J7Xs9DoY23oeCt_text)
heal_row(results_df, "asMyDTzVOLLYV94k", asMyDTzVOLLYV94k_text)
heal_row(results_df, "awpyR3n5mUyiQX8e", awpyR3n5mUyiQX8e_text)
heal_row(results_df, "Gvrfapf3xwftHMMZ", Gvrfapf3xwftHMMZ_text)
heal_row(results_df, "cU3N5G1KqrceLGLz", cU3N5G1KqrceLGLz_text)

## Saving all of these scraped files

In [73]:
import os

output_dir = "data/scraped_documents"
os.makedirs(output_dir, exist_ok=True)

# Group by matched_name and save each group to its own file as .txt
for name, group in results_df.groupby("matched_unique_id"):
    # Make a safe filename based on the matched_name
    safe_name = "".join([c if c.isalnum() or c in " ._-" else "_" for c in name.strip()])
    filename = os.path.join(output_dir, f"{safe_name}.txt")
    
    # Combine the text from all 'content' fields for this group into a single string, separated by 2 newlines
    all_text = "\n\n".join((str(x) if x is not None else "") for x in group['content'])
    
    # Write to txt file
    with open(filename, "w", encoding="utf-8") as f:
        f.write(all_text)


In [74]:
os.makedirs("data", exist_ok=True)
mapping = df[["unique_id", "name"]].drop_duplicates()
mapping.to_csv("data/unique_id_to_name.csv", index=False)


## Some exploration

In [75]:
# Print the 5 document names (matched_name) whose combined character count is smallest,
# then show all their rows (with URLs and char count)
combined_char_counts = results_df.groupby("matched_name")["char_count"].sum().nsmallest(5)
smallest_names_by_char = combined_char_counts.index.tolist()
smallest_docs_by_char = results_df[results_df["matched_name"].isin(smallest_names_by_char)]
print(smallest_docs_by_char[["matched_name", "url", "char_count"]])


KeyError: 'matched_name'